# Import

In [1]:
#path_essentials = '/Users/Setup3-PC/Desktop/qudi/logic/setup3_logic/script_logic_essentials.py'
path_essentials = 'user_scripts/Timo/from_Simon/script_logic_essentials.py'
path_toolkit = 'user_scripts/Timo/own/logic_toolkit.py'
path_curdir = 'user_scripts/Timo/own/'
import os, time, sys
import numpy as np
import copy as cp
import lmfit

from core.util.helpers import csv_2_list
from logic.pulsed.predefined_generate_methods.pentacene_methods import DeerAltModes
from logic.pulsed.predefined_generate_methods.multi_nv_methods import DQTAltModes, TomoRotations, TomoInit
from logic.pulsed.sampling_functions import DDMethods
#from logic.pulsed.sampling_function_defs.sampling_functions_nvision import EnvelopeMethods as Evm
from logic.pulsed.pulse_objects import PulseEnvelopeType as Evm
from user_scripts.Timo.own.console_toolkit import Tk_file, Tk_string

# import essentials via run
exec(open(path_essentials).read())
exec(open(path_toolkit).read())
# usual module imports
sys.path.append(path_curdir)


import logging
logging.basicConfig(filename='logfile.log', filemode='w', level=logging.DEBUG)
logger = logging.getLogger(__name__)

# configure this notebook
config_matplotlib_jupyter()

memory_dict=OrderedDict()
print_welcome_msg()
0

0

[Tue Apr 18 11:53:59 2023] Hi, jupyter ready. working dir: C:\Software\qudi


In [3]:
Evm.parabola

PulseEnvelopeType(parabola))

In [4]:
setup['gated']

True

# Common parameters

In [5]:
Evm.sin_n

PulseEnvelopeType(sin_n))

In [31]:
class Common(object):
    def __init__(self):
        """
        A | B | C | D
        --------------
        | 2838.1 | 2800  | 2772  | 2733 MHz
        | 91     | 216   | 117   | 82.5 ns  (@ mw_amp= 0.15 V)
        """
        
        
        # MW parameters
        # rabi _37: C: 138.0ns, D: 162.8ns
        #self.rabi_period_rect = 116.4e-9 #allOrientations~ 82.69e-9, shaped: 
       #self.rabi_period_rect = 53.0e-9#712.06e-9 NV1 ms=-1 30 Gauss
        #self.rabi_period_rect = 46.9e-9#88.9e-9#712.06e-9 NV1 ms=1 30 Gauss
        self.rabi_period_rect = 46.17e-9#88.9e-9#712.06e-9 NV1 ms=1 30 Gauss
        self.rabi_period_rect = 71.0e-9
        #self.rabi_period_rect = 809.0e-9
        #self.rabi_period_rect = 37.3e-9
        #self.rabi_period_rect = 109.1e-9#712.06e-9 NV1 ms=1 100 Gauss
        #self.rabi_period_rect = 98.6e-9#712.06e-9 NV1 ms=-1 100 gauss
        #self.rabi_period_rect = 1450.0e-9 #NV2 ms=-1
        #self.rabi_period_rect =  90.7e-9# NV2 ms=1
        #self.rabi_period_rect = 39.03e-9 
        #self.rabi_period_rect = 61.22e-9#22.40e-9
        #34.3e-9 #12.75e-9#57.2e-9#94.5e-9 # 87.0e-9 #118.1e-9 #M2 149.1e-9#760e-9#245.5e-9 #159.5e-9
        #self.rabi_shaped = {'shaped_on':True, 'period': 83.4e-9, 'env_type': Evm.parabola, 'order_P':1}
        #self.rabi_shaped = {'shaped_on':True, 'period': 78.9e-9, 'env_type': Evm.parabola, 'order_P':1} #  3.5*0.25/10 # max= 0.25
        self.rabi_shaped = {'shaped_on':False, 'period': 173.9e-9, 'env_type': Evm.parabola, 'order_P':1} #  3.5*0.25/10 # max= 0.25
        #self.rabi_shaped = {'shaped_on':True, 'period':125e-9, 'env_type': Evm.parabola, 'order_P':1} #  NV1 ms=1 30 Gauss
        #self.rabi_shaped = {'shaped_on':True, 'period': 90e-9, 'env_type': Evm.parabola, 'order_P':1} #  NV1 ms=-1 30 Gauss
        self.rabi_shaped = {'shaped_on':True, 'period': 59.9e-9, 'env_type': Evm.sin_n, 'order_n':2} #  NV1 ms=-1 100 Gauss, same Rabis
        #self.rabi_shaped = {'shaped_on':True, 'period': 81.2e-9, 'env_type': Evm.sin_n, 'order_n':2}
        #self.rabi_shaped = {'shaped_on':True, 'period': 109e-9, 'env_type': Evm.parabola, 'order_P':1} #  NV1 ms=1 100 Gauss
        self.rabi_shaped = {'shaped_on':True, 'period': 69.8e-9, 'env_type': Evm.sin_n, 'order_n':2} #  NV1 ms=-1 100 Gauss, same Rabis
        self.rabi_shaped = {'shaped_on':True, 'period': 73.5e-9, 'env_type': Evm.sin_n, 'order_n':2} #  NV1 ms=
        #self.rabi_shaped = {'shaped_on':True, 'period': 90e-9, 'env_type': Evm.parabola, 'order_P':1} #  NV1 ms=-1 100 Gauss
        #self.rabi_shaped = {'shaped_on':True, 'period': 41.1e-9, 'env_type': Evm.sin_n, 'order_n':1} #  NV1 ms=-1 100 Gauss, 250 mV
        #self.rabi_shaped = {'shaped_on':True, 'period': 51.7e-9, 'env_type': Evm.sin_n, 'order_n':2} #  NV1 ms=-1 100 Gauss
        #self.rabi_shaped = {'shaped_on':True, 'period': 63.8e-9, 'env_type': Evm.sin_n, 'order_n':2} #  NV1 ms=-1 100 Gauss, same Rabis
        #self.rabi_shaped = {'shaped_on':True, 'period': 86.4e-9, 'env_type': Evm.sin_n, 'order_n':1} #  NV1 ms=-1 100 Gauss, parallel
        #self.rabi_shaped = {'shaped_on':True, 'period': 107.2e-9, 'env_type': Evm.sin_n, 'order_n':1} #  NV1 ms=-1 100 Gauss
        #self.rabi_shaped = {'shaped_on':False, 'period': self.rabi_period_rect, 'env_type': Evm.parabola, 'order_P':1000} 
        
        #self.rabi_shaped = {'shaped_on':True, 'period': 58.0e-9, 'env_type': Evm.parabola, 'order_P':6}
        #self.mw_freq = 2729.7e6 #2.780e9 # B:2832.1e6 #3086.1e6 #3026.7e6 #3087.7e6 
        #self.mw_freq = 2754.7e6 #NV1 ms=-1 30 Gauss
        #self.mw_freq =  2979.45e6 #NV1 ms=1 30 Gauss perfect from pODMR
        #self.mw_freq =  2981.8e6 #NV1 ms=1 30 Gauss not perfect
        #self.mw_freq =  2616.1e6#NV1 ms=-1 100Gauss non perfect
        self.mw_freq = 3129.4e6 #NV1 ms=1 100Gauss non perfect
        #self.mw_freq = 2744.3e6
        #self.mw_freq = 2936.8e6 #NV2 ms=1
        #self.mw_freq= 2804.4e6 #NV2 ms=-1
        #self.mw_freq = 3129.2e6
        self.mw_amp = 0.09 #0.18#0.65*0.125#0.076#0.07# 0.250 # max= 0.25
        #self.mw_amp = 0.007
        self.mw_amp = 1/1.08*0.25
        #self.mw_amp = 0.8*0.25  # same Rabi freq
        #self.mw_amp = 0.25 / 2.2  # parallel
        self.p_scale = 1
        #self.rabi_period_rect = 134.0e-9 #allOrientations~ 82.69e-9, shaped: 
        #34.3e-9 #12.75e-9#57.2e-9#94.5e-9 # 87.0e-9 #118.1e-9 #M2 149.1e-9#760e-9#245.5e-9 #159.5e-9
        #self.rabi_shaped = {'shaped_on':False, 'period': 285.5e-9, 'env_type': Evm.parabola, 'order_P':1}
        #self.mw_freq = 2630.2e6 #3086.1e6 #3026.7e6 #3087.7e6 
        #nv1214.1: 2779.7 #nv1209.2: 2778.5e6 #nv1215.6:  2729.5e6#2784.1e6 #2732.0e6 # 2774.0e6# 2786.2e6 #2775.0e6#827.64e6 #M2 1503.592e6 #M15, 1507.455e6  #1507.747#7e6  #1503.336e6  #2656.5e6#NV M3 1.497129e9 #NV M2 1.503316e9 
        #nv_sA_1221.1: 
        #self.mw_freq = 2313.90e6 # only right HFS line
        #self.mw_amp = 3*0.25/10 # max= 0.25
        #self.mw_amp = 0.93*0.3584*0.25 # max= 0.25

        # Readout
        # RELOAD logic_essentials and upload new seq when changing!
        self.t_laser_pulsed = 3e-6   # for pulsed mes (non NI acq)
        #self.t_laser_pulsed = 10e-6   
        #self.t_laser_pulsed = 1.25e-6   # for pulsed mes (non NI acq)
        self.t_wait_pulsed = 1e-6
        self.t_laser = None
        self.t_laser_green_init = 3e-6
        self.t_laser_red_read = 1000e-9

        # Ramsey
        self.mw_ramsey_offset = 0e3   # Hz, detuning, 
        self.mw_ramsey_off_from_calib = 0e3 #0.1e6 #[0.66e6, 0.72e6]   # Hz, detuning,taken for checking valid

        # MFL XY8
        self.gamma_n_c13 = 1.0705e3 * 2*np.pi # 13-C
        
        # multi NV
        self.nv_order = [1,2]
    
    @property
    def rabi_period(self):

        if self.rabi_shaped['shaped_on']:
            return self.p_scale*self.rabi_shaped['period']
        return self.p_scale*self.rabi_period_rect
        
    @property
    def rabi_period_2(self):
        if self.rabi_shaped['shaped_on']:
            return list(self.p_scale*np.asarray(self.t_rabi_2_shaped))
        return list(self.p_scale*np.asarray(self.t_rabi_2_rect))
            
            
    @staticmethod
    def f_res_2_B_gauss(f_res):
        # only true if B well aligned
        return abs(f_res-2.87e9)/2.8e6
    
    def swap_nvs(self, order=[2,1]):
        
        if order == self.nv_order:
            return self
        if order == list(reversed(self.nv_order)):
            new_common = cp.deepcopy(self)

            new_common.nv_order = order
            new_common.rabi_period_rect = self.t_rabi_2_rect[0]
            new_common.t_rabi_2_rect = [self.rabi_period_rect]

            new_common.mw_freq = self.mw_freq_2[0]
            new_common.mw_freq_2 = [self.mw_freq]

            new_common.mw_amp = self.mw_amps_2[0]
            new_common.mw_amps_2 = [self.mw_amp]

            new_common.rabi_shaped['period'] = self.t_rabi_2_shaped[0]
            new_common.t_rabi_2_shaped = [self.rabi_shaped['period']]
            
        else:
            raise ValueError()
            
        return new_common 
        
    

    
common = Common()
b_str = f"B= {Common.f_res_2_B_gauss(common.mw_freq):.2f} G from common.mw_freq"
b_str
logger.info(b_str)

"""
# MW parameters
common.rabi_period_rect = 12.83e-9#57.2e-9#94.5e-9 # 87.0e-9 #118.1e-9 #M2 149.1e-9#760e-9#245.5e-9 #159.5e-9
common.rabi_shaped = {'shaped_on':False, 'period': 18.55-9, 'env_type': Evm.parabola, 'order_P':1}
common.mw_freq =  1507.23e6#2784.1e6 #2732.0e6 # 2774.0e6# 2786.2e6 #2775.0e6#827.64e6 #M2 1503.592e6 #M15, 1507.455e6  #1507.747#7e6  #1503.336e6  #2656.5e6#NV M3 1.497129e9 #NV M2 1.503316e9 
#common.mw_freq = 2811.9e6 # only left HFS line
common.mw_amp = 0.25

# Readout
# RELOAD logic_essentials and upload new seq when changing!
common.t_laser_pulsed = 3e-6   # for pulsed mes (non NI acq)
common.t_wait_pulsed = 1e-6
common.t_laser = None
common.t_laser_green_init = 3e-6
common.t_laser_red_read = 1000e-9

# Ramsey
common.mw_ramsey_offset = 1000e3   # Hz, detuning, 
common.mw_ramsey_off_from_calib = 0e3 #0.1e6 #[0.66e6, 0.72e6]   # Hz, detuning,taken for checking valid


# MFL XY8
gamma_n_c13 = 1.0705e3 * 2*np.pi # 13-C
"""

common.rabi_shaped
common.__dict__

{'rabi_period_rect': 7.1e-08, 'rabi_shaped': {'shaped_on': True, 'period': 7.55e-08, 'env_type': PulseEnvelopeType(sin_n)), 'order_n': 2}, 'mw_freq': 3129400000.0, 'mw_amp': 0.23148148148148145, 'p_scale': 1, 't_laser_pulsed': 3e-06, 't_wait_pulsed': 1e-06, 't_laser': None, 't_laser_green_init': 3e-06, 't_laser_red_read': 1e-06, 'mw_ramsey_offset': 0.0, 'mw_ramsey_off_from_calib': 0.0, 'gamma_n_c13': 6726.149871335747, 'nv_order': [1, 2]}

## for 2x NV experiments

In [34]:
#common.mw_freq_2 = [2805.04e6]#NV2 ms=-1 30 Gauss perfect from pODMR
#common.mw_freq_2 = [2808.3e6]#NV2 ms=-1 30 Gauss non perfect

#common.mw_freq_2 =[2936.3e6] #NV2 ms=1 30 Gauss
common.mw_freq_2 = [2744.3e6]#NV2 ms=-1 100 Gauss non perfect
#common.mw_freq_2 = [3042.7e6]#NV2 ms=1 100 Gauss non perfect
#common.mw_freq_2 = [3064.3e6] #[2717.91e6]#[2608.73e6]#[2628.56e6]   # old data A axis
#common.mw_amps_2 = [0.84*0.20]#[0.125]#[0.25] #7
#common.mw_amps_2 = [0.18]#[0.125]#[0.9*0.25]
common.mw_amps_2 = [0.099] # NV2 ms=-1 100 Gauss
#common.mw_amps_2 = [0.0076] # NV2 ms=-1 100 Gauss
common.mw_amps_2 = [0.25] # NV2 ms=-1 100 Gauss
common.mw_amps_2 = [1/1.08*0.25] # NV2 ms=-1 100 Gauss
#common.mw_amps_2 = [0.25/2] # NV2 ms=-1 100 Gauss, parallel
#common.t_rabi_2_rect = [46.08e-9] #NV2 ms=-1 30 Gauss
common.t_rabi_2_rect = [46.17e-9] #NV2 ms=-1 30 Gauss
#common.t_rabi_2_rect = [151e-9]# NV2 ms=1 30 Gauss
common.t_rabi_2_rect = [79.4e-9] #NV2 ms=-1 100 Gauss
#common.t_rabi_2_rect = [41.95e-9] #NV2 ms=-1 100 Gauss
#common.t_rabi_2_rect = [151e-9]# NV2 ms=1 100 Gauss
#common.t_rabi_2_rect =[61.1e-9] #[68.7e-9]#[49.9e-9]
#common.mw_freq_2 = [2825.3e6]
#common.mw_amps_2 = [7/10*0.25] #7
#common.mw_amps_2 = [0.98*0.7648*0.25] #[0.25]
#common.t_rabi_2_rect = [110.1e-9]
#common.t_rabi_2_rect = [61.4e-9] #[47.8e-9]
#common.t_rabi_2_shaped = [165.0e-9]  ##[88.1e-9]  # p=1: [118.8e-9]
#common.t_rabi_2_shaped = common.t_rabi_2_rect # debug only
common.t_rabi_2_shaped = [52.2e-9]  ##[88.1e-9]  # NV2 ms=-1 30 Gauss
common.t_rabi_2_shaped = [59.3e-9]  ##[88.1e-9]  # NV2 ms=-1 30 Gauss
common.t_rabi_2_shaped = [62.9e-9]  ##[88.1e-9]  # NV2 ms=-1 30 Gauss
#common.t_rabi_2_shaped = [90.6e-9]  ##[88.1e-9]  # NV2 ms=-1 30 Gauss
#common.t_rabi_2_shaped = [94.8e-9]  ##[88.1e-9]  # NV2 ms=1 30 Gauss
#common.t_rabi_2_shaped = [97.6e-9]  ##[88.1e-9]  # NV2 ms=-1 30 Gauss, parallel
#common.t_rabi_2_shaped = [110.07e-9]  ##[88.1e-9]  # NV2 ms=-1 30 Gauss
#common.t_rabi_2_shaped = [165.0e-9]  ##[88.1e-9]  # NV2 ms=1 30 Gauss
#common.t_rabi_2_shaped = [109.6e-9]  ##[88.1e-9]  # NV2 ms=-1 100 Gauss
#common.t_rabi_2_shaped = [165.0e-9]  ##[88.1e-9]  # NV2 ms=1 100 Gauss

common.dd_tau = 1.5e-6
common.dd_type = DDMethods.XY8
#common.dd_type = DDMethods.CPMG
#common.dd_type = DDMethods.SE
common.dd_order = 1

common.dd_tau*common.dd_type.suborder*common.dd_order 
(common.mw_freq_2[0] - common.mw_freq)/1e6
common.__dict__

{'rabi_period_rect': 7.1e-08, 'rabi_shaped': {'shaped_on': True, 'period': 7.55e-08, 'env_type': PulseEnvelopeType(sin_n)), 'order_n': 2}, 'mw_freq': 3129400000.0, 'mw_amp': 0.23148148148148145, 'p_scale': 1, 't_laser_pulsed': 3e-06, 't_wait_pulsed': 1e-06, 't_laser': None, 't_laser_green_init': 3e-06, 't_laser_red_read': 1e-06, 'mw_ramsey_offset': 0.0, 'mw_ramsey_off_from_calib': 0.0, 'gamma_n_c13': 6726.149871335747, 'nv_order': [1, 2], 'mw_freq_2': [2744300000.0], 'mw_amps_2': [0.23148148148148145], 't_rabi_2_rect': [7.94e-08], 't_rabi_2_shaped': [6.29e-08], 'dd_tau': 1.5e-06, 'dd_type': <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>, 'dd_order': 1}

In [4]:
common.swap_nvs([2,1]).__dict__
#common.__dict__

{'rabi_period_rect': 7.94e-08, 'rabi_shaped': {'shaped_on': True, 'period': 6.29e-08, 'env_type': PulseEnvelopeType(sin_n)), 'order_n': 2}, 'mw_freq': 2744400000.0, 'mw_amp': 0.23148148148148145, 'p_scale': 1, 't_laser_pulsed': 3e-06, 't_wait_pulsed': 1e-06, 't_laser': None, 't_laser_green_init': 3e-06, 't_laser_red_read': 1e-06, 'mw_ramsey_offset': 0.0, 'mw_ramsey_off_from_calib': 0.0, 'gamma_n_c13': 6726.149871335747, 'nv_order': [2, 1], 'mw_freq_2': [3129100000.0], 'mw_amps_2': [0.23148148148148145], 't_rabi_2_rect': [7.1e-08], 't_rabi_2_shaped': [7.55e-08], 'dd_tau': 1.5e-06, 'dd_type': <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>, 'dd_order': 1}

In [5]:
common.mw_amp + 0.6*common.mw_amps_2[0], 0.6*common.mw_amp + common.mw_amps_2[0]


(0.37037037037037035, 0.37037037037037035)

In [6]:
[el.name for el in TomoRotations]

['none', 'ux90_on_1', 'ux90_on_2', 'uy90_on_1', 'uy90_on_2', 'ux45_on_2', 'ux45min_on_2', 'ux90min_on_1', 'ux90min_on_2', 'uy90min_on_1', 'uy90min_on_2', 'ux180_on_1', 'ux180_on_2', 'uy180_on_1', 'uy180_on_2', 'ux180min_on_1', 'ux180min_on_2', 'uy180min_on_1', 'uy180min_on_2', 'c1not2', 'c2not1', 'c1not2_ux180_on_2', 'c2not1_ux180_on_1', 'c2phase1_dd']

# Library

## nb specific

In [7]:
# According to some problems with github. This is the new part of Timo's script
gate_translation = np.array([(TomoRotations.ux45min_on_2, 'rx', -np.pi/4, '1'),
                             (TomoRotations.ux90_on_1, 'rx', np.pi/2, '0'),
                             (TomoRotations.ux90_on_2, 'rx', np.pi/2, '1'),
                             (TomoRotations.uy90_on_1, 'ry', np.pi/2, '0'),
                             (TomoRotations.uy90_on_2, 'ry', np.pi/2, '1'),
                             (TomoRotations.ux90min_on_1, 'rx', -np.pi/2, '0'),
                             (TomoRotations.ux90min_on_2, 'rx', -np.pi/2, '1'),
                             (TomoRotations.uy90min_on_1, 'ry', -np.pi/2, '0'),
                             (TomoRotations.uy90min_on_2, 'ry', -np.pi/2, '1'),
                             (TomoRotations.ux180_on_1, 'rx', np.pi, '0'),
                             (TomoRotations.ux180_on_2, 'rx', np.pi, '1'),
                             (TomoRotations.uy180_on_1, 'ry', np.pi, '0'),
                             (TomoRotations.uy180_on_2, 'ry', np.pi, '1'),
                             (TomoRotations.ux180min_on_1, 'rx', -np.pi, '0'),
                             (TomoRotations.ux180min_on_2, 'rx', -np.pi, '1'),
                             (TomoRotations.uy180min_on_1, 'ry', -np.pi, '0'),
                             (TomoRotations.uy180min_on_2, 'ry', -np.pi, '1'),
                             #(TomoRotations.none, 'cx', np.pi, '0, 1'), # DEBUG ONLY!
                             (TomoRotations.c2not1, 'cx', np.pi, '0, 1'),
                             (TomoRotations.c2phase1_dd, 'c2phase1', np.pi, '0, 1')],
                            #],
                             dtype=[('op_target', 'i2'),
                                    ('op_code', 'U10'), ('rotation', 'f4'), ('idx_qubit', 'U10')])


def setup_charge_read_params(t_cinit_green=500e-9, t_cread_red=50e-6, t_wait=1e-6,
                             laser_red_ch='d_ch3', add_gate_ch='',
                                 gate_channel=None):

    qmeas=OrderedDict()
    
    qmeas['experiment'] = 'laser_strob'
    qmeas['name'] = 'charge_read'
    qmeas['listfile'] = True 

    
        
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed
    
    # charge read
    qmeas['t_laser_init'] = t_cinit_green
    qmeas['t_laser_read'] = t_cread_red
    qmeas['t_wait_between'] = t_wait
    qmeas['t_aom_safety'] = 750e-9

    
    qmeas['laser_read_ch'] = laser_red_ch
    qmeas['add_gate_ch'] = add_gate_ch
    
   
    qmeas['measurement_time'] = 9999 
    qmeas['optimize_time'] = None
    qmeas['fit_experiment'] = None


    return qmeas


def setup_charge_read_fci_params(t_cinit_green=500e-9, t_cinit_red=10e-6,
                                 t_cread_red=50e-6, t_wait=1e-6, 
                                 laser_red_ch='d_ch3', add_gate_ch='', done_ch='d_ch1',
                                 gate_channel=None):

    qmeas=OrderedDict()
    
    qmeas['name'] = 'charge_read_fci'
    qmeas['experiment'] = 'charge_read_fci'
    qmeas['sequence_mode'] = True 
    qmeas['listfile'] = True # changed from true to false 

    
    # ATTENTION: changes setup gate ch!
    if gate_channel != None:
        setup['gate_channel'] = gate_channel
        # need to manually change fastcounter trigger ovoltages!
        setup['analog_trigger_voltage'] = 0.25
    
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed
    
    # charge init
    qmeas['t_cinit_green'] = t_cinit_green
    qmeas['t_cinit_red'] = t_cinit_red
    qmeas['t_wait_between'] = t_wait
    qmeas['t_aom_safety'] = 750e-9
    
    # charge read
    qmeas['t_cread_red'] = t_cread_red
    
    
    
    qmeas['laser_red_ch'] = laser_red_ch
    qmeas['add_gate_ch'] = add_gate_ch
    qmeas['done_ch'] = done_ch
    
   
    qmeas['measurement_time'] = 9999 
    qmeas['optimize_time'] = None
    qmeas['fit_experiment'] = None


    return qmeas



def setup_rand_benchmark_params(
                             rotations=[[TomoRotations.none]], x_max=None, x_min=None, n_rand_max=None, 
                             range_arr=None,
                             f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             tau_cnot=100e-9, dd_type_cnot=DDMethods.SE, dd_order=1,
                             tau_dd_fix=None, xticks_ncliff=False, simultaneous_rb=False,
                             alternating=True, charge_init=False):

    def _limit_n_rand(rand_max, circs, infos, n_cliffs):

        gates_new = []
        if rand_max:
            for idx, n in enumerate(n_cliffs):
                new_n = True
                if idx > 0:
                    new_n = (n_cliffs[idx-1] != n)
                if new_n:
                    n_added = 0

                if n_added < rand_max: 
                    n_added += 1
                    gates_new.append((n_cliffs[idx], circs[idx], infos[idx]))

            n_cliffs, circs, infos = list(zip(*gates_new))

        return list(circs), infos, n_cliffs
    
    qmeas=OrderedDict()

    is_multi_drive = False
    if len(f_mw_2) > 0:
        is_multi_drive = True
    
    # for awg8190 name must be < 32 chars incl ch extension
    qmeas['name'] = f'rand_benchmark'  
    qmeas['experiment'] = 'rand_benchmark'
    qmeas['savetag'] = f'rand_benchmark'  
    
    if charge_init:
        qmeas['name'] = 'rand_benchmark_fci'
        qmeas['experiment'] = 'rand_benchmark_fci'
        qmeas['sequence_mode'] = True
        qmeas = add_fci(qmeas)

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['microwave_frequency'] = common.mw_freq
    qmeas['rabi_period'] = common.rabi_period
    
    if rotations:
        if type(rotations) == list:
            qmeas['rotations'] = list_2_csv(rotations, line_delimiter=";")
            
        elif type(rotations) == str:  # rotations given in file
            fname = os.path.abspath(rotations)    
            qmeas['rotations_file'] = rotations   # not used by predefined method, but safed
            
            circs_qudi, gate_infos = rbjson_2_gates(fname, n_cliff_max=x_max,n_cliff_min=x_min,range_=range_arr)
            n_cliff = [info_dict['n_cliff'] for info_dict in gate_infos] 
            
            circs_qudi, gate_infos, n_cliff = _limit_n_rand(n_rand_max, circs_qudi, gate_infos, n_cliff)
            
            rotations = circs_qudi
            qmeas['rotations'] = list_2_csv(circs_qudi, line_delimiter=";")
            if xticks_ncliff:
                qmeas['xticks'] = list_2_csv(n_cliff)
        else:
            raise ValueError("Don't understand given rotations.")
            
    else:
        # leaving will default init to "None" rotation
        pass
    
    qmeas['name'] = qmeas['name'] + '_2q' if "c2" in qmeas['rotations'] else qmeas['name'] + '_1q'
    
    qmeas['init_state_kwargs'] = {}   # todo, needed for cnot/ent state
    #{'dd_type': DDMethods.CPMG}
    
    qmeas['cnot_kwargs'] = {'tau_dd_fix': tau_dd_fix}   
    qmeas['init_state_kwargs'] = {}
    
    
    qmeas['tau_cnot'] = tau_cnot
    qmeas['dd_type_cnot'] = dd_type_cnot
    qmeas['dd_order'] = dd_order
    qmeas['alternating'] = alternating
    
    if tau_cnot < 0 and tau_dd_fix == None:
        raise ValueError(f"Negative tau_cnot {tau_cnot} not allowed for cnot based on deer_dd_par!")
    
    if is_multi_drive:
        # currently drives 1,2 have same amplitude, respectively
        qmeas['f_mw_2'] = list_2_csv(f_mw_2)
        qmeas['rabi_period_mw_2'] = list_2_csv(rabi_period_mw_2)
        qmeas['ampl_mw_2'] = list_2_csv(ampl_mw_2)            
    
    
    qmeas['mirror_1q_pulses'] = simultaneous_rb
    if simultaneous_rb:
        nv_ids = _get_qd_gates_nv_ids(rotations)
        if len(nv_ids) != 1:
            raise ValueError(f"Found !=1 nv_ids: {nv_ids}. Simultaneous benchmarking works only for a 1q gate string'")
    
    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'exp'#'decay_exp'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5

    qmeas = add_shaped(qmeas)

    return qmeas


def gate_step_2_qudi(step):
    op_code = step[0][0]
    op_params = step[0][1]
    idx_qubit = list_2_csv(step[1])
    
    # currently supported gate set 
    gate_set = ['rx', 'ry', 'c2phase1']
    ignore_codes = ['measure', 'barrier'] 
    
    if op_code not in gate_set + ignore_codes:
        raise ValueError(f"Unsupported op_code: {op_code}")
    if op_code in ignore_codes:
        return TomoRotations.none
    try:
        if op_code == 'cx':
            rotation = np.pi
        if op_code == 'c2phase1':
            rotation = np.pi
        else:
            rotation = np.float32(op_params[0])

        gt = gate_translation
    
        idx_translate = np.where((gt['op_code']==op_code) & (gt['idx_qubit']==idx_qubit) & (gt['rotation']==rotation))
        rot = gate_translation[idx_translate][0][0]
    except IndexError:
        logger.error(f"op_code {op_code}, qubit {idx_qubit}, rot {rotation}")
        raise ValueError(f"Couldn't find given step in translation table: {step}")
    
    return TomoRotations(rot)

def _get_qd_gates_nv_ids(gates):
    """
    Nv indices for a given list (of list) of single rotations.
    """
    
    rots_flat = [item for sublist in gates for item in sublist]
    rots_flat_filtered_1q = filter(lambda rot: rot not in [TomoRotations.c2not1, TomoRotations.c1not2, TomoRotations.none], rots_flat)
    has_cnot = TomoRotations.c2not1 in rots_flat or TomoRotations.c1not2 in rots_flat

    on_ids = [rot.name.split('_on_')[1] for rot in cp.deepcopy(rots_flat_filtered_1q)]
    
    return np.unique(on_ids)


def rbjson_2_gates(fname, n_cliff_max=None,n_cliff_min=None,range_ = None):
    
    x_max = n_cliff_max
    x_min = n_cliff_min
    
    fname = os.path.abspath(fname)
    with open(fname) as f:
        qudi_gates = json.load(f)

    circs = parse_rbjson(qudi_gates)
    circs_qudi, infos =  [], []
    for circ in circs:
        idx_info = circ['header_idx']
        skip_circ = False
        if (x_max is not None) and (x_min is not None):
            skip_circ = (idx_info['n_cliff'] > x_max) or (idx_info['n_cliff'] < x_min) 
        elif (x_max is not None) and (x_min is None):
            skip_circ = (idx_info['n_cliff'] > x_max)
        elif (x_max is None) and (x_min is not None):
            skip_circ = (idx_info['n_cliff'] < x_min)
        elif (range_ is not None):
            skip_circ = (idx_info['n_cliff'] not in range_) 
        if skip_circ:
            continue

        gates = circ['gate_steps']
        rots_qudi = []
        for step in gates:
            try:
                rots_qudi.append(gate_step_2_qudi(step))
            except:
                logger.exception(f"In step: {idx_info}")
                raise ValueError
            
        circs_qudi.append(rots_qudi)
        infos.append({'n_cliff': idx_info['n_cliff']})
        
    return circs_qudi, infos
    
def parse_rbjson(data):
    # data is a list of list
    # idx 0: specify n_cliffors
    # idx 1: specify n_random
    # idx 2: info level
    # idx 3: sequence steps
    
    out_list = []
    
    for idx_cl, circ_list in enumerate(data):
        for idx_r, _ in enumerate(circ_list):
            
            out_dict = {}
            out_dict['header_raw'] = data[idx_cl][idx_r][0]
            header_str = out_dict['header_raw'].split('\n')[1].split("#")
            out_dict['header_metainfo'] = json.loads(header_str[2])
            out_dict['header_idx'] = (json.loads(header_str[1]))['idx_info']
            
            gates = data[idx_cl][idx_r][1]
            out_gates = []
            for gate in gates:
                out_gates.append(gate)
            out_dict['gate_steps'] = out_gates
            out_list.append(out_dict)
            
    return out_list



def add_fci(qmeas=None, enable=False):
    
    if qmeas is None:
        qmeas = {}
        qmeas['experiment'] = "n/a"
    
    # sequence that support (fast) charge init have '_fci' in their name
    if '_fci' in qmeas['experiment'] or enable:
        awg8190._dynamic_sequence_mode = False
        awg8190.write(':TRIG:SOUR:ADV TRIG')
        awg8190.write(':ARM:TRIG:IMP HIGH')
        awg8190.write(':ARM:TRIG:LEV 1.5')
        
        # fci sequences must have those parameters
        if qmeas['experiment']:
            # todo: strip old generate_methods that make use of kwargs
            qmeas['t_cinit_green'] = common.t_cinit_green 
            qmeas['t_cinit_red'] = common.t_cinit_red 
            qmeas['t_cinit_wait_between'] = common.t_cinit_wait_between 
            qmeas['t_aom_safety'] = common.t_aom_safety 
            
            qmeas['laser_read_ch'] = 'd_ch3'
            qmeas['add_gate_ch'] = 'd_ch4'
            qmeas['done_ch'] = 'd_ch1'
            
            # new: setting via generation parameters
            qmeas['t_laser_fci_green'] = common.t_cinit_green 
            qmeas['t_laser_fci_red'] = common.t_cinit_red 
            qmeas['t_wait_fci'] = common.t_cinit_wait_between 
            qmeas['t_safety_fci'] = common.t_aom_safety 
            
            qmeas['laser_read_red_ch'] = 'd_ch3'
            qmeas['done_fci_ch'] = 'd_ch1'
            qmeas['add_gate_ch'] = 'd_ch4'
            
    return qmeas
#--------------------------------------------------------------------------------------------------------------------
def setup_podmr_rfpi_params(f_res, p_scale=10, f_step=0.1e6, n_f=50,
                            f_rf=1e6, t_pi_rf=1e-6, ampl_rf=5e-3, alternating_no_rf=False):

    qmeas=OrderedDict()

    if common.rabi_shaped['shaped_on']:
        raise ValueError
    else:
        qmeas['name'] = 'pulsedodmr_rf_pi'
        qmeas['experiment'] = 'pulsedodmr_deer_pi'

    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed


    qmeas['rabi_period'] = common.rabi_period*p_scale
    qmeas['freq_step'] = f_step
    qmeas['num_of_points'] = n_f
    qmeas['freq_start'] = f_res - (qmeas['freq_step'] * qmeas['num_of_points'])/2.
    qmeas['f_mw_deer'] = f_rf
    qmeas['t_pi_deer'] = t_pi_rf
    qmeas['deer_ampl'] = ampl_rf 
    qmeas['alternating_nodeerpi'] = alternating_no_rf
    
    if p_scale != 1:
        qmeas['microwave_amplitude'] = common.mw_amp/(p_scale)
    else:
        qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['measurement_time'] = 9999 
    qmeas['optimize_time'] = None
    qmeas['fit_experiment'] = 'lor_dip'

    qmeas = add_shaped(qmeas)

    return qmeas


def add_shaped(qmeas):
    if common.rabi_shaped['shaped_on']:
        # old, nvision way by passing to generate_method param
        #qmeas['env_type'] = common.rabi_shaped['env_type']
        #qmeas['order_P'] = common.rabi_shaped['order_P']
        if common.rabi_shaped['shaped_on']:
            evm = Evm(common.rabi_shaped['env_type'])
            if evm == Evm.parabola:
                evm.parameters['order_P'] = common.rabi_shaped['order_P']
            if evm == Evm.sin_n:
                evm.parameters['order_n'] = common.rabi_shaped['order_n']
            qmeas['pulse_envelope'] = evm
        
    else:
        # to make sure that default of generate method is overwritten
        qmeas['pulse_envelope'] = Evm.rectangle
        
    return qmeas
    
    

def calc_nuclear_rabi_hfs_boost(B_gauss, ms=-1, n_nuc=15):
    # Chen (2015)
    # Sangtawesin (2016)
    
    if n_nuc == 14:
        q_quadro = -4.945e6
        a_par = -2.162e6 # 14-n
        a_perp = 2.62e6
        gamma_n = -0.308e3  
    elif n_nuc == 15:
        q_quadro = 0  # spin=1/2 has no quadropolar moment
        a_par = 3.03e6
        a_perp = 3.65e6
        gamma_n = 0.432e3
        
    else:
        raise ValueError("Can only be 14-N or 15-N")
        
    gamma_e = 2.8e6
    delta = 2.87e9
    
    if ms == -1:
        # in nuc_rabi need pi init of the nv spin
        alpha = 1 + gamma_e/gamma_n* a_perp / (delta-gamma_e*B_gauss-a_par+gamma_n*B_gauss+q_quadro)
    else:
        raise ValueError("m_s can only be 0/1/-1")
    
    return abs(alpha)

def setup_ise_ramsey_dd_params(tau, n_tau, n_order_dd, f_rf, t_pi_rf, amp_rf=0.25, rf_ch='a_ch2', dd_type=DDMethods.SE,
                               t_laser=1e-6, mw_sweep_speed=3e12, f_ise_res=1e9, df_mw_sweep=30e6, amp_mw_sweep=0.25,
                               alternating=True, autosave=-1, red_readout=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'ise+ramsey+rf_dd'
    qmeas['experiment'] = 'pol_ramsey_rf_dd'
    
    if not red_readout:
        setup['laser_length'] = common.t_laser_pulsed
    else:
        qmeas['t_laser_init'] = common.t_laser_green_init
        qmeas['laser_read_ch'] = 'd_ch3'
        setup['laser_length'] = common.t_laser_red_read
    
    setup['wait_time'] = common.t_wait_pulsed
    
    qmeas['microwave_frequency'] = common.mw_freq + common.mw_ramsey_offset 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['tau'] = tau
    qmeas['n_tau'] = n_tau
    qmeas['n_order_pi_rf'] = n_order_dd
    qmeas['dd_type'] = dd_type
    qmeas['f_rf'] = f_rf
    qmeas['t_pi_rf'] = t_pi_rf
    qmeas['amp_rf'] = amp_rf
    qmeas['rf_channel'] = rf_ch
    qmeas['t_laser'] = t_laser
    qmeas['mw_sweep_speed'] = mw_sweep_speed
    qmeas['amp_mw_sweep'] = amp_mw_sweep
    qmeas['f_ise_res'] = f_ise_res
    qmeas['df_mw_sweep'] = df_mw_sweep
    
    qmeas['measurement_time'] = 99999 
    qmeas['optimize_time'] = None
    qmeas['autosave_s'] = autosave
    
    qmeas['alternating'] = alternating
    
    return qmeas

def setup_deer_dd_f_params(f_res, dd_tau=200e-9, f_step=2e6, n_f=50, t_rabi_2=None, deer_ampl=None,
                           dd_type=DDMethods.SE, dd_order=2):
    qmeas=OrderedDict()
    
    if not common.rabi_shaped['shaped_on']:
        logger.error("DEER_dd via nvision method must use pulse shaping")
        return 
    
    qmeas['experiment'] = 'DEER_DD_freq'
    qmeas['name'] = 'deer_dd_spectrum'
    
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed
    
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['freq_incr'] = f_step
    qmeas['num_of_points'] = n_f
    qmeas['freq_start'] = f_res - (f_step * qmeas['num_of_points'])/2.
    qmeas['rabi_period'] = common.rabi_period
    
    qmeas['measurement_time'] = 150000 
    qmeas['optimize_time'] = None
    
    qmeas['alternating'] = True

    qmeas['tau'] = dd_tau
    if not t_rabi_2:
        qmeas['rabi_period2'] = common.rabi_period    
    else:
        qmeas['rabi_period2'] = t_rabi_2
    if deer_ampl == None:
        qmeas['mw_amp2'] = common.mw_amp
    else:
        qmeas['mw_amp2'] = deer_ampl
    
    # 1 ns safety between pi pulses
    #qmeas['tau2'] = dd_tau/2 - qmeas['rabi_period2']/4 - common.rabi_period/4 - 1e-9
    qmeas['tau2_rel_to_pi1'] = True
    qmeas['tau2'] = 0e-9
    
    qmeas['DD_type'] = dd_type
    qmeas['order'] = dd_order
    
    qmeas = add_shaped(qmeas)
    return qmeas


def setup_deer_dd_tau_params(f2, dd_tau=200e-9, tau2_start=0e-9, tau2_step=5e-9, n_tau=50,
                             t_rabi_2=None, deer_ampl=None,
                             dd_type=DDMethods.SE, dd_order=2, read_phase=0, charge_init=False):                    
    qmeas=OrderedDict()
    
    # legacny nvision deer methods
    #if common.rabi_shaped['shaped_on']:
    #    qmeas['experiment'] = 'DEER_DD_tau'
    #    qmeas['name'] = 'deer_dd_tau_shaped'
    #else:
    if dd_tau>0:
        qmeas['experiment'] = 'deer_dd_tau'
        qmeas['name'] = 'deer_dd_rect_tau'
    else:
        qmeas['experiment'] = 'deer_dd_par_tau'
        qmeas['name'] = 'deer_dd_rect_par_tau'
    
    if charge_init:
        qmeas['name'] = 'deer_dd_tau_fci'
        qmeas['experiment'] = 'deer_dd_tau_fci'
        qmeas['sequence_mode'] = True
        qmeas = add_fci(qmeas)
    
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed
    
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['rabi_period'] = common.rabi_period
    
    #if common.rabi_shaped['shaped_on']:
        # old legacy nvision method
        #qmeas['tau2_rel_to_pi1'] = True
        #qmeas['DD_type'] = dd_type
        #qmeas['order'] = dd_order

        #qmeas['mw_freq2'] = f2
        #qmeas['num_of_points'] = n_tau
        #qmeas['tau2_start'] = tau2_start
        #qmeas['tau2_incr'] = tau2_step
        #qmeas['tau'] = dd_tau
        #if not t_rabi_2:
        #    qmeas['rabi_period2'] = common.rabi_period    
        #else:
        #    qmeas['rabi_period2'] = t_rabi_2
        #if deer_ampl == None:
        #    qmeas['mw_amp2'] = common.mw_amp
        #else:
        #    qmeas['mw_amp2'] = deer_ampl
        #    
        #qmeas['read_phase'] = read_phase
    #else:
    # own method and pulse shaping    
    # parallel or sequential pi pulses via qmeas['experiment']
    qmeas['dd_type'] = dd_type
    qmeas['dd_order'] = dd_order

    qmeas['num_of_points'] = n_tau
    qmeas['tau_start'] = tau2_start
    # old:
    # shifting pulses in parallel, so negative values make no sense
    # qmeas['tau_step'] = abs(tau2_step)
    qmeas['tau_step'] = tau2_step

    qmeas['f_mw_2'] = list_2_csv(f2)
    if not t_rabi_2:
        qmeas['rabi_period_mw_2'] = list_2_csv(common.rabi_period)    
    else:
        qmeas['rabi_period_mw_2'] = list_2_csv(t_rabi_2)
    if deer_ampl == None:
        qmeas['ampl_mw_2'] = list_2_csv(common.mw_amp)
    else:
        qmeas['ampl_mw_2'] = list_2_csv(deer_ampl)

    qmeas['read_phase_deg'] = read_phase

    # sequential pi pulses
    qmeas['tau1'] = dd_tau
    
    # 1 ns safety between pi pulses
    #qmeas['tau2'] = dd_tau/2 - qmeas['rabi_period2']/4 - common.rabi_period/4 - 1e-9


    
    qmeas['measurement_time'] = 150000 
    qmeas['optimize_time'] = None
    
    qmeas['alternating'] = True


    qmeas = add_shaped(qmeas)
    return qmeas

def setup_deer_f_params(f_res, he_tau=200e-9, f_step=2e6, n_f=50, two_deer_pi=True, pi_len=None, read_phases='0, 180',
                       deer_ampl=None):
    qmeas=OrderedDict()
    qmeas['name'] = 'deer_spectrum'
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed

    qmeas['freq_step'] = f_step
    qmeas['num_of_points'] = n_f
    qmeas['freq_start'] = f_res - (qmeas['freq_step'] * qmeas['num_of_points'])/2.
    qmeas['rabi_period'] = common.rabi_period

    qmeas['measurement_time'] = 150000 
    qmeas['optimize_time'] = None
    
    qmeas['alternating'] = True
    qmeas['two_deer_pi'] = two_deer_pi
    qmeas['he_tau'] = he_tau
    if not deer_ampl:
        qmeas['deer_amp'] = common.mw_amp
    else:
        qmeas['deer_amp'] = deer_ampl
    if not pi_len:
        qmeas['pi_len'] = common.rabi_period/2
    else:
        qmeas['pi_len'] = pi_len
    
    qmeas['read_phase_degree'] = read_phases
    
    return qmeas

def setup_deer_rabi_params(deer_freq, tau_start=10e-9, tau_step=1e-6, n_tau=50, 
                      he_tau=300e-9, read_phases='0, 180', deer_ampl=None,
                      two_deer_pi=True):
    qmeas=OrderedDict()
    qmeas['name'] = 'deer_rabi'
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed

    qmeas['tau_start'] = tau_start
    qmeas['num_of_taus'] = n_tau
    qmeas['tau_step'] = tau_step

    qmeas['measurement_time'] = 99999 
    qmeas['optimize_time'] = None
    

    qmeas['he_tau'] = he_tau
    qmeas['two_deer_pi'] = two_deer_pi
    if not deer_ampl:
        qmeas['deer_amp'] = common.mw_amp
    else:
        qmeas['deer_amp'] = deer_ampl
    qmeas['deer_freq'] = deer_freq

    
    qmeas['alternating'] = True
    qmeas['read_phase_degree'] = read_phases
    
    return qmeas


def setup_deer_params(deer_freq, tau_start=10e-9, tau_step=1e-6, n_tau=50, 
                      he_tau=300e-9, second_rabi_period=None,
                      two_deer_pi=True):
    qmeas=OrderedDict()
    qmeas['name'] = 'DEER'
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed

    qmeas['tau_start'] = tau_start
    qmeas['num_of_points'] = n_tau
    qmeas['tau_step'] = tau_step

    qmeas['measurement_time'] = 99999 
    qmeas['optimize_time'] = None
    

    qmeas['he_tau'] = he_tau
    qmeas['two_deer_pi'] = two_deer_pi

    qmeas['deer_amp'] = common.mw_amp
    qmeas['deer_freq'] = deer_freq
    if not second_rabi_period:
        qmeas['second_rabi_period'] = common.rabi_period
    else:
        qmeas['second_rabi_period'] = second_rabi_period
   
    qmeas['alternating'] = True
    
    return qmeas


def setup_ramsey_deer_pi_params(f_deer, t_pi_deer, 
                                tau_start=100e-9, n_tau=1, tau_step=None,
                                alternating_mode=DeerAltModes.NVPi3Half,
                                two_deer_pi=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'Ramsey_deer_pi'
    qmeas['experiment'] = 'ramsey_deer_pi'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_start'] = tau_start
    if tau_step is None:
        qmeas['tau_step'] = 50e-9
    else:
        qmeas['tau_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    qmeas['alternating_mode'] = alternating_mode
    
    qmeas['from_list'] = False
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if n_tau > 1:
        qmeas['fit_experiment'] = 'sine'
     
    qmeas['f_mw_deer'] = f_deer
    qmeas['t_pi_deer'] = t_pi_deer
    qmeas['two_deer_pi'] = two_deer_pi
    
    return qmeas

def setup_podmr_deer_pi_params(f_res, f_deer, t_pi_deer, 
                               p_scale=10, p_scale_onlyperiod=1, f_step=0.1e6, n_f=50, t_pi_manual=None):
    # @param p_scale_onlyperiod: scales rabi period according to given power factor.
    #                            useful, if externally controlling poewr
    qmeas=OrderedDict()
    qmeas['name'] = 'pulsedodmr_deer_pi'
    qmeas['experiment'] = 'pulsedodmr_deer_pi'
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed
    if t_pi_manual:
        qmeas['rabi_period'] = 2*t_pi_manual
    else:
        qmeas['rabi_period'] = common.rabi_period*p_scale*p_scale_onlyperiod
    qmeas['freq_step'] = f_step
    qmeas['num_of_points'] = n_f
    qmeas['freq_start'] = f_res - (qmeas['freq_step'] * qmeas['num_of_points'])/2.
    if p_scale != 1:
        qmeas['microwave_amplitude'] = common.mw_amp/(p_scale)
        qmeas['deer_amplitude'] = common.mw_amp
    else:
        qmeas['microwave_amplitude'] = common.mw_amp
        qmeas['deer_amplitude'] = common.mw_amp
    qmeas['measurement_time'] = 1500 
    qmeas['optimize_time'] = None
    qmeas['fit_experiment'] = 'lor_dip'
    
    qmeas['f_mw_deer'] = f_deer
    qmeas['t_pi_deer'] = t_pi_deer
    
    return qmeas



def run_deer_f(mes_params, save_tag=None, generate_new=True):
    laser_off()
    if save_tag is None:
        save_tag='deer_f'
    do_experiment(experiment='deer_spectrum', qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information, 
                  generate_new=generate_new, save_tag=save_tag, load_tag='')

def run_deer(mes_params, save_tag=None, generate_new=True):
    laser_off()
    if save_tag is None:
        save_tag='deer'
    do_experiment(experiment='DEER', qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information, 
                  generate_new=generate_new, save_tag=save_tag, load_tag='')


def run_deer_rabi(mes_params, save_tag=None, generate_new=True):
    laser_off()
    if save_tag is None:
        save_tag='deer_rabi'
    do_experiment(experiment='deer_rabi', qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information, 
                  generate_new=generate_new, save_tag=save_tag, load_tag='')


    
def run_deer_pi_start(mes_params, save_tag=None, generate_new=True):
    laser_off()
    if save_tag is None:
        save_tag='deer_pi_start'
    do_experiment(experiment='DEER_pi_start', qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information, 
                  generate_new=generate_new, save_tag=save_tag, load_tag='')
    
def setup_double_hh_f_params(f_start=0e6, n_f=50, f_step=1e6, spinlock_length=100e-9,
                             ampl_spinlock=0.25, ampl_dress2=0.25):
    qmeas=OrderedDict()
    qmeas['name'] = 'HH_double_f'
    qmeas['experiment'] = 'HH_double_fsweep'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['f_start'] = f_start
    if f_step is None:
        qmeas['f_step'] = 1e6
    else:
        qmeas['f_step'] = f_step
    qmeas['num_of_points'] = n_f
    
    qmeas['spinlock_length'] = spinlock_length
    qmeas['ampl_spinlock'] = ampl_spinlock
    qmeas['ampl_dress2'] = ampl_dress2
    qmeas['phase_dress2'] = 90
    
    qmeas['wait_time'] = common.t_wait_pulsed
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
 
    #qmeas = setup_penta_laser_analysis(qmeas)
    #qmeas = setup_penta_fastcounter_settings(qmeas)
    
    return qmeas

def setup_double_hh_tau_params(f_dress2=10e6, n_tau=50, tau_start=10e-9, tau_step=100e-9,
                             ampl_spinlock=0.25, ampl_dress2=0.25):
    qmeas=OrderedDict()
    qmeas['name'] = 'HH_double_tau'
    qmeas['experiment'] = 'HH_double_tau'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    
    qmeas['dress2_f'] = f_dress2
    qmeas['ampl_spinlock'] = ampl_spinlock
    qmeas['ampl_dress2'] = ampl_dress2
    qmeas['phase_dress2'] = 90
    
    qmeas['wait_time'] = common.t_wait_pulsed
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
 
    #qmeas = setup_penta_laser_analysis(qmeas)
    #qmeas = setup_penta_fastcounter_settings(qmeas)
    
    return qmeas

def setup_double_hh_dsl_tau_params(f_dress2=10e6, n_tau=50, tau_start=10e-9, tau_step=100e-9,
                             ampl_spinlock=0.25, ampl_pi2_dress2=0.1, ampl_dress2=0.25, t_rabi_dress2=100e-9,
                                  alternating_dress2_init=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'HH_double_dsl_tau'
    qmeas['experiment'] = 'HH_double_dsl_tau'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    
    qmeas['dress2_f'] = f_dress2
    qmeas['ampl_spinlock'] = ampl_spinlock
    qmeas['ampl_dress2'] = ampl_dress2
    qmeas['ampl_pi2_dress2'] = ampl_pi2_dress2
    qmeas['t_rabi_dress2'] = t_rabi_dress2
    qmeas['phase_dress2'] = 90
    qmeas['alternating_pi2_dress2'] = alternating_dress2_init
    
    qmeas['wait_time'] = common.t_wait_pulsed
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
 
    #qmeas = setup_penta_laser_analysis(qmeas)
    #qmeas = setup_penta_fastcounter_settings(qmeas)
    
    return qmeas


def setup_double_hh_ise_tau_params(f_dress2=10e6, n_tau=50, tau_start=10e-9, tau_step=100e-9,
                             ampl_spinlock=0.25, df_ise=300e3, ampl_ise=0.1, sweep_speed_ise=0.1e12,
                                  alternating_depol=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'HH_double_ise_tau'
    qmeas['experiment'] = 'HH_double_ise_tau'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    
    qmeas['dress2_f'] = f_dress2
    qmeas['ampl_spinlock'] = ampl_spinlock
    qmeas['ampl_ise'] = ampl_ise
    qmeas['sweep_speed_ise'] = sweep_speed_ise
    qmeas['df_ise'] = df_ise
    qmeas['alternating_pol'] = alternating_depol
    
    qmeas['wait_time'] = common.t_wait_pulsed
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
 
    #qmeas = setup_penta_laser_analysis(qmeas)
    #qmeas = setup_penta_fastcounter_settings(qmeas)
    
    return qmeas

def setup_double_hh_ise_ramsey_params(f_dress2=10e6, n_tau=50, tau_start=10e-9, tau_step=100e-9, tau_ise=10e-6,
                             ampl_spinlock=0.25, df_ise=300e3, ampl_ise=0.1, sweep_speed_ise=0.1e12,
                                  alternating_depol=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'HH_double_ise_ramsey'
    qmeas['experiment'] = 'HH_double_ise_ramsey'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_ram_start'] = tau_start
    qmeas['tau_ram_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    
    qmeas['tau_ise'] = tau_ise
    qmeas['dress2_f'] = f_dress2
    qmeas['ampl_spinlock'] = ampl_spinlock
    qmeas['ampl_ise'] = ampl_ise
    qmeas['sweep_speed_ise'] = sweep_speed_ise
    qmeas['df_ise'] = df_ise
    qmeas['alternating_pol'] = alternating_depol
    
    qmeas['wait_time'] = common.t_wait_pulsed
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
 
    #qmeas = setup_penta_laser_analysis(qmeas)
    #qmeas = setup_penta_fastcounter_settings(qmeas)
    
    return qmeas

def setup_double_hh_ise_dhh_f_params(f_dress2=10e6, n_tau=50, f_start=10e-9, f_step=100e-9, tau_ise=10e-6,
                             ampl_spinlock=0.25, df_ise=300e3, ampl_ise=0.1, sweep_speed_ise=0.1e12,
                             ampl_dhh_dress2=0.1, t_dhh_lock=10e-6,
                             alternating_depol=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'HH_double_ise_dhh_f'
    qmeas['experiment'] = 'HH_double_ise_dhh_f'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['f_dhh_start'] = f_start
    qmeas['f_dhh_step'] = f_step
    qmeas['num_of_points'] = n_tau
    
    qmeas['tau_ise'] = tau_ise
    qmeas['dress2_f'] = f_dress2
    qmeas['ampl_spinlock'] = ampl_spinlock
    qmeas['ampl_ise'] = ampl_ise
    qmeas['sweep_speed_ise'] = sweep_speed_ise
    qmeas['df_ise'] = df_ise
    qmeas['alternating_pol'] = alternating_depol
    qmeas['ampl_dhh_dress2'] = ampl_dhh_dress2
    qmeas['t_dhh_lock'] = t_dhh_lock
    
    qmeas['wait_time'] = common.t_wait_pulsed
    
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
 
    #qmeas = setup_penta_laser_analysis(qmeas)
    #qmeas = setup_penta_fastcounter_settings(qmeas)
    
    return qmeas

def setup_hh_tau_params(tau_start=0e-6, tau_step=1e-6, n_tau=50, amp_sl=1e-3):
    qmeas=OrderedDict()
    qmeas['name'] = 'hh_tau'
    qmeas['experiment'] = 'HHtau'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    
    qmeas['spinlock_amp'] = amp_sl
    
    qmeas['wait_time'] = common.t_wait_pulsed
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
 
    #qmeas = setup_penta_laser_analysis(qmeas)
    #qmeas = setup_penta_fastcounter_settings(qmeas)
    
    return qmeas

def setup_dd_mollow_n_params(tau=500e-9, dd_start=1, dd_step=1, n_dd=20, dd_type=DDMethods.SE, 
                             ampl_mol=1.5e-3, f_mol=0e6, phase_mod='0',
                             alternating=True):

    qmeas=OrderedDict()
    qmeas['experiment'] = 'dd_mollow_n'
    qmeas['name'] = f"dd_mollow_{str(dd_type).replace('DDMethods.','')}_n"
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['tau'] = tau
    qmeas['dd_order_start'] = dd_start
    qmeas['dd_step'] = dd_step
    qmeas['dd_type'] = dd_type
    qmeas['ampl_mol'] = ampl_mol
    qmeas['f_mol'] = f_mol
    qmeas['phase_mod'] = phase_mod
        
    
    qmeas['num_of_points'] = n_dd
    qmeas['alternating'] = alternating
        
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if n_dd > 1:
        qmeas['fit_experiment'] = 'sine_decay'
     
    qmeas = add_shaped(qmeas)
    
    return qmeas

def setup_rnovel_ppol_propi_params(dd_tau=500e-9, dd_order=1, dd_type=DDMethods.CPMG, 
                             ampl_mol=1.5e-3, n_pol=100, tau_ppol=0.5e-6, order_ppol=10, m_read_step=30,
                             alternating=True, shape_only_ppol=False):

    qmeas=OrderedDict()
    qmeas['experiment'] = 'rnovel_ppol_propi'

    qmeas['name'] = f"rnovel_ppol_propi"
    qmeas['sequence_mode'] = True
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    if shape_only_ppol:
        qmeas['dd_t_rabi_rect'] = common.rabi_period_rect
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    # rnovel params
    qmeas['dd_mol_tau'] = tau
    qmeas['dd_mol_order'] = dd_order
    qmeas['dd_mol_ampl'] = ampl_mol
    qmeas['dd_mol_type'] = dd_type
    qmeas['n_pol'] = n_pol
    # propi params
    qmeas['m_read_step'] = m_read_step
    qmeas['tau_ppol'] = tau_ppol
    qmeas['order_ppol'] = order_ppol
        

    qmeas['alternating'] = alternating
        
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    
    qmeas = add_shaped(qmeas)
        
    if m_read_step > 1:
        qmeas['fit_experiment'] = 'decay_exp'
 
    return qmeas

def setup_ppol_tau_params(tau_start=0.5e-6, tau_step=20e-9,
                         num_of_points=50, order=8,
                             alternating=True):

    qmeas=OrderedDict()
    qmeas['experiment'] = 'Poltau'

    qmeas['name'] = f"ppol_tau"
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    qmeas['order'] = order
    
    qmeas['num_of_points'] = num_of_points
    qmeas['alternating'] = alternating
        
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if num_of_points > 1:
        qmeas['fit_experiment'] = 'lor_dip'
     
    
    qmeas = add_shaped(qmeas)
        
    return qmeas

def setup_ppol_n_params(tau=500e-9, n_start=1, n_step=1,
                         num_of_points=50, alternating=True):

    qmeas=OrderedDict()
    qmeas['experiment'] = 'Pol20_order'
    
    qmeas['name'] = f"ppol_n"
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['n_start'] = n_start
    qmeas['n_step'] = n_start
    qmeas['tau'] = tau
    
    qmeas['num_of_points'] = num_of_points
    qmeas['alternating'] = alternating
        
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if num_of_points > 1:
        qmeas['fit_experiment'] = 'sine'
    
    qmeas = add_shaped(qmeas)
 
    return qmeas

def setup_ppol_propi_params(tau=500e-9, n_start=1, n_step=1,
                         num_of_points=50, alternating=True):

    qmeas=OrderedDict()
    qmeas['experiment'] = 'Pol20_order'
    qmeas['name'] = f"ppol_n"
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['n_start'] = n_start
    qmeas['n_step'] = n_start
    qmeas['tau'] = tau
    
    qmeas['num_of_points'] = num_of_points
    qmeas['alternating'] = alternating
        
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if num_of_points > 1:
        qmeas['fit_experiment'] = 'sine'
     
    qmeas = add_shaped(qmeas)
        
    return qmeas


def setup_ppol_2x_propi_params(tau=500e-9, n_pol=100, m_read=20, order_ppol=8,
                         alternating=True):

    qmeas=OrderedDict()
    qmeas['experiment'] = 'ppol_2x_propi'
    qmeas['name'] = f"ppol_2x_propi"
    qmeas['sequence_mode'] = True
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['n_pol'] = n_pol
    qmeas['m_read_step'] = m_read
    qmeas['tau_ppol'] = tau
    qmeas['order_ppol'] = order_ppol
    
    #qmeas['num_of_points'] = num_of_points
    qmeas['alternating'] = alternating
        
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if m_read > 1:
        qmeas['fit_experiment'] = 'decay_exp'
     
    qmeas = add_shaped(qmeas)
        
    return qmeas

def setup_ise_propi_params(tau_ppol=500e-9, n_pol=100, m_read=20, order_ppol=8,
                           df_mw_sweep=3e6, t_ise=1e-6, mw_sweep_speed=1e12, amp_mw_sweep=0.1,
                           ppol_read_dir='down',
                           alternating=True):

    qmeas=OrderedDict()
    qmeas['experiment'] = 'ise_ppol_propi'
    qmeas['name'] = f"ise_ppol_propi"
    qmeas['sequence_mode'] = True
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['n_pol'] = n_pol
    qmeas['m_read_step'] = m_read
    qmeas['tau_ppol'] = tau_ppol
    qmeas['order_ppol'] = order_ppol
    qmeas['ppol_read_dir'] = ppol_read_dir
    
    qmeas['t_ise'] = t_ise
    qmeas['df_mw_sweep'] = df_mw_sweep
    qmeas['mw_sweep_speed'] = mw_sweep_speed
    qmeas['amp_mw_sweep'] = amp_mw_sweep
    
    
    #qmeas['num_of_points'] = num_of_points
    qmeas['alternating'] = alternating
        
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if m_read > 1:
        qmeas['fit_experiment'] = 'decay_exp'
     
    qmeas = add_shaped(qmeas)
        
    return qmeas


def setup_dd_dqt_sigamp_params(tau=100e-9, n_amp=50, amp_step=0.01e-3,
                               t_rabi1=None, t_rabi2=None,  
                               f_mw1=[], f_mw2=[], ampl_mw2=0.0,
                               dd_type=DDMethods.XY8, dd_order=1, alternating=True):

    qmeas=OrderedDict()
    
    if not t_rabi1==None:
        qmeas['rabi_period'] = t_rabi1
    else:
        qmeas['rabi_period'] = common.rabi_period 
    
    is_multi_drive = False
    if len(f_mw2) or len(f_mw1) > 1:
        is_multi_drive = True
        
    
    if common.rabi_shaped['shaped_on']:
        raise ValueError("Shaping not supported")
    else:
        qmeas['name'] = 'dd_dqt_sigamp'
        qmeas['experiment'] = 'dd_dqt_sigamp'

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['microwave_amplitude'] = common.mw_amp
 
    qmeas['tau'] = tau
    qmeas['amp_start'] = 0e-3
    qmeas['amp_step'] = amp_step
    qmeas['dd_type'] = dd_type
    qmeas['dd_order'] = dd_order
    qmeas['alternating'] = alternating
    qmeas['num_of_points'] = n_amp
    
    
    if is_multi_drive:
        # currently drives 1,2 have same amplitude, respectively
        qmeas['f_mw2'] = list_2_csv(f_mw2)
        qmeas['t_rabi_mw2'] = t_rabi2 if t_rabi2 else t_rabi1
        qmeas['ampl_mw2'] = ampl_mw2
        if len(f_mw1) == 0:
            qmeas['microwave_frequency'] = common.mw_freq
        else:
            qmeas['microwave_frequency'] = f_mw1[0]
            qmeas['f_mw1_add'] = list_2_csv(f_mw1[1:])
            
    else:
         qmeas['microwave_frequency'] = common.mw_freq #+ 0.5*3.03e6
    

    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'sine'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5

    qmeas = add_shaped(qmeas)

    return qmeas

def setup_ent_create_bell_params(tau_start=0.5e-6, tau_step=0.01e-6, n_tau=50,
                             f_mw_1=[], f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             dd_type=DDMethods.SE, dd_order=1, alternating=True):

    qmeas=OrderedDict()

    is_multi_drive = False
    if len(f_mw_2) > 0 or len(f_mw_1) > 1:
        is_multi_drive = True
        
    if common.rabi_shaped['shaped_on']:
        raise ValueError("Shaping not supported")
    else:
        qmeas['name'] = 'ent_create_bell'
        qmeas['experiment'] = 'ent_create_bell'

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['microwave_frequency'] = common.mw_freq
    qmeas['rabi_period'] = common.rabi_period
 
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    qmeas['read_phase_deg'] = 90
    qmeas['dd_type'] = dd_type
    qmeas['dd_order'] = dd_order
    qmeas['alternating'] = alternating
    qmeas['num_of_points'] = n_tau

    
    if is_multi_drive:
        # currently drives 1,2 have same amplitude, respectively
        qmeas['f_mw_2'] = list_2_csv(f_mw_2)
        qmeas['rabi_period_mw_2'] = list_2_csv(rabi_period_mw_2)
        qmeas['ampl_mw_2'] = list_2_csv(ampl_mw_2)            
    

    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'sine'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5

    qmeas = add_shaped(qmeas)

    return qmeas

def setup_bell_hahnecho_params(tau_start=0.5e-6, tau_step=0.01e-6, n_tau=50, tau_cnot=100e-9,
                             tau_dd_fix=None,
                             f_mw_1=[], f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             dd_type=DDMethods.SE, dd_order=1, alternating=True):
   
    qmeas = setup_bell_ramsey_params(tau_start=tau_start, tau_step=tau_step, n_tau=n_tau, tau_cnot=tau_cnot,
                             tau_dd_fix=tau_dd_fix,
                             f_mw_1=f_mw_1, f_mw_2=f_mw_2, ampl_mw_2=ampl_mw_2, rabi_period_mw_2=rabi_period_mw_2,
                             dd_type=dd_type, dd_order=dd_order, alternating=alternating)
    qmeas['name'] = 'bell_hahn'
    qmeas['experiment'] = 'bell_hahnecho'
                                     
    qmeas['fit_experiment'] = 'decay_exp'
    return qmeas

def setup_bell_ramsey_params(tau_start=0.5e-6, tau_step=0.01e-6, n_tau=50, tau_cnot=100e-9,
                             tau_dd_fix=None,
                             f_mw_1=[], f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             dd_type=DDMethods.SE, dd_order=1, alternating=True):

    qmeas=OrderedDict()

    is_multi_drive = False
    if len(f_mw_2) > 0 or len(f_mw_1) > 1:
        is_multi_drive = True
        
    if common.rabi_shaped['shaped_on']:
        raise ValueError("Shaping not supported")
    else:
        qmeas['name'] = 'bell_ramsey'
        qmeas['experiment'] = 'bell_ramsey'

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['microwave_frequency'] = common.mw_freq
    qmeas['rabi_period'] = common.rabi_period
 
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    qmeas['tau_cnot'] = tau_cnot
    qmeas['cnot_kwargs'] = {'tau_dd_fix': tau_dd_fix} 
        
    qmeas['dd_type'] = dd_type
    qmeas['dd_order'] = dd_order
    qmeas['alternating'] = alternating
    qmeas['num_of_points'] = n_tau

    
    if is_multi_drive:
        # currently drives 1,2 have same amplitude, respectively
        qmeas['f_mw_2'] = list_2_csv(f_mw_2)
        qmeas['rabi_period_mw_2'] = list_2_csv(rabi_period_mw_2)
        qmeas['ampl_mw_2'] = list_2_csv(ampl_mw_2)            
    
  

    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'sine'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5

    qmeas = add_shaped(qmeas)

    return qmeas

def setup_tomography_params(tau_start=0e-9, tau_step=None, n_tau=50, 
                             read_rotation=TomoRotations.none, init_state=TomoInit.none,
                             rabi_on_nv=1, rabi_phase_deg=0,
                             f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             tau_cnot=100e-9, dd_type_cnot=DDMethods.SE, dd_order=1,
                             use_shaped_cnot=False, tau_dd_fix=None,
                             alternating=True):

    qmeas=OrderedDict()

    is_multi_drive = False
    if len(f_mw_2) > 0:
        is_multi_drive = True

    # for awg8190 name must be < 32 chars incl ch extension
    #qmeas['name'] = f'tomo_rot={read_rotation.name}'  
    qmeas['name'] = f'tomo_rot'  
    qmeas['experiment'] = 'tomography'
    #qmeas['savetag'] = f'tomography_on{rabi_on_nv}_rot={read_rotation.name}_phi={rabi_phase_deg}'
    qmeas['savetag'] = f'tomography_on{rabi_on_nv}_rot_phi={rabi_phase_deg}'

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['microwave_frequency'] = common.mw_freq
    qmeas['rabi_period'] = common.rabi_period
    
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    if tau_step is None:
        if rabi_on_nv == 1:
            qmeas['tau_step'] = common.rabi_period/20
        elif rabi_on_nv == 2:
            qmeas['tau_step'] = common.rabi_period_2[0]/20
        else:
            raise ValueError
        
    qmeas['rabi_on_nv'] = rabi_on_nv
    qmeas['rabi_phase_deg'] = rabi_phase_deg
    qmeas['rotation'] = read_rotation
    qmeas['init_state'] = init_state
    qmeas['init_state_kwargs'] = {}   # todo, needed for cnot/ent state
    #{'dd_type': DDMethods.CPMG}
    
    qmeas['cnot_kwargs'] = {'tau_dd_fix': tau_dd_fix}   
    if use_shaped_cnot:
        cnot_kwargs = {'env_type':Evm.parabola, 
                   'order_P': 1,
                   'tau_dd_fix': tau_dd_fix,
                   'rabi_period': common.rabi_shaped['period'],
                   'rabi_period_mw_2': list_2_csv(common.t_rabi_2_shaped)}
        qmeas['cnot_kwargs'] = cnot_kwargs
    
    
    qmeas['tau_cnot'] = tau_cnot
    qmeas['dd_type_cnot'] = dd_type_cnot
    qmeas['dd_order'] = dd_order
    qmeas['alternating'] = alternating
    qmeas['num_of_points'] = n_tau
    
    if tau_cnot < 0 and tau_dd_fix == None:
        raise ValueError(f"Negative tau_cnot {tau_cnot} not allowed for cnot based on deer_dd_par!")
    
    if is_multi_drive:
        # currently drives 1,2 have same amplitude, respectively
        qmeas['f_mw_2'] = list_2_csv(f_mw_2)
        qmeas['rabi_period_mw_2'] = list_2_csv(rabi_period_mw_2)
        qmeas['ampl_mw_2'] = list_2_csv(ampl_mw_2)            
    

    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'sine'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5

    qmeas = add_shaped(qmeas)

    return qmeas


def setup_tomography_single_params(
                             read_rotation=[TomoRotations.none], init_state=[TomoInit.none],
                             f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             tau_cnot=100e-9, dd_type_cnot=DDMethods.SE, dd_order=1,
                             tau_dd_fix=None,
                             alternating=True):

    qmeas=OrderedDict()

    is_multi_drive = False
    if len(f_mw_2) > 0:
        is_multi_drive = True
        
    if common.rabi_shaped['shaped_on']:
        raise ValueError("Shaping not supported")
    else:
        # for awg8190 name must be < 32 chars incl ch extension
        qmeas['name'] = f'tomography_single'  
        qmeas['experiment'] = 'tomography_single'
        qmeas['savetag'] = f'tomography_single'  

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['microwave_frequency'] = common.mw_freq
    qmeas['rabi_period'] = common.rabi_period

    qmeas['rotations'] = str(read_rotation)
    qmeas['init_states'] = str(init_state)
    qmeas['init_state_kwargs'] = {}   # todo, needed for cnot/ent state
    #{'dd_type': DDMethods.CPMG}
    
    qmeas['cnot_kwargs'] = {'tau_dd_fix': tau_dd_fix}   
    qmeas['init_state_kwargs'] = {}
    
    
    qmeas['tau_cnot'] = tau_cnot
    qmeas['dd_type_cnot'] = dd_type_cnot
    qmeas['dd_order'] = dd_order
    qmeas['alternating'] = alternating
    
    if tau_cnot < 0 and tau_dd_fix == None:
        raise ValueError(f"Negative tau_cnot {tau_cnot} not allowed for cnot based on deer_dd_par!")
    
    if is_multi_drive:
        # currently drives 1,2 have same amplitude, respectively
        qmeas['f_mw_2'] = list_2_csv(f_mw_2)
        qmeas['rabi_period_mw_2'] = list_2_csv(rabi_period_mw_2)
        qmeas['ampl_mw_2'] = list_2_csv(ampl_mw_2)            
    

    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'sine'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5

    qmeas = add_shaped(qmeas)

    return qmeas

0

0

## Basics

In [86]:

import pickle
from scipy.optimize import curve_fit
from scipy.stats import poisson

# new funtion inserted
def toggle_pg(on=False):
    if on:
        pulsedmeasurementlogic.toggle_measurement_pause(pause=False)
    else:
        pulsedmeasurementlogic.toggle_measurement_pause(pause=True)

def enable_listfile(qmeas):
    mes_params = qmeas
    try:
        enable_listfile = False
        if 'listfile' in mes_params.keys():
            if mes_params['listfile']:
                enable_listfile = True
        if enable_listfile:
            data_dir =  os.path.abspath(savelogic.get_daily_directory())
            name = f"{mes_params['savetag']}.mpa"
            myfastcounter.change_save_mode(2)
            myfastcounter.change_filename(data_dir + r'\\' + name)
            logger.info(f"Enabling listfile, will .lst and .mpa write to: {myfastcounter.get_filename()}")
            # saving mpa done in logic_essentials.py. Need to change_save_mode(0) and save again
        else:
            # no list file
            myfastcounter.change_save_mode(0)
    except:
        logger.warning("Enabling listfile failed. Does you fastcounter support it?")
#-----------------------------------------------------------------------------------------------------------------------------------
def setup_nuc_podmr_params(f_start=2.7e6,  f_step=5e3, n_f=50, t_nuc_rabi=50e6, rf_ampl=50e-3):
    qmeas=OrderedDict()

    if common.rabi_shaped['shaped_on']:
        raise NotImplemented
    else:
        qmeas['name'] = 'nuc_pulsedodmr'
        qmeas['experiment'] = 'nuc_pulsedodmr'

    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed
    
    qmeas['microwave_frequency'] = common.mw_freq #+ 0.5*3.03e6
    qmeas['rabi_period'] = common.rabi_period
    qmeas['freq_start'] = f_start
    qmeas['freq_step'] = f_step
    qmeas['num_of_points'] = n_f
    
    qmeas['rf_chnl'] = 'a_ch2'
    qmeas['rf_ampl'] = rf_ampl
    qmeas['t_nuc_rabi'] = t_nuc_rabi
    qmeas['nv_pi_init'] = True
    qmeas['alternating_no_rf'] = True
    

    qmeas['measurement_time'] = 1500 
    qmeas['optimize_time'] = None
    qmeas['fit_experiment'] = 'lor_dip'

    return qmeas

def setup_nuc_rabi_params(tau_step=1e-6, rf_freq=2.6e6, rf_ampl=50e-3, n_tau=50):

    qmeas=OrderedDict()

    if common.rabi_shaped['shaped_on']:
        raise NotImplemented
    else:
        qmeas['name'] = 'nuc_rabi'
        qmeas['experiment'] = 'nuc_rabi'

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['rabi_period'] = common.rabi_period

    qmeas['tau_step'] = tau_step
    qmeas['tau_start'] = 1e-6    
    qmeas['num_of_points'] = n_tau
    qmeas['microwave_frequency'] = common.mw_freq #+ 0.5*3.03e6
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['rf_chnl'] = 'a_ch2'
    qmeas['rf_ampl'] = rf_ampl
    qmeas['rf_freq'] = rf_freq
    qmeas['nv_pi_init'] = True
    qmeas['alternating_no_rf'] = True
    
    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'sine'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5


    return qmeas

def setup_podmr_params(f_res, p_scale=10, p_scale_onlyperiod=1, f_step=0.1e6, n_f=50, t_pi_manual=None):
    # @param p_scale_onlyperiod: scales rabi period according to given power factor.
    #                            useful, if externally controlling poewr
    qmeas=OrderedDict()

    if common.rabi_shaped['shaped_on']:
        qmeas['name'] = 'pulsedodmr'
        qmeas['experiment'] = 'pulsedodmr'
        # empirically, don't know where the scaling difference comes from
        #p_scale_onlyperiod *= 0.5
    else:
        qmeas['name'] = 'pulsedodmr'
        qmeas['experiment'] = 'pulsedodmr'

    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed

    if t_pi_manual:
        qmeas['rabi_period'] = 2*t_pi_manual
    else:
        qmeas['rabi_period'] = common.rabi_period*p_scale*p_scale_onlyperiod
    qmeas['freq_step'] = f_step
    qmeas['num_of_points'] = n_f
    qmeas['freq_start'] = f_res - (qmeas['freq_step'] * qmeas['num_of_points'])/2.
    if p_scale != 1:
        qmeas['microwave_amplitude'] = common.mw_amp/(p_scale)
    else:
        qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['measurement_time'] = 1500 
    qmeas['optimize_time'] = None
    qmeas['fit_experiment'] = 'lor_dip'

    qmeas = add_shaped(qmeas)

    return qmeas

# The commented out function was a old version
#def list_2_csv(in_list):
 #   str_list = ""
  #  
   # if type(in_list) != list:
    #    in_list = [in_list]
        
    #for el in in_list:
     #   str_list += f"{el}, "
    
    #if len(str_list) > 0:
     #   str_list = str_list[:-2]
        
    #return str_list
def list_2_csv(in_list, line_delimiter=";"):
    """
    :param line_delimter: if given lists of lists, will create lines per out list 
                          that are seperated by the line_delimiter
    """
    str_list = ""
    
    if type(in_list) != list:
        in_list = [in_list]
        
    for el in in_list:
        if type(el) == list:
            str_list += f"{list_2_csv(el)}{line_delimiter} " 
        else:
            str_list += f"{repr(el)}, "
        #str_list += f"{el}, "

        
    if len(str_list) > 0:
        str_list = str_list[:-2]
        
    return str_list

def setup_rabi_params(tau_step=None, p_scale=1, n_tau=50, leave_out_tau_idx='', 
                      f_mw1=[], f_mw2=[], ampl_mw2=0.0,
                      alternating=True, charge_init=False):

    qmeas=OrderedDict()
    
    is_multi_drive = False
    if len(f_mw2) > 0:
        is_multi_drive = True
        
    if is_multi_drive:
        qmeas['name'] = 'rabi_multi'
        qmeas['experiment'] = 'rabi_dqt_p'
    else:
        qmeas['name'] = 'rabi'
        qmeas['experiment'] = 'rabi'
    if charge_init:
        qmeas['name'] = 'rabi_fci'
        qmeas['experiment'] = 'rabi_fci'
        qmeas['sequence_mode'] = True
        qmeas = add_fci(qmeas)

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['rabi_period'] = common.rabi_period

    if tau_step is None:   
        tau_step = 5e-9
    qmeas['tau_step'] = tau_step
    qmeas['tau_start'] = 0e-9
    qmeas['alternating'] = alternating
    qmeas['leave_out_tau_idx'] = leave_out_tau_idx
    if is_multi_drive:
        # currently drives 1,2 have same amplitude, respectively
        qmeas['f_mw_2'] = list_2_csv(f_mw2)
        qmeas['ampl_mw_2'] = ampl_mw2
        if alternating:
            qmeas['alternating_mode'] = DQTAltModes.DQT_12_alternating
        else:
            qmeas['alternating_mode'] = DQTAltModes.DQT_both
            
        if len(f_mw1) == 0:
            qmeas['microwave_frequency'] = common.mw_freq
        else:
            qmeas['microwave_frequency'] = f_mw1[0]
            qmeas['f_mw_1_add'] = list_2_csv(f_mw1[1:])
            
    else:
         qmeas['microwave_frequency'] = common.mw_freq #+ 0.5*3.03e6
    
    qmeas['num_of_points'] = n_tau
    if p_scale != 1:
        qmeas['microwave_amplitude'] = common.mw_amp/(p_scale)
    else:
        qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'sine'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5

    qmeas = add_shaped(qmeas)
    if common.rabi_shaped['shaped_on']:
        qmeas['name'] += '_shaped'

    return qmeas

def setup_ramsey_params(tau_start=100e-9, n_tau=1, tau_step=None, alternating=False, read_phases='0, 180',
                       red_readout=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'Ramsey'
    qmeas['experiment'] = 'ramsey_s3p'
    if not red_readout:
        setup['laser_length'] = common.t_laser_pulsed
    else:
        qmeas['t_laser_init'] = common.t_laser_green_init
        qmeas['laser_read_ch'] = 'd_ch3'
        setup['laser_length'] = common.t_laser_red_read

    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq + common.mw_ramsey_offset
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_start'] = tau_start
    if tau_step is None:
        qmeas['tau_step'] = 50e-9
    else:
        qmeas['tau_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    qmeas['alternating'] = alternating

    qmeas['from_list'] = False

    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if n_tau > 1:
        qmeas['fit_experiment'] = 'sine_decay'

    qmeas['read_phases_degree'] = read_phases

    return qmeas

def setup_hahn_params(tau_start=50e-9, n_tau=50, tau_step=0.5e-6, alternating=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'hahnecho'
    qmeas['experiment'] = 'hahnecho'
    setup['laser_length'] = common.t_laser_pulsed

    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_start'] = tau_start
    if tau_step is None:
        qmeas['tau_step'] = 50e-9
    else:
        qmeas['tau_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    qmeas['alternating'] = alternating

    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if n_tau > 1:
        qmeas['fit_experiment'] = 'exp'

    return qmeas


def setup_xy8_tausweep_params(tau_start=100e-9, n_tau=20, tau_step=None, xy8_order=1, alternating=False):
    qmeas=OrderedDict()
    qmeas['experiment'] = 'xy8_tau'
    qmeas['name'] = 'xy8_tau'
    setup['laser_length'] = common.t_laser_pulsed

    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp

    qmeas['xy8_order'] = xy8_order
    qmeas['tau_start'] = tau_start
    if tau_step is None:
        qmeas['tau_step'] = 100e-9
    else:
        qmeas['tau_step'] = tau_step


    qmeas['num_of_points'] = n_tau
    qmeas['alternating'] = alternating

    qmeas['from_list'] = False

    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if n_tau > 1:
        qmeas['fit_experiment'] = 'Lorentz dip'

    return qmeas

def setup_xy8_nsweep_params(tau=500e-9, n_n=20, xy8_start=1, xy8_step=1, alternating=False):
    qmeas=OrderedDict()
    qmeas['experiment'] = 'xy8_nsweep'
    qmeas['name'] = 'xy8_nsweep'
    setup['laser_length'] = common.t_laser_pulsed

    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp

    qmeas['tau'] = tau
    qmeas['xy8_start'] = xy8_start
    qmeas['xy8_step'] = xy8_step



    qmeas['num_of_points'] = n_n
    qmeas['alternating'] = alternating

    qmeas['from_list'] = False

    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if n_n > 1:
        qmeas['fit_experiment'] = 'sine_decay'

    return qmeas

def get_savetag(qmeas):
    if 'savetag' not in qmeas.keys():
        save_tag=f"{qmeas['name']}"
    else:
        save_tag = qmeas['savetag']
        
    return save_tag

def run_generic(mes_params, save_tag=None, generate_new=True):
    laser_off()
    if save_tag is None and 'savetag' in mes_params.keys():
        save_tag = mes_params['savetag']
    if save_tag is None:
        save_tag=f"{mes_params['name']}"
    
    mes_params['savetag'] = save_tag
    
    enable_listfile(mes_params)
        
    do_experiment(experiment=mes_params['experiment'], qm_dict=mes_params, meas_type=conventional_measurement,
                  meas_info=add_conventional_information,
                  generate_new=generate_new, save_tag=save_tag, load_tag='')

def run_generic_autoexp(qmeas_list, generate_new=False, optimize_between_exps=False, opt_on_poi=None,
                        poi_list=None):
        
    autoexp = OrderedDict()
    automeas = OrderedDict()

    automeas['list_pois'] = poi_list if poi_list else [""]
    automeas['generate_new'] = generate_new
    automeas['optimize_between_experiments'] = optimize_between_exps
    automeas['optimize_on_poi'] = opt_on_poi

        

    for qmeas in qmeas_list:
        exp_name = qmeas['name']
        qmeas['type'] = qmeas['experiment']
        qmeas['meas_type'] = conventional_measurement
        qmeas['meas_info'] = add_conventional_information
        
        if exp_name in autoexp.keys():
            raise ValueError(f"Experiment names must be unique in auto exp. Already found: {exp_name}")
        autoexp[exp_name] = cp.deepcopy(qmeas)
    
    do_automized_measurements(qm_dict=automeas, autoexp=autoexp)
    

def run_pdomr(mes_params, save_tag=None, generate_new=True):

    up_to = (mes_params['freq_start'] + mes_params['freq_step'] * mes_params['num_of_points'])
    laser_off()
    if save_tag is None:
        save_tag='pOdmr_{:.2f}_{:.2f}MHz'.format(mes_params['freq_start']/1e6, up_to/1e6)
    do_experiment(experiment=mes_params['experiment'], qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information, 
                  generate_new=generate_new, save_tag=save_tag, load_tag='')

def run_rabi(mes_params):
    laser_off()
    do_experiment(experiment=mes_params['experiment'], qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information,
                  generate_new=True, save_tag='Rabi_'+str(round(mes_params['microwave_amplitude']*1000))+'mV',load_tag='')

def run_xy8_tausweep(mes_params):
    laser_off()
    do_experiment(experiment='xy8_tau', qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information,
                  generate_new=True, save_tag='xy8-o={}_tau_'.format(mes_params['xy8_order']),load_tag='')

def run_xy8_nsweep(mes_params):
    laser_off()
    do_experiment(experiment='xy8_nsweep', qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information,
                  generate_new=True, save_tag='xy8_tau={}_nsweep'.format(mes_params['tau']),load_tag='')



def run_ramsey(mes_params, n_sweeps_stop=0):

    if n_sweeps_stop > 0:
        # else don't stop by sweeps
        run_ramsey_pj_params['n_sweeps'] = n_sweeps_stop

    laser_off()
    do_experiment(experiment=mes_params['experiment'], qm_dict=mes_params, meas_type=conventional_measurement, 
        meas_info=add_conventional_information, generate_new=True, save_tag='Ramsey',load_tag='')   

    if get_current_pulsed_mes_running():
        logger.warning("Mes still running after do_experiment().")
        return (None, None)

    else:
        mes = get_current_pulsed_mes()
        x = mes.signal_data[0]
        y = mes.signal_data[1]

    return (x,y)  


def create_temp_dir():
    #qudi_dir = os.getcwd()

    folder_dir = 'temp'
    try:
        os.mkdir(folder_dir)
    except FileExistsError as e:
        pass   # already exists

    return folder_dir 

def get_counts(t_avg_s=0.1):

    # stop refocusing for getting counts
    refocus_running = True
    try:
        if poimanagerlogic.time_until_refocus == -1:
            refocus_running = False
        else:
            poimanagerlogic.toggle_periodic_refocus(False)
    except NameError:
        refocus_running = False


    f_count = counterlogic.get_count_frequency()
    n_samples = int(t_avg_s * f_count)
    n_samples_max = counterlogic.get_count_length()
    if n_samples > n_samples_max:
        n_samples = n_samples_max

    # wait until refocus done for sure
    if refocus_running:
        timeout = False
        timeout_t_s = 5
        t_start = time.time()

        while(time.time() - poimanagerlogic._last_refocus < t_avg_s and not timeout):
            time.sleep(0.1)
            if time.time() - t_start > timeout_t_s:
                timeout = True

    cts = np.median(counterlogic.countdata[0][-n_samples:])    # counts per s

    if refocus_running:
        poimanagerlogic.toggle_periodic_refocus(True)

    return cts

0

0

# Laser options

## Optimize position

In [ ]:
#laser_on()
optimize_position()

In [ ]:
opt_on_poi

In [ ]:
optimize_poi(opt_on_poi, update_shift=True)

## Laser On

In [142]:
#laser_on(pulser_on = False)
laser_on()
counterlogic.startCount()

## Laser Off

In [ ]:
laser_off(pulser_on=False)
#counterl'o'gic.stopCount()


# Frequency Optimization parameters

In [ ]:
optimize_freq_dict=OrderedDict()
optimize_freq_dict['mw_optimization_method'] = 'pulsedodmr'
optimize_freq_dict['optimize_generate_new'] = False
optimize_freq_dict['save_tag'] = 'optimize_frequency'
optimize_freq_dict['optimize_fit_method'] = 'N15'
optimize_freq_dict['parameters2update'] = {'center': 'microwave_frequency'}

optimize_freq_dict['name'] = 'Pulsed_ODMR'
optimize_freq_dict['rabi_period'] = 1800e-9
optimize_freq_dict['freq_start'] = 4.127e9
optimize_freq_dict['freq_step'] = 0.2e6
optimize_freq_dict['num_of_points'] = 50
optimize_freq_dict['microwave_amplitude'] = common.mw_amp/15
optimize_freq_dict['meas_type'] = conventional_measurement
optimize_freq_dict['meas_info'] = add_conventional_information

optimize_freq_dict['measurement_time'] = 30
optimize_freq_dict['optimize_time'] = None
optimize_freq_dict['optimize_generate_new'] = True

#  Preperation mes

## Pulsed ODMR

In [ ]:
p_scale_manual = 1
f_center = 2.780e9

"""
common.rabi_shaped['shaped_on'] = False
if common.rabi_shaped['shaped_on']:
    logger.warning("pODMR does not support shaped pulses yet. Using potentially wrong calibrated pi pulse.")
"""

if p_scale_manual != 1:
    logger.warning("Manually reduce AWG ampl / AMP gain by a factor of {} / {}!".format(2*p_scale_manual, np.sqrt(2*p_scale_manual)))
    #qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=50, p_scale_onlyperiod=p_scale_manual, f_step=20e3) # range ~ 1 MHz
#qmeas = setup_podmr_params(f_res=800e6, p_scale=1, f_step=4e6, n_f=100)
#qmeas = setup_podmr_params(f_res=3e9, p_scale=1, f_step=3e6, n_f=150)     # wide band, ~ 150 MHz


common = common.swap_nvs([2,1])

qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=3e6, n_f=150)
#qmeas = setup_podmr_params(f_res=f_center, p_scale=1, f_step=2e6, n_f=100)
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=2000e3, n_f=200)     # wide band
#qmeas = setup_podmr_params(f_res=2.87e9, p_scale=1, f_step=3000e3, n_f=200)
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=3000e3, n_f=100)     
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=1000e3, n_f=150)
#qmeas =  setup_podmr_params(f_res=f_center, p_scale=1, f_step=3000e3, n_f=70)

#common.rabi_shaped['period'] = 80e-9
#qmeas = setup_podmr_params(f_res=2.77e9, p_scale=1, f_step=3000e3, n_f=100)
#qmeas = setup_podmr_params(f_res=2.88e9, p_scale=3, f_step=3000e3, n_f=200)
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=500e3, n_f=75)

#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=5, f_step=1000e3, n_f=75)     # wide band, ~ 150 MHz
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=5, f_step=500e3, n_f=75)

#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=20, f_step=250e3, n_f=100) # range ~ 5 MHz
#qmeas = setup_podmr_params(f_res=2.78e9, p_scale=20, f_step=200e3, n_f=100)#, t_pi_manual=8e-6)
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=30, f_step=100e3, n_f=120)

#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=40, f_step=250e3, n_f=75) # range ~ 5 MHz
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=40, f_step=100e3, n_f=100) # range ~ 3*6 MHz
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=60, f_step=50e3, n_f=100) # range ~ 1 MHz, 
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=60, f_step=100e3, n_f=150) # range ~ 1 MHz, 

# for extremely good T2* / Rabi drive efficiency
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=500, f_step=20e3, n_f=150) # range ~ 1 MHz, 
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=250, f_step=20e3, n_f=150) # range ~ 1 MHz, 
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=1500, f_step=20e3, n_f=250)


common = common.swap_nvs([1,2])
uglobals.abort.clear()
run_pdomr(qmeas)



### fine pODMR on 2x NVs

In [30]:
uglobals.abort.clear()

#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=4, f_step=0.8e6, n_f=70)
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=0.2e6, n_f=70)
qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=2, f_step=1.4e6, n_f=50)
qmeas['measurement_time'] = 4*60
qmeas['name'] = 'podmr_fine_A'

run_generic(qmeas)


qmeas = setup_podmr_params(f_res=common.mw_freq_2[0], p_scale=3, f_step=1.0e6, n_f=100)
qmeas = setup_podmr_params(f_res=common.mw_freq_2[0], p_scale=1, f_step=0.8e6, n_f=70)
qmeas = setup_podmr_params(f_res=common.mw_freq_2[0], p_scale=2, f_step=1.4e6, n_f=50)
qmeas['measurement_time'] = 4.0*60
qmeas['rabi_period'] = common.rabi_period_2[0] /(qmeas['microwave_amplitude']/common.mw_amp)
qmeas['microwave_amplitude'] = common.mw_amps_2[0] *qmeas['microwave_amplitude']/common.mw_amp
qmeas['name'] = 'podmr_fine_B'

run_generic(qmeas)
qmeas


OrderedDict([('name', 'podmr_fine_B'), ('experiment', 'pulsedodmr'), ('rabi_period', 1.258e-07), ('freq_step', 1400000.0), ('num_of_points', 50), ('freq_start', 2709400000.0), ('microwave_amplitude', 0.11574074074074073), ('measurement_time', 240.0), ('optimize_time', None), ('fit_experiment', 'lor_dip'), ('pulse_envelope', PulseEnvelopeType(sin_n))), ('savetag', 'podmr_fine_B'), ('gated', True), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 50), ('freq_optimize_time', None), ('analysis_interval', None), ('sampling_freq', 25000000000.0), ('bin_width', 4e-09), ('wait_time', 1e-06), ('laser_delay', 5e-07), ('laser_safety', 2e-07), ('sync_channel', 'd_ch1'), ('gate_channel', 'd_ch3'), ('laser_channel', 'd_ch4'), ('laser_length', 3e-06), ('wait_length', 1e-06), ('trigger_length', 2e-08), ('delay_length', 4.7e-07), ('channel_amp', 1.0), ('microwave_channel', 'a_ch1'), ('optimize_channel', None), ('readout_end', 3e-07), ('max_tau', 0.001), ('ma

In [ ]:
common.mw_freq

### power series

In [ ]:
uglobals.abort.clear()
pscales = np.linspace(1, 25, 10)[::]
common.mw_amp = 0.25
p_start = 1

for i, p in enumerate(pscales):
    if uglobals.abort.is_set():
        break
    try:
        p_phys = p_start*p
        qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=p_phys, f_step=500e3, n_f=75)

        #qmeas = setup_rabi_params(tau_step=250e-9, p_scale=10)
        qmeas['measurement_time'] = 40
        savetag = f"podmr_idx={i}_pscale={p_phys:.2f}"
        run_generic(qmeas, save_tag=savetag)
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))

logger.info("Autopilot has landed.")
            

In [ ]:
pscales

### pODMR + pi RF

In [ ]:
uglobals.abort.clear()
qmeas = setup_podmr_rfpi_params(f_res=common.mw_freq, p_scale=60, f_step=100e3, n_f=20,
                            f_rf=366.82e3, t_pi_rf=4.554e-6, ampl_rf=0.125, alternating_no_rf=True)
run_generic(qmeas)

In [ ]:
np.linspace(0.5, 1.5, 9)[::]

### t_pi series

In [ ]:
uglobals.abort.clear()
tscales = np.linspace(0.2, 1.8, 9)[::]
t_start = 4.554e-6

for i, t in enumerate(tscales):
    if uglobals.abort.is_set():
        break
    try:
        t_phys = t_start*t
        qmeas = setup_podmr_rfpi_params(f_res=common.mw_freq, p_scale=60, f_step=100e3, n_f=20,
                                    f_rf=366.82e3, t_pi_rf=4.554e-6, ampl_rf=0.125, alternating_no_rf=True)
        #qmeas = setup_rabi_params(tau_step=250e-9, p_scale=10)
        qmeas['measurement_time'] = 10*60
        savetag = f"podmr_rfpi_idx={i}_tpi={1e6*t_phys:.2f}uV"
        run_generic(qmeas, save_tag=savetag)
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))

logger.info("Autopilot has landed.")
            

## Rabi

In [ ]:
uglobals.abort.clear()
#uglobals.abort.set()

#qmeas = setup_rabi_params(tau_step=500*1e-9, p_scale=250, alternating=False, n_tau=200)   # 10e-9
#qmeas = setup_rabi_params(tau_step=25*1e-9, p_scale=25, alternating=False)   # 10e-9
#qmeas = setup_rabi_params(tau_step=1e-9, p_scale=1, alternating=False)   # 10e-9

#qmeas = setup_rabi_params(tau_step=common.rabi_period/20, n_tau=40, alternating=True) 
#qmeas = setup_rabi_params(tau_step=common.rabi_period/20, n_tau=75, alternating=True) 

"""
leave_out_idx = np.asarray([np.arange(20,100), np.arange(120,200), np.arange(220,300)]).flatten() 
qmeas = setup_rabi_params(tau_step=2.5e-9*2, n_tau=320, p_scale=2, leave_out_tau_idx=np.array2string(leave_out_idx,
                                                                                                      separator=',',
                                                                                                      max_line_width=9999))   # 10e-9
"""

qmeas = setup_rabi_params(tau_step=common.rabi_period/20, n_tau=550, alternating=False) 

# on nv2
#qmeas = setup_rabi_params(tau_step=common.t_rabi_2[0]/20, n_tau=75, alternating=False) 
#qmeas['microwave_frequency'] = common.mw_freq_2[0]
#qmeas['name'] = "rabi_B"

run_rabi(qmeas)


#qmeas


### on 2x NVs

In [33]:
uglobals.abort.clear()

qmeas = setup_rabi_params(tau_step=common.rabi_period/20, n_tau=75, alternating=False, p_scale=common.p_scale) 
qmeas['name'] = "rabi_A"
if (common.mw_freq < 2.87e9):
    qmeas['name'] = "rabi_A_ms=-1"
    qmeas['savetag'] = "rabi_A_ms=-1"
elif (common.mw_freq > 2.87e9):
    qmeas['name'] = "rabi_A_ms=1"
    qmeas['savetag'] = "rabi_A_ms=1"
else:
    qmeas['name'] = "rabi_A"
    qmeas['savetag'] = "rabi_A"
if common.rabi_shaped['shaped_on']:
    qmeas['name'] += "_shaped" 
    qmeas['savetag'] += "_shaped"
qmeas['measurement_time'] = 10*60

run_generic(qmeas)

#"""  
# on nv2
qmeas = setup_rabi_params(tau_step=common.rabi_period_2[0]/20, n_tau=75, alternating=False, p_scale=common.p_scale) 
qmeas['microwave_frequency'] = common.mw_freq_2[0]
qmeas['microwave_amplitude'] = common.mw_amps_2[0]/common.p_scale
qmeas['rabi_period'] = common.rabi_period_2[0]
qmeas['name'] = "rabi_B"
if (common.mw_freq_2[0] < 2.87e9):
    qmeas['name'] = "rabi_B_ms=-1"
    qmeas['savetag'] = "rabi_B_ms=-1"
elif (common.mw_freq_2[0] > 2.87e9):
    qmeas['name'] = "rabi_B_ms=1"
    qmeas['savetag'] = "rabi_B_ms=1"
else:
    qmeas['name'] = "rabi_B"
    qmeas['savetag'] = "rabi_B"
if common.rabi_shaped['shaped_on']:
    qmeas['name'] += "_shaped" 
    qmeas['savetag'] += "_shaped"
qmeas['measurement_time'] = 4*60

run_generic(qmeas)
#"""
qmeas


OrderedDict([('name', 'rabi_B_ms=-1_shaped'), ('experiment', 'rabi'), ('rabi_period', 6.29e-08), ('tau_step', 3.145e-09), ('tau_start', 0.0), ('alternating', False), ('leave_out_tau_idx', ''), ('microwave_frequency', 2744300000.0), ('num_of_points', 75), ('microwave_amplitude', 0.23148148148148145), ('measurement_time', 240), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('pulse_envelope', PulseEnvelopeType(sin_n))), ('savetag', 'rabi_B_ms=-1_shaped'), ('gated', True), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 75), ('analysis_interval', None), ('sampling_freq', 25000000000.0), ('bin_width', 4e-09), ('wait_time', 1e-06), ('laser_delay', 5e-07), ('laser_safety', 2e-07), ('sync_channel', 'd_ch1'), ('gate_channel', 'd_ch3'), ('laser_channel', 'd_ch4'), ('laser_length', 3e-06), ('wait_length', 1e-06), ('trigger_length', 2e-08), ('delay_length', 4.7e-07), ('channel_amp', 1.0), ('microwave_channel', 'a_ch

In [ ]:
common.rabi_period_2[0]/20
common.rabi_shaped['shaped_on']

### drive on mutliple transitions

In [ ]:
# drive all HFS lines
common.mw_freq = 2735.99e6
common.mw_amp = 0.25/2.2

ampl_mw2 = 0.25/1.8#[common.mw_amp]

f_mw1_add = [2725.11e6, 2729.41e6]
f_mw1 = sorted([common.mw_freq] + f_mw1_add)
f_mw2 = [2946.45e6, 2948.6e6, 2950.75e6]

# drive in left, right middle between 3 HFS lines
f_mw2 = [np.mean(f_mw2[0:2]), np.mean(f_mw2[1:])]
f_mw1 = [np.mean(f_mw1[0:2]), np.mean(f_mw1[1:])]



In [ ]:
common.mw_freq = 2735.99e6
common.mw_amp = 0.25/(2*1.3)

mw_freq_nv2 = 2820.47e6
mw_amp_nv2 = 0.25/1.6
t_rabi_nv2 = 162.1e-9#common.rabi_period

ampl_mw2 = mw_amp_nv2#[common.mw_amp]
f_mw2 = [mw_freq_nv2]
f_mw1 = [common.mw_freq]

mw_amp_nv2 + common.mw_amp

In [ ]:
# for calibration, drive transition 1,2 alternating
qmeas = setup_rabi_params(tau_step=1.5*0.5*common.rabi_period/20, n_tau=75,
                          f_mw2=f_mw2, f_mw1=f_mw1, ampl_mw2=ampl_mw2,
                          alternating=True)

# do DQT
#qmeas = setup_rabi_params(tau_step=3*common.rabi_period/20, n_tau=75,
#                          f_mw2=f_mw2, ampl_mw2=ampl_mw2,
#                          alternating=False)

uglobals.abort.clear()
run_rabi(qmeas)
#qmeas

In [ ]:
f_mw1

In [ ]:

uglobals.abort.clear()
pscales = np.linspace(1, 0.05, 75)#[::-1]


common.mw_freq = 2727.26e6
common.mw_amp = 0.25/1.8

f_mw1_add = [2725.11e6, 2729.41e6]
f_mw2 = [2946.45e6, 2948.6e6, 2950.75e6]
ampl_mw2 = 0.25#[common.mw_amp]

p_start_1 = common.mw_amp
p_start_2 = ampl_mw2


for i, p in enumerate(pscales):
    if uglobals.abort.is_set():
        break
    try:
        p_phys_1 = p_start_1*p
        common.mw_amp = p_phys_1
        p_phys_2 = p_start_2*p
        
        #qmeas = setup_rabi_params(tau_step=2*common.rabi_period/20, n_tau=75,
        #                  f_mw2=f_mw2, ampl_mw2=p_phys_2,
        #                  alternating=False)
        
        qmeas = setup_rabi_params(tau_step=2*common.rabi_period/20, n_tau=200,
                           f_mw2=f_mw2, ampl_mw2=p_phys_2, f_mw1_add=f_mw1_add,
                           alternating=True)
        
        #qmeas = setup_rabi_params(tau_step=250e-9, p_scale=10)
        qmeas['measurement_time'] = 300
        qmeas['fit_experiment'] = "sine_2x"
        savetag = f"rabi_multi_idx={i}_ampl1={1e3*p_phys_1:.2f}mV_ampl2={1e3*p_phys_2:.2f}mV"
        run_generic(qmeas, save_tag=savetag)
        
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))

logger.info("Autopilot has landed.")
            

In [ ]:

uglobals.abort.clear()
dfs = np.linspace(-5e6, 5e6, 20)#[::-1]
p_start = common.mw_amp

for i, df in enumerate(dfs):
    if uglobals.abort.is_set():
        break
    try:
        f2 = f_mw2[0] + df
        qmeas = setup_rabi_params(tau_step=2*common.rabi_period/20, n_tau=75,
                          f_mw2=[f2], ampl_mw2=ampl_mw2,
                          alternating=False)
        del(qmeas['alternating_mode'])
        #qmeas = setup_rabi_params(tau_step=250e-9, p_scale=10)
        qmeas['measurement_time'] = 60
        qmeas['fit_experiment'] = "sine_2x"
        savetag = f"rabi_multi_idx={i}_df2={1e-3*df:.2f}MHz"
        run_generic(qmeas, save_tag=savetag)
        
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))

logger.info("Autopilot has landed.")
            

In [ ]:
uglobals.abort.clear()
dfs = np.linspace(-5e6, 5e6, 20)#[::-1]
p_start = common.mw_amp

for i, df in enumerate(dfs):
    if uglobals.abort.is_set():
        break
    try:
        f2 = f_mw2[0] + df
        qmeas = setup_rabi_params(tau_step=2*common.rabi_period/20, n_tau=75,
                          f_mw2=[f2], ampl_mw2=ampl_mw2,
                          alternating=False)
        del(qmeas['alternating_mode'])
        #qmeas = setup_rabi_params(tau_step=250e-9, p_scale=10)
        qmeas['measurement_time'] = 60
        qmeas['fit_experiment'] = "sine_2x"
        savetag = f"rabi_multi_idx={i}_df2={1e-3*df:.2f}MHz"
        run_generic(qmeas, save_tag=savetag)
        
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))

logger.info("Autopilot has landed.")

### power series

In [35]:
uglobals.abort.clear()
pscales = np.linspace(1.1, 0.8, 6)[::-1]


common = common.swap_nvs([2,1])
p_start = common.mw_amp


for i, p in enumerate(pscales):
    if uglobals.abort.is_set():
        break
    try:
        p_phys = p_start*p/common.p_scale
        qmeas = setup_rabi_params(tau_step=common.rabi_period/20,
                                  n_tau=75, p_scale=p_start/p_phys, alternating=False)   # 10e-9
        #qmeas = setup_rabi_params(tau_step=250e-9, p_scale=10)
        qmeas['measurement_time'] = 15*60
        qmeas['fit_experiment'] = 'sine_3x'
        
        savetag = f"rabi_idx={i}_ampl={1e3*p_phys:.2f}mV"
        run_generic(qmeas, save_tag=savetag)
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))

common = common.swap_nvs([1,2])

qmeas, p, p_phys, p_start
            

(OrderedDict([('name', 'rabi'), ('experiment', 'rabi'), ('rabi_period', 9.528e-07), ('tau_step', 4.764e-08), ('tau_start', 0.0), ('alternating', False), ('leave_out_tau_idx', ''), ('microwave_frequency', 2744300000.0), ('num_of_points', 75), ('microwave_amplitude', 0.008085), ('measurement_time', 900), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('pulse_envelope', PulseEnvelopeType(rectangle))), ('savetag', 'rabi_idx=3_ampl=8.09mV'), ('gated', True), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 75), ('analysis_interval', None), ('sampling_freq', 25000000000.0), ('bin_width', 4e-09), ('wait_time', 1e-06), ('laser_delay', 5e-07), ('laser_safety', 2e-07), ('sync_channel', 'd_ch1'), ('gate_channel', 'd_ch3'), ('laser_channel', 'd_ch4'), ('laser_length', 3e-06), ('wait_length', 1e-06), ('trigger_length', 2e-08), ('delay_length', 4.7e-07), ('channel_amp', 1.0), ('microwave_channel', 'a_ch1'), ('optimize_c

In [ ]:
add_shaped(qmeas)

## Stability

In [ ]:
uglobals.abort.clear()


idx = 0
while True:
    if uglobals.abort.is_set():
        break
    try:
        #pass
        #"""
        laser_on()
        time.sleep(1)
        optimize_position()
        optimize_position()
        laser_off()
        #"""
    except Exception as e:
        logger.warning("auto run {}, skipping failed tracking: {}".format(i, str(e)))
    if uglobals.abort.is_set():
        break
    
    logger.info(f"Starting mes {idx} in stability test")
    qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=5, f_step=0.8e6, n_f=70)
    qmeas['measurement_time'] = 1.5*60
    qmeas['name'] = 'podmr_fine_A'
    run_generic(qmeas)

    qmeas = setup_rabi_params(tau_step=common.rabi_period/20, n_tau=75, alternating=False) 
    qmeas['name'] = "rabi_A"
    if common.rabi_shaped['shaped_on']:
        qmeas['name'] += "_shaped" 
    qmeas['measurement_time'] = 1.5*60

    run_generic(qmeas)
    idx += 1

        
  

## Nuclear (15-N)

### nuclear pODMR

In [ ]:
t_nuc_rabi = 15.8e-6
rf_ampl = 40e-3

qmeas = setup_nuc_podmr_params(f_start=2.6e6,  f_step=10e3, n_f=50, t_nuc_rabi=t_nuc_rabi, rf_ampl=rf_ampl)

uglobals.abort.clear()
run_generic(qmeas)

In [ ]:
qmeas

### nuclear Rabi

In [ ]:
f_nuc_rabi = 2814e3

qmeas = setup_nuc_rabi_params(tau_step=1e-6, rf_freq=2.81e6, rf_ampl=rf_ampl, n_tau=50)
savetag = f"nucRabi_{round(qmeas['rf_ampl']*1000)}mV"

uglobals.abort.clear()


In [ ]:
#run_generic(qmeas, save_tag=savetag)

### power series

In [ ]:
uglobals.abort.clear()
pscales = np.linspace(1, 0.1, 8)[::]
p_start = 60e-3 #common.mw_amp

for i, p in enumerate(pscales):
    if uglobals.abort.is_set():
        break
    try:
        p_phys = p_start*p
        qmeas = setup_nuc_rabi_params(tau_step=t_nuc_rabi/20*rf_ampl/p_phys,
                                     rf_freq=f_nuc_rabi, rf_ampl=p_phys, n_tau=50)   # 10e-9
        #qmeas = setup_rabi_params(tau_step=250e-9, p_scale=10)
        qmeas['measurement_time'] = 40
        savetag = f"nucRabi_idx={i}_ampl={1e3*p_phys:.2f}mV"
        run_generic(qmeas, save_tag=savetag)
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))

logger.info("Autopilot has landed.")
            

## Ramsey


In [54]:
tau_start = 0.05e-6
n_sweeps = 0#1e3

uglobals.abort.clear()
qmeas = setup_ramsey_params(tau_start, n_tau=50, tau_step=20e-9, alternating=True)
#qmeas = setup_ramsey_params(tau_start, n_tau=70, tau_step=25e-9, alternating=True, read_phases='-90, +90', red_readout=False)
#qmeas = setup_ramsey_params(tau_start, n_tau=50, tau_step=50e-9, alternating=True, read_phases='-90, +90', red_readout=False)

#qmeas = setup_ramsey_params(tau_start, n_tau=50, tau_step=20e-9, alternating=True, read_phases='0, 180', red_readout=False)

#qmeas = setup_ramsey_params(tau_start, n_tau=100, tau_step=100e-9)     
#qmeas = setup_ramsey_params(tau_start, n_tau=120, tau_step=20e-9, alternating=True)
#qmeas = setup_ramsey_params(tau_start, n_tau=200, tau_step=250e-9)      # 75ns: res like mfl
#qmeas = setup_ramsey_params(tau_start, n_tau=400, tau_step=50e-9)     # high res    
#qmeas = setup_ramsey_params(tau_start, n_tau=400, tau_step=100e-9)     # high res  

# debug
#qmeas = setup_ramsey_params(300e-9, n_tau=1, tau_step=20e-9, alternating=True, read_phases='-90, +90', red_readout=False)


x, z = run_ramsey(qmeas, n_sweeps_stop=n_sweeps)
z_binary = 0 #majority_vote(z[0], z_thres=common.mfl_z_thres)

logger.info("Ramsey with offset {} MHz, {} sweeps: {} -> {}".format(common.mw_ramsey_offset/1e6, n_sweeps, z, z_binary))
qmeas

OrderedDict([('name', 'Ramsey'), ('experiment', 'ramsey_s3p'), ('rabi_period', 1.385e-06), ('microwave_frequency', 2615600000.0), ('microwave_amplitude', 0.0075), ('tau_start', 5e-08), ('tau_step', 2e-08), ('num_of_points', 50), ('alternating', True), ('from_list', False), ('optimize_time', None), ('fit_experiment', 'sine_decay'), ('read_phases_degree', '0, 180'), ('gated', True), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 100), ('measurement_time', None), ('freq_optimize_time', None), ('analysis_interval', None), ('sampling_freq', 25000000000.0), ('bin_width', 4e-09), ('wait_time', 1e-06), ('laser_delay', 5e-07), ('laser_safety', 2e-07), ('sync_channel', 'd_ch1'), ('gate_channel', 'd_ch3'), ('laser_channel', 'd_ch4'), ('laser_length', 3e-06), ('wait_length', 1e-06), ('trigger_length', 2e-08), ('delay_length', 4.7e-07), ('channel_amp', 1.0), ('microwave_channel', 'a_ch1'), ('optimize_channel', None), ('readout_end', 3e-07), ('max_tau',

### Ramsey for (AC Stark) crosstalk

In [ ]:
tau_start = 0.05e-6

f_mw_2 = common.mw_freq_2[0] - 20e6  # can't use NV2, as we're also reading it!
ampls_2 = np.linspace(0, common.mw_amps_2[0], 8)[::-1]
t_rabi_2 = common.rabi_period_2[0]


uglobals.abort.clear()
try:
    for ampl_2 in ampls_2:
                
        qmeas = setup_ramsey_params(tau_start, n_tau=int(0.75*100), tau_step=1/0.75*15e-9, alternating=True)
        qmeas['experiment'] = 'ramsey_crosstalk'
        qmeas['savetag'] = f'ramsey_crosstalk_ampl2={1e3*ampl_2:.1f}mV'
        qmeas['measurement_time'] = 10*60


        qmeas['f_mw_2'] = list_2_csv(f_mw_2)
        qmeas['ampl_mw_2'] = list_2_csv(ampl_2)

        run_generic(qmeas)

    # reference Ramsey
    qmeas = setup_ramsey_params(tau_start, n_tau=100, tau_step=15e-9, alternating=True)
    qmeas['measurement_time'] = 15*60
    run_generic(qmeas)
except:
    logger.exception("")
qmeas

### Compensation of AC Stark

In [ ]:
def add_ac_stark_compensation(qmeas, f_mw_2=None, ampl_comp=[0,0], df_offset=[0e6,0e6], fac_comp=5):
    """
    @param list ampl_comp: amplitude for [0] compensation of effect of f2 on f1. [1] f1 on f2.
    @param float df_offset: linearly add offset to compensation, eg. to balance calibration taken at different f2
    """
    if not 'f_mw_2' in qmeas.keys() or not 'ampl_mw_2' in qmeas.keys():
        raise ValueError("Supplied mes dict doesn't support multi frequency pulses.")

    f_mw_1 = common.mw_freq
    if f_mw_2 == None:
        f_mw_2 = csv_2_list(qmeas['f_mw_2']) 
    if len(f_mw_2) > 1:
        raise NotImplementedError("Jupyter notebook atm only supports 1 frequency per NV")
    f_mw_2 = f_mw_2[0] 
    
    delta_stark_symm_2on1 = f_mw_2 - f_mw_1 
    delta_stark_2on1 = delta_stark_symm_2on1/fac_comp
    f_compensation_2on1 = f_mw_1 - delta_stark_2on1 + df_offset[0]
    
    delta_stark_symm_1on2 = f_mw_1 - f_mw_2 
    delta_stark_1on2 = delta_stark_symm_1on2/fac_comp
    f_compensation_1on2 = f_mw_2 - delta_stark_1on2 + df_offset[1]
    
    # order of fs is important to match t_rabi in pulse generation
    # pi_on_1 requires in parallel f_comp_1on2
    qmeas['f_mw_2'] = list_2_csv([f_compensation_1on2, f_mw_2, f_compensation_2on1])
    ampl_2 = csv_2_list(qmeas['ampl_mw_2'])[0]
    t_rabi_2 = csv_2_list(qmeas['rabi_period_mw_2'])[0]
    
    qmeas['ampl_mw_2'] = list_2_csv([ampl_comp[1], ampl_2, ampl_comp[0]])
    # same length for compensation pulse and actually intended pulse
    qmeas['rabi_period_mw_2'] = list_2_csv([common.rabi_period, t_rabi_2, t_rabi_2])
    
    return qmeas

tau_start = 0.05e-6

common = common.swap_nvs([1,2])

df_calib_offet = -15e6
f_mw_2 = common.mw_freq_2[0] + df_calib_offet
ampl_2 = common.mw_amps_2[0]

ampl_acstark_comp = np.sqrt(2)*0.032
fac_comp = 3

uglobals.abort.clear()
try:
    qmeas = setup_ramsey_params(tau_start, n_tau=int(80), tau_step=1/0.75*15e-9, alternating=True)
    qmeas['experiment'] = 'ramsey_crosstalk'
   
    qmeas['measurement_time'] = 1000*60
    # create AC Stark bias field (at some offset from NV2 resonance)
    qmeas['f_mw_2'] = list_2_csv([f_mw_2])
    qmeas['ampl_mw_2'] = list_2_csv([ampl_2])
    qmeas['rabi_period_mw_2'] = list_2_csv([ampl_2])
 
    qmeas = add_ac_stark_compensation(qmeas, ampl_comp=[ampl_acstark_comp,0], 
                                  fac_comp=fac_comp, df_offset=[0,0])
    qmeas['savetag'] = f'ramsey_crosstalk_ampl2={1e3*ampl_2:.1f}mV_amplComp={1e3*ampl_acstark_comp:.1f}mV'

    run_generic(qmeas)

except:
    logger.exception("")
qmeas

In [ ]:
# compensation from resonance
(csv_2_list(qmeas['f_mw_2'])[2] - common.mw_freq)/1e6

In [ ]:
# power series (of compensation)

In [ ]:
(np.concatenate([np.linspace(4/5*ampl_comp_max, ampl_comp_max, 6)])-5*0.004)

In [ ]:
tau_start = 0.05e-6

common = common.swap_nvs([2,1])




# place the compensation with low power close to f_mw1 (instead of same power with opposite detuning)
# in this way, ac stark on other nv will be lower (due to fraction of delta_starks)
fac_comp_stark = 3
df_calib_offet = -15e6

f_mw_2 = common.mw_freq_2[0] + df_calib_offet
ampl_2 = common.mw_amps_2[0]
t_rabi_2 = common.rabi_period_2[0]

# set up compensation
delta_stark_symm = f_mw_2 - common.mw_freq
delta_stark = delta_stark_symm/fac_comp_stark
f_compensation = common.mw_freq - delta_stark
ampl_comp_max = 0.25 - ampl_2
ampl_comp_max = 0.11
ampls_comp = np.linspace(0, ampl_comp_max, 12)#[::-1]
ampls_comp = np.linspace(4/5*ampl_comp_max, ampl_comp_max, 6)#[::-1]
ampls_comp = (np.linspace(4/5*ampl_comp_max, ampl_comp_max, 6)-5*0.0044)[::-1]
#ampls_comp = np.ravel([ampls_comp[::2], ampls_comp[1::2][::-1]],order='F')
# this is the expected point of good compensation
ampl_comp_theo = np.sqrt(1/fac_comp_stark)*ampl_2

logger.info(f"AC Stark compensation at delta= {delta_stark/1e6:.1f} MHz, "\
            f"ampl_theo={1e3*ampl_comp_theo:.1f} mV, ampl<= {1e3*ampl_comp_max:.1f} mV")

uglobals.abort.clear()
try:
    for ampl_3 in ampls_comp:  
        
        qmeas = setup_ramsey_params(tau_start, n_tau=int(100), tau_step=1/0.75*15e-9, alternating=True)
        qmeas['experiment'] = 'ramsey_crosstalk'
        qmeas['savetag'] = f'ramsey_crosstalk_ampl2={1e3*ampl_2:.1f}mV_amplComp={1e3*ampl_3:.1f}mV'
        qmeas['measurement_time'] = 20*60
        # create AC Stark bias field (at some offset from NV2 resonance)
        qmeas['f_mw_2'] = list_2_csv([f_mw_2])
        qmeas['ampl_mw_2'] = list_2_csv([ampl_2])
        # turn compensation on
        qmeas['f_mw_2'] = list_2_csv([0, f_mw_2, f_compensation])
        qmeas['ampl_mw_2'] = list_2_csv([0, ampl_2, ampl_3])

        run_generic(qmeas)

    # reference Ramsey
    #qmeas = setup_ramsey_params(tau_start, n_tau=int(0.75*100), tau_step=1/0.75*15e-9, alternating=True)
    #qmeas['measurement_time'] = 15*60
    #run_generic(qmeas)
except:
    logger.exception("")
qmeas

In [ ]:
delta_stark/1e6, common.mw_freq, f_mw_2

## Hahn


In [ ]:
uglobals.abort.clear()

he_tau_step = 0.250e-6
he_n_tau = 25
#he_n_tau = 50

logger.info(f"Tau max= {he_tau_step*he_n_tau}")

qmeas = setup_hahn_params(tau_start=100e-9, n_tau=he_n_tau, tau_step=he_tau_step, alternating=True)
#qmeas = setup_hahn_params(tau_start=14e-6, n_tau=he_n_tau, tau_step=he_tau_step, alternating=True)
run_generic(qmeas)

## XY 8 

In [ ]:
common.__dict__

tau sweep

In [ ]:
#tau_start = 0.2e-6
tau_start = 50e-9
tau_start = 10e-9

uglobals.abort.clear()
# narrow resonance NV M17
qmeas = setup_xy8_tausweep_params(tau_start, n_tau=30, tau_step=50e-9, xy8_order=4, alternating=True)
qmeas['measurement_time'] = 20*60
run_xy8_tausweep(qmeas)

common = common.swap_nvs([2,1])
qmeas = setup_xy8_tausweep_params(tau_start, n_tau=30, tau_step=50e-9, xy8_order=4, alternating=True)
qmeas['measurement_time'] = 20*60
common = common.swap_nvs([1,2])



#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=10, tau_step=25e-9, xy8_order=800, alternating=True)


# larger sweeps
#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=75, tau_step=10e-9, xy8_order=4, alternating=True) 
#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=75, tau_step=3e-9, xy8_order=4, alternating=True) 
#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=250, tau_step=20e-9, xy8_order=4, alternating=True)
#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=75, tau_step=5e-9, xy8_order=8, alternating=True) 

#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=50, tau_step=100e-9, xy8_order=8, alternating=True)


run_xy8_tausweep(qmeas)

n sweep

In [ ]:
tau = 900e-9#440.0e-9
tau = 100e-9#440.0e-9
#tau = 441.6e-9

uglobals.abort.clear()
#qmeas = setup_xy8_nsweep_params(tau, n_n=25, xy8_start=4, xy8_step=1, alternating=True)
#qmeas = setup_xy8_nsweep_params(tau, n_n=15, xy8_start=4, xy8_step=100, alternating=True)  # long t_evol, mem limit
qmeas = setup_xy8_nsweep_params(tau, n_n=10, xy8_start=1, xy8_step=1, alternating=True)  
qmeas['name'] = 'xy8_n'
run_xy8_nsweep(qmeas)

In [ ]:
qmeas

In [ ]:
a few xy8 tau sweeps

In [ ]:
uglobals.abort.clear()

tau_start = 0.5e-6
xy_orders = np.arange(5, 16, 1)

for i, o in enumerate(xy_orders):
    if uglobals.abort.is_set():
        break
    try:
        #pass
        #"""
        laser_on()
        time.sleep(1)
        optimize_position()
        optimize_position()
        laser_off()
        #"""
    except Exception as e:
        logger.warning("auto XY8 run {}, skipping failed tracking: {}".format(i, str(e)))
    if uglobals.abort.is_set():
        break
        
    qmeas = setup_xy8_tausweep_params(tau_start, n_tau=50, tau_step=20e-9, xy8_order=o, alternating=True)     
    qmeas['measurement_time'] = 180
    run_xy8_tausweep(qmeas)

### DD DQT

In [ ]:
# drive all HFS lines
common.mw_freq = 2727.26e6
common.mw_amp = 0.25/1.8


t_rabi_dqt = 93.8e-9
t_rabi1 = 97.2e-9  # driving in between HFS lines
t_rabi1 = 93.56e-9  # driving only central line
ampl_mw2 = 0.25#[common.mw_amp]

f_mw1_add = [2725.11e6, 2729.41e6]
#f_mw1 = sorted([common.mw_freq] + f_mw1_add)
f_mw1 = [common.mw_freq] 
f_mw2 = [2946.45e6, 2948.6e6, 2950.75e6]

# drive in left, right middle between 3 HFS lines
#f_mw2 = [np.mean(f_mw2[0:2]), np.mean(f_mw2[1:])]
#f_mw1 = [np.mean(f_mw1[0:2]), np.mean(f_mw1[1:])]



In [ ]:
f_mw1, f_mw2

In [ ]:
# only lower drive (with potential multiple drives for HFS), no DQT
qmeas = setup_dd_dqt_sigamp_params(tau=200e-9, n_amp=50, amp_step=5e-3,
                                    dd_order=1, t_rabi1=t_rabi1,  
                      f_mw1=f_mw1, f_mw2=[], ampl_mw2=0,
                      dd_type=DDMethods.XY8, alternating=True)
#qmeas['amp_start'] = 20e-3

uglobals.abort.clear()
run_generic(qmeas)

In [ ]:
qmeas = setup_dd_dqt_sigamp_params(tau=200e-9, n_amp=50, amp_step=0.002e-3,
                                    dd_order=1, t_rabi1=t_rabi_dqt,  
                      f_mw1=f_mw1, f_mw2=f_mw2, ampl_mw2=ampl_mw2,
                      dd_type=DDMethods.XY8, alternating=True)

uglobals.abort.clear()
#run_generic(qmeas)

In [ ]:
qmeas

## Hartmann Hahn

In [ ]:
uglobals.abort.clear()

qmeas = setup_hh_tau_params(tau_start=0e-6, tau_step=0.6e-6, n_tau=50, amp_sl=1.35e-3)


run_generic(qmeas)

### power series

In [ ]:
uglobals.abort.clear()
pscales = np.linspace(1, 0.68, 20)
p_start_sl = 1.6e-3  # 1.65 ~ 523 kHz = Lamor

for i, p in enumerate(pscales):
    if uglobals.abort.is_set():
        break
    try:

        qmeas = setup_hh_tau_params(tau_start=0e-6, tau_step=0.65e-6, n_tau=50, amp_sl=p_start_sl*p)
        #qmeas = setup_rabi_params(tau_step=250e-9, p_scale=10)
        qmeas['measurement_time'] = 2*60
        savetag = f"hh_tau_idx={i}_ampl_lock={1e3*p_start_sl*p:.2f}mV"
        run_generic(qmeas, save_tag=savetag)
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))

logger.info("Autopilot has landed.")

Double resonance

## DEER

In [ ]:
# deprecated, better use params in common.(..)

he_tau = 0.3e-6

t_rabi_2 = 100e-9#200.0e-9
f_center = 100e6#2.8307e9

common

### Spectrum

In [ ]:
uglobals.abort.clear()

f_center = 1.407e9
f_center = 3.05e9
#f_center = 1.305e9
#f_center = 0.735e9
#f_center = 0.850e9
#t_pi_fix = 6000e-9   # if already calibrated by DEER Rabi
t_pi_fix = None #138.0e-9/2    # else

# wide band
#qmeas = setup_deer_f_params(f_center, f_step=5e6, n_f=100, two_deer_pi=True,
#                            pi_len=t_pi_fix)
#qmeas = setup_deer_f_params(f_center, f_step=3e6, n_f=150, two_deer_pi=True,
#                            he_tau=common.he_tau, pi_len=t_pi_fix, read_phases='-90, 90')
qmeas = setup_deer_f_params(f_center, f_step=3e6, n_f=50, two_deer_pi=True,
                            he_tau=common.he_tau, pi_len=t_pi_fix, read_phases='0, 180')
#qmeas = setup_deer_f_params(f_center, f_step=6e6, n_f=75, two_deer_pi=False,
#                            he_tau=common.he_tau, pi_len=t_pi_fix, read_phases='-90, 90', deer_ampl=0.25/5.)
#qmeas = setup_deer_f_params(f_center, f_step=1e6, n_f=200, two_deer_pi=False,
#                            pi_len=t_pi_fix)
# high res
#qmeas = setup_deer_f_params(f_center, f_step=500e3, n_f=75, two_deer_pi=True,
#                           he_tau=common.he_tau, pi_len=t_pi_fix)
# wide band
#qmeas = setup_deer_f_params(f_center, f_step=2000e3, n_f=120, two_deer_pi=True,
#                           he_tau=300e-9)

#run_deer_f(qmeas)

### DD Spectrum

Remember to set common.mw_freq and rabi to the correct NV

In [ ]:
# main resonance
# {'shaped_on':True, 'period': 244.5e-9, 'env_type': Evm.parabola, 'order_P':1}
#        self.mw_freq = 3086.1e6 #3026
#t_rabi_2 = 2*t_pi_fix if t_pi_fix else None
#common.he_tau = 0.2e-6

f_center = common.mw_freq_2[0]
dd_tau = common.dd_tau
dd_order = common.dd_order
dd_type = common.dd_type #DDMethods.XY8
t_rabi_2 = common.rabi_period_2[0]

try:
    qmeas = setup_deer_dd_f_params(f_center, dd_tau=dd_tau, f_step=3e6, n_f=50,
                                   t_rabi_2=t_rabi_2, dd_type=dd_type, dd_order=dd_order)
    #qmeas = setup_deer_dd_f_params(f_center, dd_tau=common.he_tau, f_step=3e6, n_f=2,
    #                               t_rabi_2=t_rabi_2, dd_type=DDMethods.SE, dd_order=2)
    #qmeas['order_P'] = 10
    #qmeas['mw_amp2'] = 0
except:
    logger.exception("")
uglobals.abort.clear()
run_generic(qmeas)

In [ ]:
qmeas

### DD tau / CNOT

Implement CNOT by read_phase= 90! Mind that this works better with pulse shaping turned on, especially if nuclei are around.
ATTENTION: In mixed rect/shaped mode, for calibration of CNOT shaping must be turned on!

In [ ]:
common.dd_tau, common.dd_order,  common.dd_type#, -tau2_pspacing/2

In [102]:
common.rabi_period
tau2_start
#tau2_pspacing
#dd_tau

-4.619999999999999e-07

In [76]:
#common.he_tau = 0.2e-6
#t_rabi_2 = 196.5e-9
#f_deer= f_center

# to switch between C2NOT1 (order=1,2) and C1NOT2
common = common.swap_nvs(order=[1,2])

f_deer = common.mw_freq_2[0]
ampl_2 = common.mw_amps_2[0]
t_rabi_2 = common.rabi_period_2[0]
dd_tau = common.dd_tau  # -1 for deer_dd_par
#dd_tau = -1
dd_order = common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8


# Test contrast
# may reduce dd_order to see whether coherence is as expected
qmeas = setup_deer_dd_tau_params(f_deer, dd_tau=dd_tau, tau2_start=0e-9, tau2_step=-6e-9, n_tau=30,
                             t_rabi_2=t_rabi_2, deer_ampl=ampl_2, dd_type=dd_type, dd_order=4,  # dd_order  
                             read_phase=0) 
qmeas['tau2_rel_to_pi1'] = False
#qmeas['ampl_mw_2'] = '0'

#tau2_pspacing = dd_tau-common.rabi_period/2-t_rabi_2/2
tau2_pspacing = dd_tau-common.rabi_period
# for optimal control pulses of length 100ns
#tau2_pspacing = dd_tau-5*100e-9
#tau2_pspacing = dd_tau-1*100e-9
#tau2_pspacing = dd_tau

if dd_tau > 0:
    n_tau = 20
    tau2_step = tau2_pspacing/n_tau
    tau2_start = -tau2_pspacing/2
    #n_tau = 20
    #tau2_step = tau2_pspacing/(2*n_tau)
    #tau2_start = 0#-tau2_pspacing/2
    n_tau = 20
    tau2_step = tau2_pspacing/(2*n_tau)
    tau2_start = -tau2_pspacing/2
else:
    n_tau = 4*40
    tau2_start = common.rabi_period/2
    tau2_step = (1/70e3/(common.dd_order*common.dd_type.suborder))/(n_tau)
    n_tau = 20
    tau2_start = 0
    tau2_step = (1/70e3/(common.dd_order*common.dd_type.suborder))/(2*n_tau)

uglobals.abort.clear()
inits = [[0.5,0.]]#,[0,1],[1,0],[1,1]]#[::-1]

#inits = [[0.5,0], [0.5,0],[0.5,0], [0.5,0], [0.5,0.5], [0.5,0.5],[0.5,0.5], [0.5,0.5]]#[::-1]
#ends = [[0.5, 0], [0.5, 0], [0, 0.5], [0, 0.5],[0.5, 0], [0.5, 0], [0, 0.5], [0, 0.5]]
#ends_phi = [0, 90, 0, 90, 0, 90, 0, 90]
for idx, (init_1, init_2) in enumerate(inits):
    qmeas = setup_deer_dd_tau_params(f_deer, dd_tau=dd_tau, 
                                 tau2_start=tau2_start, tau2_step=tau2_step, n_tau=n_tau, 
                             t_rabi_2=t_rabi_2, deer_ampl=ampl_2, dd_type=dd_type, dd_order=dd_order, ## dd_order
                             read_phase=90) #90 for CNOT
    qmeas['tau2_rel_to_pi1'] = False

    qmeas['init_pix_on_1'] = init_1  #1
    qmeas['init_pix_on_2'] = init_2  #1
    qmeas['end_pix_on_2'] = 1  #1

    
    # for shiting
    #qmeas['env_type_1'] = Evm.optimal
    #qmeas['env_type_2'] = Evm.optimal
    #qmeas['env_type_1'] = Evm.parabola
    #qmeas['env_type_2'] = Evm.parabola
    # for parallel
    #qmeas['env_type'] = Evm.optimal
    
   

    # AC Stark compensation, get values from doing a calibration with AC Stark Ramsey
    # ATTENTION: adding compensation must not exceed maximum MW amplitude.
    # Safe to do, if NV1/2 pulses don't overlap.
    #ampl_acstark_comp_2on1 = 0.045
    #ampl_acstark_comp_1on2 = 0#0.075
    #df_calib_offset = -15e6
    #df_offset_2on1 = (ampl_acstark_comp_2on1/common.mw_amp)**2*df_calib_offset
    #df_offset_1on2 = (ampl_acstark_comp_1on2/common.mw_amps_2[0])**2*df_calib_offset
    #fac_comp = 3

    #qmeas = add_ac_stark_compensation(qmeas, ampl_comp=[ampl_acstark_comp_2on1,ampl_acstark_comp_1on2], 
    #                                 fac_comp=fac_comp, df_offset=[df_offset_2on1,df_offset_1on2])

    # cphase like
    qmeas['start_pix_on_1'] = 0
    #qmeas['init_pix_on_1'] = 0.5
    #qmeas['init_pix_on_2'] = 0.5
    qmeas['end_pix_on_1'] = ends[idx][0]
    qmeas['read_pix_on_2'] = ends[idx][1]

    qmeas['read_phase_deg'] = 0  #ends_phi[idx]
    qmeas['dd_order'] = 2*dd_order
    # for floating last pi
   
    #qmeas['microwave_amplitude'] = 0.1
    #qmeas['dd_order'] = 1
    #qmeas['dd_type'] = DDMethods.CPMG
    #qmeas['num_of_points'] = 1
    

    qmeas['measurement_time'] = 30*60
    qmeas['optimize_time'] = 20*60
    qmeas['optimize_func_toggle_pause'] = toggle_pg
    
    qmeas['alternating'] = True
    #qmeas['generate_new'] = False  # todo: not working
    
    qmeas['savetag'] = f"deer_dd_{dd_type.name}-{dd_order}"\
                       f"_init={init_1}_{init_2}_read={qmeas['end_pix_on_1']},{qmeas['read_pix_on_2']}"\
                       f"_readph={qmeas['read_phase_deg']}"
    qmeas['savetag'] = qmeas['savetag'] + "_fci" if 'fci' in qmeas['experiment'] else qmeas['savetag']
    
    run_generic(qmeas)

#qmeas = setup_deer_dd_tau_params(f_deer, dd_tau=dd_tau, 
#                                 tau2_start=-tau2_pspacing/2, tau2_step=tau2_pspacing/40, n_tau=40, # 75
#                             t_rabi_2=t_rabi_2, deer_ampl=ampl_2, dd_type=DDMethods.XY8, dd_order=4, ## dd_order
#                             read_phase=90) #90 for CNOT
#qmeas['tau2_rel_to_pi1'] = False
#qmeas['end_pix_on_2'] = 1  #1
#qmeas['init_pix_on_2'] = 0  #1

qmeas

OrderedDict([('experiment', 'deer_dd_tau'), ('name', 'deer_dd_rect_tau'), ('microwave_frequency', 3129400000.0), ('microwave_amplitude', 0.23148148148148145), ('rabi_period', 7.55e-08), ('dd_type', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('dd_order', 2), ('num_of_points', 20), ('tau_start', -7.1225e-07), ('tau_step', 3.56125e-08), ('f_mw_2', '2744300000.0'), ('rabi_period_mw_2', '6.29e-08'), ('ampl_mw_2', '0.23148148148148145'), ('read_phase_deg', 0), ('tau1', 1.5e-06), ('measurement_time', 1800), ('optimize_time', 1200), ('alternating', True), ('pulse_envelope', PulseEnvelopeType(sin_n))), ('tau2_rel_to_pi1', False), ('init_pix_on_1', 0.5), ('init_pix_on_2', 0.0), ('end_pix_on_2', 1), ('start_pix_on_1', 0), ('end_pix_on_1', 0.5), ('read_pix_on_2', 0), ('optimize_func_toggle_pause', <function toggle_pg at 0x000000B007A057B8>), ('savetag', 'deer_dd_XY8-1_init=0.5_0.0_read=0.5,0_readph=0'), ('gated', True), ('sequence_mode', False), ('ctr_single_sweeps', False), (

For debugging, stop sequence for a single tau2 after each of its pi (on NV1 and NV2) pulses.

In [ ]:
qmeas['experiment'] = 'deer_dd_tau_interm'
qmeas['name'] = 'deer_dd_tau_interm'
qmeas['tau_start'] = 0
qmeas['tau2'] = qmeas['tau_start'] 

In [ ]:
run_generic(qmeas)

In [ ]:
df_offset_2on1/1e6, df_offset_1on2/1e6

In [ ]:
uglobals.abort.clear()
# optimize read phase
read_phases = np.linspace(90, 0, 20)
debug = []

for phi in read_phases:
    qmeas = setup_deer_dd_tau_params(f_deer, dd_tau=common.dd_tau, tau2_start= 0e-9, tau2_step=-dd_tau/75, n_tau=75,
                             t_rabi_2=t_rabi_2, deer_ampl=ampl_2, dd_type=dd_type, dd_order=dd_order,
                             read_phase=phi) #90 for CNOT


    qmeas['end_pix_on_2'] = 1  #1
    qmeas['init_pix_on_2'] = 0  #1
    qmeas['measurement_time'] = 30*60
    qmeas['fit_experiment'] = 'sine'   
    qmeas['savetag'] = f"deer_dd_cnot_readphi={phi:.1f}_init_on2={qmeas['init_pix_on_2']}"
    
    debug.append(qmeas)
    run_generic(qmeas)
    
    qmeas['end_pix_on_2'] = 1  #1
    qmeas['init_pix_on_2'] = 1  #1
    qmeas['measurement_time'] = 30*60
    qmeas['fit_experiment'] = 'sine'   
    qmeas['savetag'] = f"deer_dd_cnot_readphi={phi:.1f}_init_on2={qmeas['init_pix_on_2']}"
    
    run_generic(qmeas)
    
debug[-1]
    

Vary decoupling order

In [25]:
#common.he_tau = 0.2e-6
#t_rabi_2 = 196.5e-9
#f_deer= f_center

# to switch between C2NOT1 (order=1,2) and C1NOT2
common = common.swap_nvs(order=[1,2])

f_deer = common.mw_freq_2[0]
ampl_2 = common.mw_amps_2[0]
t_rabi_2 = common.rabi_period_2[0]
dd_tau = common.dd_tau  # -1 for deer_dd_par
#dd_tau = -1
dd_order = common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8


# Test contrast
# may reduce dd_order to see whether coherence is as expected
qmeas = setup_deer_dd_tau_params(f_deer, dd_tau=dd_tau, tau2_start=0e-9, tau2_step=-6e-9, n_tau=30,
                             t_rabi_2=t_rabi_2, deer_ampl=ampl_2, dd_type=dd_type, dd_order=4,  # dd_order  
                             read_phase=0) 
qmeas['tau2_rel_to_pi1'] = False
#qmeas['ampl_mw_2'] = '0'

#tau2_pspacing = dd_tau-common.rabi_period/2-t_rabi_2/2
tau2_pspacing = dd_tau-common.rabi_period
# for optimal control pulses of length 100ns
#tau2_pspacing = dd_tau-5*100e-9
#tau2_pspacing = dd_tau-1*100e-9
#tau2_pspacing = dd_tau

if dd_tau > 0:
    n_tau = 20
    tau2_step = tau2_pspacing/(n_tau)
    tau2_start = -tau2_pspacing/2

    #n_tau = 20
    #tau2_step = tau2_pspacing/(2*n_tau)
    #tau2_start = 0
else:
    n_tau = 4*40
    tau2_start = common.rabi_period/2
    tau2_step = (1/70e3/(common.dd_order*common.dd_type.suborder))/(n_tau)
    n_tau = 20
    tau2_start = 0
    tau2_step = (1/70e3/(common.dd_order*common.dd_type.suborder))/(2*n_tau)

uglobals.abort.clear()
init = [0.,0.]#,[0,1],[1,0],[1,1]]#[::-1]
orders = [1,2,3,4,5]
for idx, o_dd in enumerate(orders[::]):
    dd_order = o_dd   
    n_tau_i = int(n_tau*o_dd/orders[0])  # measure multiple DEER oscillatioins
    tau2_step = tau2_pspacing/(n_tau_i)
    tau2_start = -tau2_pspacing/(2*n_tau_i)
    
    n_tau_i = n_tau  # measure only first oscillation
    
    
    qmeas = setup_deer_dd_tau_params(f_deer, dd_tau=dd_tau, 
                                 tau2_start=tau2_start, tau2_step=tau2_step, n_tau=n_tau_i, 
                             t_rabi_2=t_rabi_2, deer_ampl=ampl_2, dd_type=dd_type, dd_order=dd_order, ## dd_order
                             read_phase=90) #90 for CNOT
    qmeas['tau2_rel_to_pi1'] = False

    qmeas['init_pix_on_1'] = init[0]  #1
    qmeas['init_pix_on_2'] = init[1]  #1
    qmeas['end_pix_on_2'] = 1  #1

    
    qmeas['savetag'] = f"deer_dd_{dd_type.name}-{dd_order}_init={init[0]}_{init[1]}_idx={idx}"

    # AC Stark compensation, get values from doing a calibration with AC Stark Ramsey
    # ATTENTION: adding compensation must not exceed maximum MW amplitude.
    # Safe to do, if NV1/2 pulses don't overlap.
    #ampl_acstark_comp_2on1 = 0.045
    #ampl_acstark_comp_1on2 = 0#0.075
    #df_calib_offset = -15e6
    #df_offset_2on1 = (ampl_acstark_comp_2on1/common.mw_amp)**2*df_calib_offset
    #df_offset_1on2 = (ampl_acstark_comp_1on2/common.mw_amps_2[0])**2*df_calib_offset
    #fac_comp = 3

    #qmeas = add_ac_stark_compensation(qmeas, ampl_comp=[ampl_acstark_comp_2on1,ampl_acstark_comp_1on2], 
    #                                 fac_comp=fac_comp, df_offset=[df_offset_2on1,df_offset_1on2])

    # DEBUG ONLY
    qmeas['start_pix_on_1'] = 0
    qmeas['init_pix_on_1'] = 0.5
    qmeas['init_pix_on_2'] = 0.5
    qmeas['end_pix_on_1'] = 0.5
    qmeas['end_pix_on_2'] = 1.5
    qmeas['read_phase_deg'] = 0
    qmeas['dd_order'] = 2*dd_order
    

    qmeas['measurement_time'] = 8*60*o_dd/orders[0]
    qmeas['optimize_time'] = 30*60
    qmeas['optimize_func_toggle_pause'] = toggle_pg
    
    qmeas['alternating'] = True
    #qmeas['generate_new'] = False  # todo: not working
    
    qmeas['savetag'] = qmeas['savetag'] + "_fci" if 'fci' in qmeas['experiment'] else qmeas['savetag']
    #break
    run_generic(qmeas)

#qmeas = setup_deer_dd_tau_params(f_deer, dd_tau=dd_tau, 
#                                 tau2_start=-tau2_pspacing/2, tau2_step=tau2_pspacing/40, n_tau=40, # 75
#                             t_rabi_2=t_rabi_2, deer_ampl=ampl_2, dd_type=DDMethods.XY8, dd_order=4, ## dd_order
#                             read_phase=90) #90 for CNOT
#qmeas['tau2_rel_to_pi1'] = False
#qmeas['end_pix_on_2'] = 1  #1
#qmeas['init_pix_on_2'] = 0  #1

qmeas

OrderedDict([('experiment', 'deer_dd_tau'), ('name', 'deer_dd_rect_tau'), ('microwave_frequency', 3129100000.0), ('microwave_amplitude', 0.23148148148148145), ('rabi_period', 7.55e-08), ('dd_type', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('dd_order', 10), ('num_of_points', 15), ('tau_start', -4.6225e-07), ('tau_step', 6.163333333333334e-08), ('f_mw_2', '2744400000.0'), ('rabi_period_mw_2', '6.29e-08'), ('ampl_mw_2', '0.23148148148148145'), ('read_phase_deg', 0), ('tau1', 1e-06), ('measurement_time', 2400.0), ('optimize_time', 1800), ('alternating', True), ('pulse_envelope', PulseEnvelopeType(sin_n))), ('tau2_rel_to_pi1', False), ('init_pix_on_1', 0.5), ('init_pix_on_2', 0.5), ('end_pix_on_2', 1.5), ('savetag', 'deer_dd_XY8-5_init=0.0_0.0_idx=4'), ('start_pix_on_1', 0), ('end_pix_on_1', 0.5), ('optimize_func_toggle_pause', <function toggle_pg at 0x00000040C1942730>), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('

#### deer_dd (vary tau1)

Shifting version of gate, but tau2 is kept constant and tau1 is varied.

In [33]:
# to switch between C2NOT1 (order=1,2) and C1NOT2
common = common.swap_nvs(order=[1,2])

try:
    f_deer = common.mw_freq_2[0]
    ampl_2 = common.mw_amps_2[0]
    t_rabi_2 = common.rabi_period_2[0]
    dd_order = common.dd_order #15
    dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8
    dd_cnot_tau = +396.7e-9

    n_tau = 40
    tau1_step = 50e-9
    tau1_start = 1e-6 # 2*dd_cnot_tau + common.rabi_period


    uglobals.abort.clear()
    inits = [[0.5,0.5]]#,[0,1],[1,0],[1,1]]#[::-1]

        
    for (init_1, init_2) in inits:
        qmeas = setup_deer_dd_tau_params(f_deer, dd_tau=dd_tau, 
                                     tau2_start=tau1_start, tau2_step=tau1_step, n_tau=n_tau, 
                                 t_rabi_2=t_rabi_2, deer_ampl=ampl_2, dd_type=dd_type, dd_order=dd_order, ## dd_order
                                 read_phase=90) #90 for CNOT
        qmeas['experiment'] = 'deer_dd_tau1'
        qmeas['tau2'] = dd_cnot_tau
        qmeas['tau2_rel_to_pi1'] = False

        qmeas['init_pix_on_1'] = init_1  #1
        qmeas['init_pix_on_2'] = init_2  #1
        qmeas['end_pix_on_2'] = 1  #1



        qmeas['savetag'] = f"deer_dd_{dd_type.name}_tau1-{dd_order}_init={init_1}{init_2}"

        # DEBUG ONLY
        # check for input state pi2_x, pi2_x and n=2 repetitions of cphase gate (=identitiy)
        qmeas['dd_order'] = 2*dd_order
        qmeas['start_pix_on_1'] = 0
        qmeas['end_pix_on_1'] = 0.5
        qmeas['end_pix_on_2'] = 1+0.5
        qmeas['read_phase_deg'] = 0


        qmeas['measurement_time'] = 120*60
        qmeas['optimize_time'] = 30*60
        qmeas['optimize_func_toggle_pause'] = toggle_pg

        qmeas['alternating'] = True
        #qmeas['generate_new'] = False  # todo: not working

        run_generic(qmeas)

except:
    logger.exception("")

qmeas

OrderedDict([('experiment', 'deer_dd_tau1'), ('name', 'deer_dd_rect_tau'), ('microwave_frequency', 3129100000.0), ('microwave_amplitude', 0.25), ('rabi_period', 7.538400000000001e-08), ('dd_type', <DDMethods.CPMG: [90.0, 90.0]>), ('dd_order', 2), ('num_of_points', 40), ('tau_start', 1e-06), ('tau_step', 5e-08), ('f_mw_2', '2744400000.0'), ('rabi_period_mw_2', '6.7932e-08'), ('ampl_mw_2', '0.23148148148148145'), ('read_phase_deg', 0), ('tau1', 8e-06), ('measurement_time', 7200), ('optimize_time', 1800), ('alternating', True), ('pulse_envelope', PulseEnvelopeType(sin_n))), ('tau2', 3.967e-07), ('tau2_rel_to_pi1', False), ('init_pix_on_1', 0.5), ('init_pix_on_2', 0.5), ('end_pix_on_2', 1.5), ('savetag', 'deer_dd_CPMG_tau1-1_init=0.50.5'), ('start_pix_on_1', 0), ('end_pix_on_1', 0.5), ('optimize_func_toggle_pause', <function toggle_pg at 0x0000009F7B6BE598>), ('gated', True), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 80), ('freq_optimize_

### DEER Rabi

In [ ]:
uglobals.abort.clear()

f_deer = 1.3989e9
f_deer = 1.9e9
#f_deer = 0.737e9


qmeas = setup_deer_rabi_params(f_deer, tau_start=5e-9, tau_step=25e-9, n_tau=40, he_tau=common.he_tau, 
                               two_deer_pi=False, read_phases='-90, 90', deer_ampl=0.25)

#qmeas = setup_deer_rabi_params(f_deer, tau_start=27.5e-9, tau_step=1e-9, n_tau=1, he_tau=common.he_tau)
#qmeas = setup_deer_rabi_params(f_deer, tau_start=5e-9, tau_step=4e-9, n_tau=80, he_tau=300e-9, two_deer_pi=True)

#qmeas = setup_deer_rabi_params(f_deer, tau_start=5e-9, tau_step=4e-9, n_tau=80, he_tau=300e-9, two_deer_pi=False)
run_deer_rabi(qmeas)


In [ ]:
qmeas

### time DEER

In [ ]:
uglobals.abort.clear()

f_deer = 1.407e9
#f_deer = 1.00e9
f_deer = 1.3795e9
f_deer = 3026.7e6
deer_rabi_period = 138.0e-9

# wide band
#qmeas = setup_deer_f_params(f_center, f_step=5e6, n_f=50, two_deer_pi=True)
# high res

qmeas = setup_deer_params(f_deer, tau_start=5e-9, tau_step=100e-9, n_tau=50, 
                      he_tau=common.he_tau, second_rabi_period=deer_rabi_period,
                      two_deer_pi=True)
#qmeas = setup_deer_params(f_deer, tau_start=282e-9, tau_step=0e-9, n_tau=1, 
#                      he_tau=common.he_tau, second_rabi_period=deer_rabi_period,
#                      two_deer_pi=True)
#qmeas['wait_time'] = 1e-6
qmeas['two_deer_pi'] = True


run_deer(qmeas)

#qmeas['name'] = 'deer_pi_start'
#run_deer_pi_start(qmeas)

In [ ]:
qmeas

### T1 like DEER

If the second electron spin is polarized by the laser pulse and its T1 << T1(NV), we can do the following experiment: increasing the t_wait of the NV initialization won't hurt the NV sensor too much. However, the second electron spin starts to decay right after the laser pulse. If it's an ensemble spin, the lowered polarization due to its T1 decay will lower the effective coupling to the sensor. Thus, the frequency observed in the FFT of deer should lower

In [ ]:
t_waits = np.linspace(1e-6, 20e-6, 10)[1:]#[::-1]
uglobals.abort.clear()

for t in t_waits:
    qmeas = setup_deer_params(f_deer, tau_start=5e-9, tau_step=2e-9, n_tau=40, 
                          he_tau=300e-9, second_rabi_period=deer_rabi_period,
                          two_deer_pi=True)
    qmeas['wait_time'] = t
    
    qmeas['n_sweeps'] = 1e6
    qmeas['name'] = 'deer_pi_start'

    qmeas['save_tag'] = f'DEER_pi_start_t_wait={t}'

    #run_deer(qmeas, save_tag=qmeas['save_tag'])
    qmeas['two_deer_pi'] = False
    run_deer_pi_start(qmeas, save_tag=qmeas['save_tag'])

### pODMR DEER pi

In [ ]:
qmeas

In [ ]:
f_deer = 0.737e9
#f_deer = 1.2e9   # off-resonant dummy frequency
t_pi_deer = 68e-9 / 2

uglobals.abort.clear()

qmeas = setup_podmr_deer_pi_params(f_res=common.mw_freq, f_deer=f_deer, t_pi_deer=t_pi_deer,
                           p_scale=60, f_step=500e3, n_f=150) # range ~ 1 MHz, lowest AWG power

run_generic(qmeas)

### Ramsey DEER pi

Ramsey with a pi pulse on the 2nd electron spin right after the 1st pi/2 on the NV.

In [ ]:
f_deer = 0.737e9
f_deer = 1.2e9   # off-resonant dummy frequency
t_pi_deer = 68e-9 / 2


uglobals.abort.clear()
#qmeas = setup_ramsey_deer_pi_params(f_deer, t_pi_deer, 
#                                tau_start=50e-9, n_tau=50, tau_step=20e-9,
#                                alternating_mode=DeerAltModes.NVPi3Half)
qmeas = setup_ramsey_deer_pi_params(f_deer, t_pi_deer, 
                                tau_start=50e-9, n_tau=160, tau_step=10e-9,
                                alternating_mode=DeerAltModes.DeerPiOff_plus_NVPi3Half,
                                two_deer_pi=True)


run_generic(qmeas)

In [ ]:
f_deer

## Charge read

Need to:
1. acquire in fastcomtec listmode to create photon histogram => change_save_mode(2)
2. set red laser power correctly (very low!)

In [ ]:
# default settings
myfastcounter.change_save_mode(0)
#awg8190.set_sample_rate(10e9)  # todo: better do manually!?

In [ ]:
qmeas = setup_charge_read_params(t_cinit_green=30e-6, t_cread_red=8e-3, t_wait=1e-6, #15e-3
                             laser_red_ch='d_ch3', add_gate_ch='d_ch4')
#qmeas = setup_charge_read_params(t_cinit_green=30e-6, t_cread_red=5e-3, t_wait=1e-6,
#                             laser_red_ch='d_ch3', add_gate_ch='d_ch4')
# debug: short red pulse
#qmeas = setup_charge_read_params(t_cinit_green=10e-6, t_cread_red=100e-6, t_wait=1e-6,
#                             laser_red_ch='d_ch3', add_gate_ch='d_ch4')

# if using external fci circuit, other fastcomtec gate/sync trigger channel 
setup['gate_channel'] = 'a_ch2'
qmeas['bin_width'] = 4.096e-7
#setup['gated'] = True
#setup['sync_channel'] = ''
#setup['gate_channel'] = 'd_ch1'

# need to manually change fastcounter trigger ovoltages!
setup['analog_trigger_voltage'] = 0.25

qmeas['savetag'] = f"charge_read"


uglobals.abort.clear()
run_generic(qmeas)

qmeas


In [ ]:

t_reads = np.linspace(2e-3, 12e-3, 6)
#t_reads = np.concatenate([t_reads, np.linspace(3.2e-3, 25e-3, 22)])

uglobals.abort.clear()

for idx, t_read in enumerate(t_reads):
    t_cinit_green = 30e-6
    t_read_red = t_read

    if uglobals.abort.is_set():
        break
        
    try:
        #pass
        #"""
        laser_on()
        time.sleep(1)
        optimize_position()
        #optimize_position()
        laser_off()
        #"""
    
        qmeas = setup_charge_read_params(t_cinit_green=t_cinit_green, t_cread_red=t_read_red, t_wait=1e-6,
                                     laser_red_ch='d_ch3', add_gate_ch='d_ch4')

        setup['gate_channel'] = 'a_ch2'
        qmeas['bin_width'] = 4.096e-7
        setup['analog_trigger_voltage'] = 0.25

        qmeas['savetag'] = f"charge_read_t_read={1e3*t_read_red:.1f}ms_idx={idx}"
        qmeas['measurement_time'] = 5*60
        uglobals.abort.clear()
        

        run_generic(qmeas)

    except:
        logger.exception("")
        uglobals.abort.set()

qmeas

### Plot laser trace in qdplot

In [ ]:
def moving_average(a, n=3) :
    return np.convolve(a, np.ones(n)/n, mode='same')

y_laser = pulsedmeasurementlogic.laser_data.flatten()
bin_width = pulsedmasterlogic.fast_counter_settings['bin_width']
x_laser = np.arange(y_laser.size, dtype=float) * bin_width

# smooth
n_smooth = 1
y_laser = moving_average(y_laser, n=n_smooth)
# undersample
fac_us = 1
x_laser = x_laser[::fac_us]
y_laser = y_laser[::fac_us]


qdplotlogic.set_data(x_laser, y_laser)

len(x_laser), len(y_laser)

### Charge (NV minus) init

Read the current sequence table

In [ ]:
get_sequence_parameter_list('charge_read_fci')[0][1]

In [ ]:
[(idx, entry[1]['ensemble'], f"rep={entry[1]['repetitions']}") for (idx,entry) in enumerate(get_sequence_parameter_list('charge_read_fci'))]

Other than for MFL, we don't use the dynamic input port for conditional jumping. Dynamic mode doesn't allow to use advancement modes different than 'Auto' on AWG8190A. Instead use the Input Trigger as advancement event and set segment advance mode = conditional.

ATTENTION: Need to
1. Execute cell below
2. Manually, Set Fastcomtec to correct START Trig level. Using a_ch2 with max_ampl=0.25 V here!

In [ ]:
awg8190.set_sample_rate(1e9)
try:
    add_fci(enable=True)
except:
    logger.exception("")
awg8190.get_sample_rate()


In [ ]:
# send reset to fast circuit. not needed if reset after every green-red cycle
channel_reset = '1'  # d_ch1

qmeas = {}
qmeas['experiment'] = 'n_digital_high'
qmeas['name'] = 'circuit_reset'
qmeas['digital_channels'] = channel_reset

qmeas['measurement_time'] = 2 

run_generic(qmeas)

In [ ]:
# for init
t_init_red = 20e-6  # fast charge init like Hopper
t_init_red = 3e-3   # slow, pre-processing regime
# for readout
t_read_red = 8e-3 
#t_read_red = 350e-6
# TODO: Check that sequencer is reset correctly!

qmeas = setup_charge_read_fci_params(t_cinit_green=500e-9, t_cinit_red=t_init_red, 
                                     #t_cread_red=60e-6,
                                     t_cread_red=t_read_red,
                                     gate_channel='a_ch2', add_gate_ch='d_ch4')


qmeas['bin_width'] = 4.096e-7
setup['analog_trigger_voltage'] = 0.25

qmeas['savetag'] = f"charge_read_fci_t_init={1e6*t_init_red:.1f}us_t_read={1e3*t_read_red:.1f}ms"

uglobals.abort.clear()
run_generic(qmeas)

qmeas

In [ ]:
qmeas['savetag']

Vary t_read_red

In [ ]:
t_reads

In [ ]:
t_reads = np.linspace(2e-3, 3e-3, 8)
t_reads = np.concatenate([t_reads, np.linspace(3.2e-3, 25e-3, 22)])
t_reads

In [ ]:
filename_default_lst = 'fastcounter.lst'
t_reads = np.linspace(2e-3, 10e-3, 6)
#t_reads = np.concatenate([t_reads, np.linspace(3.2e-3, 25e-3, 22)])

uglobals.abort.clear()

for idx, t_read in enumerate(t_reads):
    t_init_red = 20e-6
    t_init_red =  5e-3
    t_read_red = t_read

    if uglobals.abort.is_set():
        break
        
    try:
        #pass
        #"""
        laser_on()
        time.sleep(1)
        optimize_position()
        #optimize_position()
        laser_off()
        #"""
    
        qmeas = setup_charge_read_fci_params(t_cinit_green=500e-9, t_cinit_red=t_init_red, 
                                             #t_cread_red=60e-6,
                                             t_cread_red=t_read_red,
                                             gate_channel='a_ch2', add_gate_ch='d_ch4')


        qmeas['bin_width'] = 4.096e-7
        setup['analog_trigger_voltage'] = 0.25

        qmeas['savetag'] = f"charge_read_fci_t_init={1e6*t_init_red:.1f}us_t_read={1e3*t_read_red:.1f}ms"
        qmeas['savetag'] += "_thresh=6"
        qmeas['measurement_time'] = 20*60
        uglobals.abort.clear()   

        run_generic(qmeas)

        #move_list_file(qmeas)
    except:
        logger.exception("")
        uglobals.abort.set()

qmeas

# Randomized benchmarking

In [ ]:
gate_translation[-1]
gate_translation
common.dd_tau

In [ ]:
#data_dir = r"E:\Data\2023\02\20230202\rand_bench"
#fname = '20230206-0849-14_rb_seq_2q_ncxfix.json'

data_dir = r"C:\Software\qudi\user_scripts\Timo\own\rand_benchmark"
#fname = '20230213-1230-17_rb_seq_1q.json'
fname = '20230206-0849-14_rb_seq_2q_ncxfix.json'
#fname = '20230202-1138-34_rb_seq_1q.json'

with open(data_dir + "/" + fname) as f:
   # gates_raw = json.load(f)
    gates_raw = json.load(f)

gates_raw = parse_rbjson(gates_raw)
gates, gate_infos = rbjson_2_gates(data_dir + "/" + fname, n_cliff_max=5)
nv_ids = _get_qd_gates_nv_ids(gates)

#gates[0]
#gate_infos
gates_raw

## (non random) repeat CNOT

In [107]:
import itertools
import qutip

def rot_2_init(rotation):
    return TomoRotations[TomoInit.ux180_on_1.name]
def init_2_rot(init):
    return TomoInits[TomoInit.ux180_on_1.name]
    
def swap_basis_def(rho):
    # Genkos state definition is different!
    return np.rot90(rho,2).conj().T

def get_u_gate(rotation):
    # shaddows PostProcess::get_u_gate() 
    if type(rotation) == TomoInit:
        rotation = init_2_rot(rotation)
    
    u_rots = {TomoRotations.none: np.asarray(qutip.qeye(4)),
          TomoRotations.ux90_on_1: np.asarray(qutip.tensor(qutip.rx(np.pi/2.), qutip.qeye(2))),
          TomoRotations.uy90_on_1: np.asarray(qutip.tensor(qutip.ry(np.pi/2.), qutip.qeye(2))),
          TomoRotations.ux90_on_2: np.asarray(qutip.tensor(qutip.qeye(2), qutip.rx(np.pi/2.))),
          TomoRotations.uy90_on_2: np.asarray(qutip.tensor(qutip.qeye(2), qutip.ry(np.pi/2.))),
          TomoRotations.ux180_on_1: np.asarray(qutip.tensor(qutip.rx(np.pi), qutip.qeye(2))),
          TomoRotations.uy180_on_1: np.asarray(qutip.tensor(qutip.ry(np.pi), qutip.qeye(2))),
          TomoRotations.ux180_on_2: np.asarray(qutip.tensor(qutip.qeye(2), qutip.rx(np.pi))),
          TomoRotations.uy180_on_2: np.asarray(qutip.tensor(qutip.qeye(2), qutip.ry(np.pi))),
          TomoRotations.c2not1: np.asarray(qutip.cnot(control=1, target=0)),
          TomoRotations.c1not2: np.asarray(qutip.cnot(control=0, target=1))
            }
    try:
        u = u_rots[rotation]

    except KeyError:
        raise NotImplemented(f"Unknown rot: {rotation}")
        
    return u

def rotation_2_state(rotation, init=TomoRotations.none):
    if type(rotation) != list:
        rotation = [rotation]
    
    if init != TomoInit.none:
        raise NotImplemented
    else:
        cur_state = np.asarray([1,0,0,0])
    
    for rot in rotation:
        cur_state = np.matmul(get_u_gate(rot), cur_state)
    
    return cur_state
        
def get_u_from_name(gate_name):
    
    gate_map = {}
    
    u =  np.asarray([[1,0,0,0],[0,0,0,1],[0,0,1j,0],[0,-1j,0,0]])  # from Genkos sim, simplified
    gate_map.update({'c2not1p': u})
    u =  np.asarray([[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]])  # c2not1 without any phase
    gate_map.update({'c2not1': u})
    #u =   np.asarray([[0,0,1j,0],[0,0,0,1],[-1,0,0,0],[0,-1j,0,0]])  # without pi2 pulses, buggy
    u =   np.asarray([[1,0,0,0],[0,1j,0,0],[0,0,1j,0],[0,0,0,1]])  # without pi2 pulses, buggy
    gate_map.update({'cphase_dd': u})
    
    return gate_map[gate_name]
        

def reverse_gate(init_rotation, name_gate='cphase_dd', n_gate=2):
    
    u_gate = np.linalg.matrix_power(get_u_from_name(name_gate), n_gate)
    
    # pulses available for reverssal of output of gate to ground state
    guess_u2s = {TomoRotations.none:[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],
              TomoRotations.ux180_on_1: np.asarray(qutip.tensor(qutip.rx(np.pi), qutip.qeye(2))),
              TomoRotations.uy180_on_1: np.asarray(qutip.tensor(qutip.ry(np.pi), qutip.qeye(2))),
              TomoRotations.ux180_on_2: np.asarray(qutip.tensor(qutip.qeye(2), qutip.rx(np.pi))),
              TomoRotations.uy180_on_2: np.asarray(qutip.tensor(qutip.qeye(2), qutip.ry(np.pi))),
              TomoRotations.ux90_on_1: np.asarray(qutip.tensor(qutip.rx(np.pi/2.), qutip.qeye(2))),
              TomoRotations.uy90_on_1: np.asarray(qutip.tensor(qutip.ry(np.pi/2.), qutip.qeye(2))),
              TomoRotations.ux90_on_2: np.asarray(qutip.tensor(qutip.qeye(2), qutip.rx(np.pi/2.))),
              TomoRotations.uy90_on_2: np.asarray(qutip.tensor(qutip.qeye(2), qutip.ry(np.pi/2.))),
              TomoRotations.ux90min_on_1: np.asarray(qutip.tensor(qutip.rx(-np.pi/2.), qutip.qeye(2))),
              TomoRotations.uy90min_on_1: np.asarray(qutip.tensor(qutip.ry(-np.pi/2.), qutip.qeye(2))),
              TomoRotations.ux90min_on_2: np.asarray(qutip.tensor(qutip.qeye(2), qutip.rx(-np.pi/2.))),
              TomoRotations.uy90min_on_2: np.asarray(qutip.tensor(qutip.qeye(2), qutip.ry(-np.pi/2.)))
              }

    # calculate the u_need to reverse the output state to what is epected from c2not1^2.init_state
    # still need to reverse the init itself to come back to ground state
    state_init = rotation_2_state(init_rotation, init=TomoRotations.none)
    state_fin = np.matmul(u_gate, state_init)
    logger.debug(f"State after pseudo c2not1: {state_fin} onto {state_init}")

    state_wanted = state_init# np.matmul(rev_u2, state_fin)  # for c2not1^(2n)
    logger.debug(f"Should be {state_wanted} from init {state_init}")

    # check whether we can get the wanted state by 1q rotations on the final state
    # drop globals phases by taking absolute values like |<psi_test, psi_wanted>|
    u_need = None
    # test against all permutations of 2 pulses, ensure first guess is no pulse
    rot_combs = [(TomoRotations.none, TomoRotations.none)] + list(itertools.permutations(guess_u2s.keys(), r=2))
    for rots2 in rot_combs:
        if init_rotation == [TomoRotations.ux90_on_2]:
            pass
            # DEBUG ONLY
            #u_need = [TomoRotations.ux90min_on_2]
            #break

        mat1 = guess_u2s[rots2[0]]
        mat2 = guess_u2s[rots2[1]]
        state_test = np.matmul(mat1, state_fin)
        state_test = np.matmul(mat2, state_test)
        #logger.debug(f"Trying state {state_test} after pulses {rots2}")
        if np.isclose(abs(np.vdot(state_test, state_wanted)), 1):
            u_need = [el for el in rots2 if el != TomoRotations.none]
            break  # ensures that first guess_u2 == TomoRot.none has priority

    if u_need is None:
        raise ValueError(f"Couldn't find a reverse pulse for init={init_rotation}")

    # reverse the init pulse
    u_total = []
    u_total.extend(u_need)
    u_total.extend(reverse_init(init_rotation))

    return u_total

def reverse_c2not1p(init_rotation, n_cnot=2):
    return reverse_gate(init_rotation, n_gate=n_cnot, name_gate='c2not1p')

def reverse_c2not1(init_rotation, n_cnot=1):
    return reverse_gate(init_rotation, n_gate=n_cnot, name_gate='c2not1')
            
def reverse_init(init_rotation):
    u_rev = []
    
    if type(init_rotation) != list:
        init_rotation = [init_rotation]
    
    rot_rev = {TomoRotations.none:TomoRotations.none,
              TomoRotations.ux180_on_1: TomoRotations.ux180min_on_1,
              TomoRotations.uy180_on_1: TomoRotations.uy180min_on_1,
              TomoRotations.ux180_on_2: TomoRotations.ux180min_on_2,
              TomoRotations.uy180_on_2: TomoRotations.uy180min_on_2,
              TomoRotations.ux90_on_1: TomoRotations.ux90min_on_1,
              TomoRotations.uy90_on_1: TomoRotations.uy90min_on_1,
              TomoRotations.ux90_on_2: TomoRotations.ux90min_on_2,
              TomoRotations.uy90_on_2: TomoRotations.uy90min_on_2,
              }
    for init in init_rotation:
        u_rev.extend([rot_rev[init]])
    
    return u_rev
    


            
#inits[2], reverse_c2not1(inits[2], 1)
#np.matmul(reverse_c2not1p(TomoRotations.none, 2), 1j*reverse_c2not1p(TomoRotations.ux180_on_1, 2))
a = None
try:
    #reverse_c2not1p(TomoRotations.ux180_on_1, 2)
    #a = reverse_c2not1p([TomoRotations.ux90_on_2], 2) 
    a = reverse_gate([TomoRotations.ux90_on_2], name_gate="cphase_dd", n_gate=4)
except:
    logger.exception("")

a

[<TomoRotations.ux90min_on_2: 8>]

In [133]:
guess_u2s = {TomoRotations.none:[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],
              TomoRotations.ux180_on_1: np.asarray(qutip.tensor(qutip.rx(np.pi), qutip.qeye(2))),
              TomoRotations.uy180_on_1: np.asarray(qutip.tensor(qutip.ry(np.pi), qutip.qeye(2))),
              TomoRotations.ux180_on_2: np.asarray(qutip.tensor(qutip.qeye(2), qutip.rx(np.pi))),
              TomoRotations.uy180_on_2: np.asarray(qutip.tensor(qutip.qeye(2), qutip.ry(np.pi))),
              }
        

rot_combs = [(TomoRotations.none, TomoRotations.none)] + list(itertools.permutations(guess_u2s.keys(), r=2))
rot_combs

[(<TomoRotations.none: 0>, <TomoRotations.none: 0>), (<TomoRotations.none: 0>, <TomoRotations.ux180_on_1: 9>), (<TomoRotations.none: 0>, <TomoRotations.uy180_on_1: 11>), (<TomoRotations.none: 0>, <TomoRotations.ux180_on_2: 10>), (<TomoRotations.none: 0>, <TomoRotations.uy180_on_2: 12>), (<TomoRotations.ux180_on_1: 9>, <TomoRotations.none: 0>), (<TomoRotations.ux180_on_1: 9>, <TomoRotations.uy180_on_1: 11>), (<TomoRotations.ux180_on_1: 9>, <TomoRotations.ux180_on_2: 10>), (<TomoRotations.ux180_on_1: 9>, <TomoRotations.uy180_on_2: 12>), (<TomoRotations.uy180_on_1: 11>, <TomoRotations.none: 0>), (<TomoRotations.uy180_on_1: 11>, <TomoRotations.ux180_on_1: 9>), (<TomoRotations.uy180_on_1: 11>, <TomoRotations.ux180_on_2: 10>), (<TomoRotations.uy180_on_1: 11>, <TomoRotations.uy180_on_2: 12>), (<TomoRotations.ux180_on_2: 10>, <TomoRotations.none: 0>), (<TomoRotations.ux180_on_2: 10>, <TomoRotations.ux180_on_1: 9>), (<TomoRotations.ux180_on_2: 10>, <TomoRotations.uy180_on_1: 11>), (<TomoRotatio

In [98]:
testU = np.asarray([[1-1j,0,0,0],[0,0,0,1-1j],[0,0,1+1j,0],[0,-1-1j,0,0]])
testU = np.asarray([[0.5,0,-0.5j,0],[0,0,0,0],[0.5j,0,0.5,0],[0,0,0,0]])
swap_basis_def(testU)

array([[ 0. -0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ],
       [ 0. -0.j ,  0.5-0.j ,  0. -0.j , -0. +0.5j],
       [ 0. -0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ],
       [ 0. -0.j ,  0. -0.5j,  0. -0.j ,  0.5-0.j ]])

In [209]:
#common = common.swap_nvs([2,1])

f_deer = [common.mw_freq_2[0]]
ampl_2 = common.mw_amps_2[0]
t_rabi_2 = common.rabi_period_2[0]

dd_cnot_tau = 320.0e-9  # -395e-9#-75.5e-9
dd_cnot_tau = 1300.0e-9  # -395e-9#-75.5e-9
dd_cnot_tau_fix = common.dd_tau  # set 'None' for deer_dd_par_tau
dd_order =  common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8


# load from manually defined gate list of lists
#rots = [
#        [TomoRotations.ux90_on_1, TomoRotations.ux90_on_2, TomoRotations.ux90_on_1, TomoRotations.ux90_on_2,
#        TomoRotations.c2not1]]

#--------------------------------------------------------------------------------------------
# repetitions of cnots

#init = [TomoRotations.none]
#init = [TomoRotations.ux180_on_1] #10
#init = [TomoRotations.ux180_on_2] #01
#init = [TomoInit.ux180_on_both] #11
end = [TomoRotations.ux180min_on_1]
end = [TomoRotations.ux180min_on_2] # DEBUG only
#inits = [[TomoRotations.none],[TomoRotations.ux180_on_1],[TomoRotations.ux180_on_2]]
#inits = [[TomoRotations.none],[TomoRotations.ux180_on_1],[TomoRotations.ux180_on_2],[TomoRotations.ux180_on_1,TomoRotations.ux180_on_2],
 #        [TomoRotations.uy90_on_1]]

#inits = [[TomoRotations.ux180_on_1,TomoRotations.ux180_on_2],
 #        [TomoRotations.uy90_on_1]]

#inits = [[TomoRotations.uy90min_on_1,TomoRotations.ux180_on_2],
 #        [TomoRotations.uy90min_on_2],[TomoRotations.ux180_on_1,TomoRotations.uy90_on_2],
  #       [TomoRotations.uy90min_on_1,TomoRotations.uy90_on_2]]

#inits = [[TomoRotations.ux180_on_1,TomoRotations.uy90_on_2],
#         [TomoRotations.uy90min_on_1,TomoRotations.uy90_on_2]]
#inits = [[TomoRotations.ux180_on_1]]
#ends =  [[TomoRotations.ux180min_on_1]]

inits = []
ends = []


# 00, 10, 01, 11
#inits = [[TomoRotations.none], [TomoRotations.ux180_on_2], [TomoRotations.ux180_on_1],
#        [TomoRotations.ux180_on_1, TomoRotations.ux180_on_2]]
#ends = [[TomoRotations.none], [TomoRotations.ux180min_on_2],[TomoRotations.ux180min_on_1],
#        [TomoRotations.ux180min_on_1, TomoRotations.ux180min_on_2]]  # need to bring back to ground state 00

# -i0, 0-i, -i1, 1-i
inits.extend([[TomoRotations.ux90_on_1], [TomoRotations.ux90_on_2], [TomoRotations.ux90_on_1, TomoRotations.ux180_on_2],
         [TomoRotations.ux90_on_2, TomoRotations.ux180_on_1]])
ends.extend([[TomoRotations.ux90min_on_1], [TomoRotations.ux90min_on_2], [TomoRotations.ux90min_on_1, TomoRotations.ux180min_on_2],
         [TomoRotations.ux90min_on_2, TomoRotations.ux180min_on_1]]) 


# +0, 0+, +1, 1+
#inits.extend([[TomoRotations.uy90_on_1], [TomoRotations.uy90_on_2], [TomoRotations.uy90_on_1, TomoRotations.ux180_on_2],
#         [TomoRotations.uy90_on_2, TomoRotations.ux180_on_1]])
#ends.extend([[TomoRotations.uy90min_on_1], [TomoRotations.uy90min_on_2], [TomoRotations.uy90min_on_1, TomoRotations.ux180min_on_2],
#         [TomoRotations.uy90min_on_2, TomoRotations.ux180min_on_1]])

# +-i, -i+, ++, -i-i
inits.extend([[TomoRotations.uy90_on_1, TomoRotations.ux90_on_2], [TomoRotations.ux90_on_1, TomoRotations.uy90_on_2],
         [TomoRotations.uy90_on_1, TomoRotations.uy90_on_2], [TomoRotations.ux90_on_1, TomoRotations.ux90_on_2]])
ends.extend([[TomoRotations.uy90min_on_1], [TomoRotations.ux90min_on_2],
             [TomoRotations.ux90min_on_1, TomoRotations.uy90min_on_2],
             [TomoRotations.uy90min_on_1, TomoRotations.uy90min_on_2],
             [TomoRotations.ux90min_on_1, TomoRotations.ux90min_on_2]]) 



rots, xticks = [[TomoRotations.none]], [0]
rots, xticks = [], []
auto_reverse_to_ground = True
#auto_reverse_to_ground = True

uglobals.abort.clear()

try:
    for idx_i, init in enumerate(inits):
        end = ends[idx_i]
        for n_cnots in [2,4,6,8]:  # [2,4,6,8,12]: #[2,4,6,8,12,18]: [1,2, 3,4, 9,10, 12,13, 17,18]:
            rot_idx = cp.copy(init)
            #rot_idx.extend([TomoRotations.c2not1]*n_cnots)
            rot_idx.extend([TomoRotations.c2phase1_dd]*n_cnots)
            #rot_idx.extend([TomoRotations.none]*n_cnots)

            if auto_reverse_to_ground:
                rot_idx.extend(reverse_gate(init, name_gate='cphase_dd', n_gate=n_cnots))
            else:
                rot_idx.extend(end)

            rots.append(rot_idx)
            xticks.append(n_cnots)

    #rots, xticks = rots, xticks = [[TomoRotations.ux90_on_1]], [0]
    qmeas = setup_rand_benchmark_params(rotations=rots, x_max=50,
                                 f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                 tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
                                 tau_dd_fix=dd_cnot_tau_fix,
                                 alternating=True, charge_init=False)

    qmeas['xticks'] = list_2_csv(xticks)
    #qmeas['t_idle'] = 25e-9
    #qmeas['ampl_idle_mult'] = 0.8
    #qmeas['mirror_1q_pulses'] = False

    qmeas['measurement_time'] = 600*60
    qmeas['optimize_time'] = 25*60
    qmeas['optimize_func_toggle_pause'] = toggle_pg
    #qmeas['alternating'] = False
    #qmeas['xticks'] = ''

    run_generic(qmeas)
except:
    logger.exception("")
    



qmeas.keys()
qmeas['rabi_period_mw_2']
#save_rots = rots
rots[-1]

[<TomoRotations.ux90_on_1: 1>, <TomoRotations.ux90_on_2: 2>, <TomoRotations.c2phase1_dd: 23>, <TomoRotations.c2phase1_dd: 23>, <TomoRotations.c2phase1_dd: 23>, <TomoRotations.c2phase1_dd: 23>, <TomoRotations.c2phase1_dd: 23>, <TomoRotations.c2phase1_dd: 23>, <TomoRotations.c2phase1_dd: 23>, <TomoRotations.c2phase1_dd: 23>, <TomoRotations.ux90min_on_1: 7>, <TomoRotations.ux90min_on_2: 8>]

In [17]:
rots

[[<TomoRotations.none: 0>, <TomoRotations.cphase_dd: 21>, <TomoRotations.ux90min_on_1: 5>]]

## from file

In [122]:
f_deer = [common.mw_freq_2[0]]
ampl_2 = common.mw_amps_2[0]
t_rabi_2 = common.rabi_period_2[0]

dd_cnot_tau = +340e-9#-75.5e-9
dd_cnot_tau_fix = common.dd_tau  # set 'None' for deer_dd_par_tau
dd_order =  common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8


#-------------------------------------------------------------------------------
# load from (qiskit generated) file
data_dir =  os.path.abspath(savelogic.get_daily_directory())
#data_dir = r"E:\Data\2023\02\20230202\rand_bench"
data_dir = r"C:\Software\qudi\user_scripts\Timo\own\rand_benchmark"
#data_dir = r"E:\Data\2023\01\20230127\rand_bench"
#fname = '20230202-1138-34_rb_seq_1q.json'
#fname = '20230127-1553-55_rb_seq_2q.json'
#fname = '20230206-0849-14_rb_seq_2q_ncxfix.json'
#fname = '20230202-1815-55_rb_seq_2q_ncxfix.json'
fname = '20230220-1057-17_rb_seq_2q_ncxfix.json' #with cnot
#fname = '20230220-1057-17_rb_seq_2q_ncxfix_noCNOT.json' # without cnot
fname = '20230404-1004-21_rb_seq_2q_ncxfix.json'  # fixed cnot


try:
    qmeas = setup_rand_benchmark_params(rotations=data_dir + "/" + fname, x_max=4, x_min=None, n_rand_max=6,
                                        range_arr=None,
                                   f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                   tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
                                   tau_dd_fix=dd_cnot_tau_fix, 
                                   simultaneous_rb=False, alternating=True, xticks_ncliff=True)

    #qmeas['t_idle'] = 25e-9
    #qmeas['ampl_idle_mult'] = 0.8
    #qmeas['mirror_1q_pulses'] = False

    #qmeas['measurement_time'] = 30*60
    #qmeas['optimize_time'] = 5*60
    #qmeas['optimize_func_toggle_pause'] = toggle_pg

    #qmeas['alternating'] = False


    uglobals.abort.clear()
    run_generic(qmeas)


    #common = common.swap_nvs([1,2])
    #qmeas['t_idle']

    common = common.swap_nvs([1,2])
except:
    logger.exception("")

qmeas,len(qmeas['xticks'])

(OrderedDict([('name', 'rand_benchmark_2q'), ('experiment', 'rand_benchmark'), ('savetag', 'rand_benchmark'), ('microwave_amplitude', 0.19999999999999998), ('microwave_frequency', 2616100000.0), ('rabi_period', 6.38e-08), ('rotations_file', 'C:\\Software\\qudi\\user_scripts\\Timo\\own\\rand_benchmark/20230404-1004-21_rb_seq_2q_ncxfix.json'), ('rotations', '<TomoRotations.ux90_on_1: 1>, <TomoRotations.ux90_on_2: 2>, <TomoRotations.uy90_on_1: 3>, <TomoRotations.uy90_on_2: 4>, <TomoRotations.ux90_on_1: 1>, <TomoRotations.ux45min_on_2: 6>, <TomoRotations.uy90min_on_2: 10>, <TomoRotations.c2phase1_dd: 23>, <TomoRotations.ux90_on_1: 1>, <TomoRotations.uy90_on_2: 4>, <TomoRotations.uy90_on_1: 3>, <TomoRotations.ux45min_on_2: 6>, <TomoRotations.ux90min_on_1: 7>, <TomoRotations.uy90_on_2: 4>, <TomoRotations.ux90min_on_1: 7>, <TomoRotations.ux90_on_2: 2>, <TomoRotations.uy90min_on_1: 9>, <TomoRotations.none: 0>, <TomoRotations.uy90_on_1: 3>, <TomoRotations.ux90min_on_2: 8>, <TomoRotations.ux90_o

In [32]:
qmeas['xticks']

'(1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4)'

Non randomized 2q benchmarking (repeat n CNOTs) for different inits

In [90]:
common = common.swap_nvs([1,2])

f_deer = [common.mw_freq_2[0]]
ampl_2 = common.mw_amps_2[0]
t_rabi_2 = common.rabi_period_2[0]

dd_cnot_tau = -395e-9#-75.5e-9
dd_cnot_tau_fix = common.dd_tau  # set 'None' for deer_dd_par_tau
dd_order =  common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8


uglobals.abort.clear()

inits = [TomoRotations.none, TomoRotations.ux180_on_1, TomoRotations.ux180_on_2,
         [TomoRotations.ux180_on_1, TomoRotations.ux180_on_2], TomoRotations.ux90_on_1]
ends = [TomoRotations.none, TomoRotations.ux180min_on_1, TomoRotations.ux180min_on_2,
        [TomoRotations.ux180min_on_1, TomoRotations.ux180min_on_2], TomoRotations.ux90min_on_1]


for i, init in enumerate(inits):
    if uglobals.abort.is_set():
        break
    try:
        # repetitions of cnots
        rots, xticks = [[TomoRotations.none]], [0]
        rots, xticks = [], []
        init = [init] if type(init)!= list else init
        end = [ends[i]] if type(ends[i])!= list else ends[i]
        for idx in range(0, 10):
            n_cnots = 2*(idx+1)
            rot_idx = cp.copy(init)
            rot_idx.extend([TomoRotations.c2not1]*n_cnots)
            rot_idx.extend(end)

            rots.append(rot_idx)
            xticks.append(n_cnots)
            

        

        qmeas = setup_rand_benchmark_params(rotations=rots, x_max=1,
                                         f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                         tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
                                         tau_dd_fix=dd_cnot_tau_fix,
                                         alternating=True, charge_init=False)

        qmeas['xticks'] = list_2_csv(xticks)

        qmeas['measurement_time'] = 30*60
        qmeas['optimize_time'] = 8*60
        qmeas['optimize_func_toggle_pause'] = toggle_pg
        init_str = str([i.name for i in init]).replace("[","").replace("]","").replace("'","")
        qmeas['savetag'] = f"rep_cnot_init={init_str}"
        
        run_generic(qmeas)
        


    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))





logger.info("Autopilot has landed.")
            

qmeas['savetag']
#[i.name for i in init]

'rep_cnot_init=ux180_on_1, ux180_on_2'

## Simultaneous_rb (crosstalk, leakage)

For now, this requires same Rabi times on both NVs.

In [7]:


# load from (qiskit generated) file
data_dir =  os.path.abspath(savelogic.get_daily_directory())
#data_dir = r"E:\Data\2023\02\20230202\rand_bench"
#data_dir = r"E:\Data\2023\01\20230127\rand_bench"
data_dir = r"C:\Software\qudi\user_scripts\Timo\own\rand_benchmark"
#fname = '20230202-1138-34_rb_seq_1q.json'
#fname = '20230213-1230-17_rb_seq_1q.json'
#fname = '20230215-1452-46_rb_seq_1q.json'
fname = '20230215-1513-36_rb_seq_1q.json'
#fname = '20230220-1057-17_rb_seq_2q_ncxfix.json' #with cnot
#fname = '20230220-1057-17_rb_seq_2q_ncxfix_noCNOT.json' #without CNOt
uglobals.abort.clear()
mode_str = ['1q_A', '1q_B', '1q_simult']

for idx, mode in enumerate(mode_str):
    if uglobals.abort.is_set():
        break
    if mode == '1q_A':
        common = common.swap_nvs([1,2])
        mirror = False
    elif mode == '1q_B':
        common = common.swap_nvs([2,1])
        mirror = False
    elif mode == '1q_simult':
        common = common.swap_nvs([1,2])
        mirror = True
    else:
        raise ValueError
    
    
    f_deer = [common.mw_freq_2[0]]
    ampl_2 = common.mw_amps_2[0]
    t_rabi_2 = common.rabi_period_2[0]

    dd_cnot_tau = 400e-9#-75.5e-9
    dd_cnot_tau_fix = common.dd_tau  # set 'None' for deer_dd_par_tau
    dd_order =  common.dd_order #15
    dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8
    
    #array_range = [1,2,4,5,10,15,41,50]
    array_range = [15,20,29,41,50]
    #array_range = [1,2,3,4]
    qmeas = setup_rand_benchmark_params(rotations=data_dir + "/" + fname, x_max=None,x_min=None,range_arr=array_range,
                                     f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                     tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
                                     tau_dd_fix=dd_cnot_tau_fix, 
                                     simultaneous_rb=mirror, alternating=True, xticks_ncliff=True)
    
    
    x_ticks_set = set(qmeas['xticks'])
    #qmeas['t_idle'] = 25e-9
    qmeas['ampl_idle_mult'] = 0.
    
    qmeas['measurement_time'] = 30*60
    qmeas['optimize_time'] = 10*60
    qmeas['optimize_func_toggle_pause'] = toggle_pg

    qmeas['savetag'] = f"rand_bench_{mode}_idx={idx}"


    run_generic(qmeas)
    common = common.swap_nvs([1,2])

    
qmeas

#common = common.swap_nvs([1,2])
#qmeas['t_idle']

common = common.swap_nvs([1,2])

common.__dict__, qmeas,len(qmeas['xticks'])

({'rabi_period_rect': 1.258e-07, 'rabi_shaped': {'shaped_on': True, 'period': 1.25e-07, 'env_type': PulseEnvelopeType(parabola)), 'order_P': 1}, 'mw_freq': 3130640000.0, 'mw_amp': 0.1135, 't_laser_pulsed': 3e-06, 't_wait_pulsed': 1e-06, 't_laser': None, 't_laser_green_init': 3e-06, 't_laser_red_read': 1e-06, 'mw_ramsey_offset': 4000000.0, 'mw_ramsey_off_from_calib': 0.0, 'gamma_n_c13': 6726.149871335747, 'nv_order': [1, 2], 'mw_freq_2': [2744300000.0], 'mw_amps_2': [0.0945], 't_rabi_2_rect': [1.252e-07], 't_rabi_2_shaped': [1.252e-07], 'dd_tau': 2e-06, 'dd_type': <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>, 'dd_order': 1}, OrderedDict([('name', 'rand_benchmark_1q'), ('experiment', 'rand_benchmark'), ('savetag', 'rand_bench_1q_simult_idx=2'), ('microwave_amplitude', 0.1135), ('microwave_frequency', 3130640000.0), ('rabi_period', 1.25e-07), ('rotations_file', 'C:\\Software\\qudi\\user_scripts\\Timo\\own\\rand_benchmark/20230215-1513-36_rb_seq_1q.json'), ('rotations', '<

In [ ]:
# Look for having the channels in the pulseblock "rand_bench_1q"
# rand_bench_2q has th channels
#blockname ='rand_benchmark_2q'
blockname ='rand_benchmark_1q'
#blockname ='tomography'
#blockname ='tomography_single_point'
#blockname = 'rabi'
block =sequencegenerator._saved_pulse_blocks.get(blockname)
block.channel_set # If the pulseblock has the channels the sampling works

vary power. Note that we don't repeat Rabi experiments at every p setting, which is actually needed due to non linear t_rabi(ampl_awg).

In [ ]:
#f_deer
a,b =rbjson_2_gates(data_dir + "/" + fname, n_cliff_max=None)
#a
#b

In [ ]:
A = []
for i in [1,2,3,4]:
    A.append([TomoRotations.c2not1]*i)
    
A

# Entanglement

## Tomography

In [183]:

f_deer = [common.mw_freq_2[0]]
ampl_2 = common.mw_amps_2[0]
t_rabi_2 = common.rabi_period_2[0]

# tomo rotattions
#init = TomoInit.ent_create_bell
init = TomoInit.none
init = [TomoInit.ux90_on_1, TomoInit.ux90_on_2] 

phase = 0
read_on_nv =1

# all needed rotations for full tomography
# tuple(on_nv, phase, rotation)
tomo_rotations = [
                  #(1, 0, TomoRotations.none),
                  #(1, 90, TomoRotations.none),
                  #(2, 0, TomoRotations.none),
                  #(2, 90, TomoRotations.none),

                  #(1, 0, TomoRotations.ux180_on_2),
                  #(1, 90, TomoRotations.ux180_on_2),
                  #(2, 0, TomoRotations.ux180_on_2),
                  #(2, 90, TomoRotations.ux180_on_2),
                  #(1, 0, TomoRotations.ux180_on_1),
                  #(1, 90, TomoRotations.ux180_on_1),
                  #(2, 0, TomoRotations.ux180_on_1),
                  #(2, 90, TomoRotations.ux180_on_1),
                  #(2, 0, TomoRotations.c2not1_ux180_on_1),
                  #(2, 90, TomoRotations.c2not1_ux180_on_1),
                  #(1, 0, TomoRotations.c2not1_ux180_on_1),
                  #(1, 90, TomoRotations.c2not1_ux180_on_1),
                  #(1, 0, TomoRotations.c1not2),
                  #(1, 90, TomoRotations.c1not2),
                  #(2, 0, TomoRotations.c1not2),
                  #(2, 90, TomoRotations.c1not2),
    
                  # for debug, not needed for reconstruction
                 # (1, 0, TomoRotations.c2not1),
                 # (1, 90, TomoRotations.c2not1),
                 # (2, 0, TomoRotations.c2not1),
                 # (2, 90, TomoRotations.c2not1)
                  (2, 0, [TomoRotations.cphase_dd,TomoRotations.cphase_dd]),
                  (2, 90, [TomoRotations.cphase_dd,TomoRotations.cphase_dd]),
                  (1, 0, [TomoRotations.cphase_dd,TomoRotations.cphase_dd]),
                  (1, 90, [TomoRotations.cphase_dd,TomoRotations.cphase_dd]),
                  
                 ]
        
# for cnot


dd_cnot_tau = +396.7e-9
#dd_cnot_tau = -tau2_pspacing/2   # first data point in calibration mes above
dd_cnot_tau_fix = common.dd_tau  # set 'None' for deer_dd_par_tau
dd_order = common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8


tomo_rotations = tomo_rotations[::-1]
dd_cnot_tau, tomo_rotations 

(3.967e-07, [(1, 90, [<TomoRotations.cphase_dd: 21>, <TomoRotations.cphase_dd: 21>]), (1, 0, [<TomoRotations.cphase_dd: 21>, <TomoRotations.cphase_dd: 21>]), (2, 90, [<TomoRotations.cphase_dd: 21>, <TomoRotations.cphase_dd: 21>]), (2, 0, [<TomoRotations.cphase_dd: 21>, <TomoRotations.cphase_dd: 21>])])

In [ ]:
dd_cnot_tau

In [ ]:
rotation = TomoRotations.c2not1
on_nv=1
try:

    qmeas = setup_tomography_params(n_tau=1, 
                                    read_rotation=rotation, init_state=init,
                                    rabi_on_nv=on_nv, rabi_phase_deg=phase,
                                    f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                     tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
                                     use_shaped_cnot=False, tau_dd_fix=dd_cnot_tau_fix,
                                     alternating=True)


    #qmeas['num_of_points'] = 1

    # tomography with rect pulses, but shaped cnot
    """
    cnot_kwargs = {'env_type':Evm.parabola, 
                   'order_P': 1,
                   'tau_dd_fix': 500e-9,
                   'rabi_period': common.rabi_shaped['period'],
                   'rabi_period_mw_2': list_2_csv(common.t_rabi_2_shaped)}
    """
    #qmeas['cnot_kwargs'] = cnot_kwargs
    #qmeas['tau_cnot'] = 70e-9
except:
    logger.exception("")
    
uglobals.abort.clear()

run_generic(qmeas)
qmeas

In [ ]:
setup_tomography_params

### Full tomographic set

In [184]:
tomo_inits =  [TomoInit(val) for val in TomoInit][:10][::-1][3:]
tomo_inits = [init]
#tomo_inits = []
#tomo_inits += [TomoInit.none]
#tomo_inits += [TomoInit.ux180_on_2]
#tomo_inits += [TomoInit.ux180_on_1]
#tomo_inits += [TomoInit.ux180_on_both]
#tomo_inits += [TomoInit.ux90_on_1]
#tomo_inits += [TomoInit.ux90_on_1_ux180_on_2]
tomo_inits

[[<TomoInit.ux90_on_1: 1>, <TomoInit.ux90_on_2: 2>]]

Attention: Need to run 'Tomography' cell above, especiall if changed common params (eg. shaping off/on).

In [177]:
tomo_rots_init

[[1, 90, [<TomoRotations.cphase_dd: 21>, <TomoRotations.cphase_dd: 21>], <TomoInit.ux90_on_both: 3>], [1, 0, [<TomoRotations.cphase_dd: 21>, <TomoRotations.cphase_dd: 21>], <TomoInit.ux90_on_both: 3>], [2, 90, [<TomoRotations.cphase_dd: 21>, <TomoRotations.cphase_dd: 21>], <TomoInit.ux90_on_both: 3>], [2, 0, [<TomoRotations.cphase_dd: 21>, <TomoRotations.cphase_dd: 21>], <TomoInit.ux90_on_both: 3>]]

In [207]:
uglobals.abort.clear()

use_shaped_cnot = False
use_ac_stark_comp = False

ampl_acstark_comp_2on1 = 0.045
ampl_acstark_comp_1on2 = 0#0.053
df_calib_offset = -15e6
df_offset_2on1 = (ampl_acstark_comp_2on1/common.mw_amp)**2*df_calib_offset
df_offset_1on2 = (ampl_acstark_comp_1on2/common.mw_amps_2[0])**2*df_calib_offset
fac_comp = 3


#tomo_inits =  [TomoInit(val) for val in TomoInit][:10][::-1][
debug = []
tomo_rots_init = []

try:
    # Create list of measurements
    for init in tomo_inits:  
        # reference measurements for init of any state, except 00
        if init != [TomoInit.none]:
            tomo_add = [
                  (1, 0, TomoRotations.none, TomoInit.none),
                  (1, 90, TomoRotations.none, TomoInit.none),  # not needed as ref, but usefull to check init state
                  (2, 0, TomoRotations.none, TomoInit.none),
                  (2, 90, TomoRotations.none, TomoInit.none)   # not needed as ref, but usefull to check init state
            ]
            #[tomo_rots_init.append(rot) for rot in tomo_add]
        
        # for 00, we add again (possibly missing) basic tomo rots. They also server as ref measurements.
        elif init == [TomoInit.none]:
            tomo_add = [
                  (1, 0, TomoRotations.none, TomoInit.none),
                  (1, 90, TomoRotations.none, TomoInit.none),  # must, because not only ref, but also tomography
                  (2, 0, TomoRotations.none ,TomoInit.none),
                  (2, 90, TomoRotations.none, TomoInit.none)   # must, because not only ref, but also tomography
            ]
            #[tomo_rots_init.append(rot) for rot in tomo_add if rot[0:-1] not in tomo_rotations]
        
        for (on_nv, phase,  rot) in tomo_rotations:
            tomo_rots_init.append([on_nv, phase, rot, init])
        
    # do experiments
    for (on_nv, phase,  rot, init) in tomo_rots_init:
        if type(rot) != list:
            rot = [rot]
        qmeas = setup_tomography_params(n_tau=25, 
                                    read_rotation=rot, init_state=init,
                                    rabi_on_nv=on_nv, rabi_phase_deg=phase,
                                    f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                     tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
                                     #tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=1,  # DEBUG ONLY
                                     use_shaped_cnot=use_shaped_cnot, tau_dd_fix=dd_cnot_tau_fix,
                                     alternating=True)

        qmeas['measurement_time'] = 3*60
        #[qmeas['savetag'] += f"_init_{init.n}" for i in init] 
        for idx, i in enumerate(init):
            qmeas['savetag'] += ("_init=" if idx == 0 else "")
            qmeas['savetag'] += f"{i.name}_" 
        
        if use_ac_stark_comp:
            qmeas = add_ac_stark_compensation(qmeas, ampl_comp=[ampl_acstark_comp_2on1,ampl_acstark_comp_1on2], 
                                  fac_comp=fac_comp, df_offset=[df_offset_2on1,df_offset_1on2])
        
        if np.any([r.name[:2] in ['c1', 'c2', 'cp'] for r in rot]):
            qmeas['measurement_time'] = 15*60

        debug.append((rot, qmeas))
        run_generic(qmeas)
            

except:
    logger.exception("")

#[qm[1]['savetag'] for qm in debug]
debug[-1]


([<TomoRotations.cphase_dd: 21>, <TomoRotations.cphase_dd: 21>], OrderedDict([('name', 'tomo_rot'), ('experiment', 'tomography'), ('savetag', 'tomography_on2_rot_phi=0_init=ux90_on_1_ux90_on_2_'), ('microwave_amplitude', 0.25), ('microwave_frequency', 2616500000.0), ('rabi_period', 4.11e-08), ('tau_start', 0.0), ('tau_step', 2.61e-09), ('rabi_on_nv', 2), ('rabi_phase_deg', 0), ('rotation', [<TomoRotations.cphase_dd: 21>, <TomoRotations.cphase_dd: 21>]), ('init_state', [<TomoInit.ux90_on_1: 1>, <TomoInit.ux90_on_2: 2>]), ('init_state_kwargs', {}), ('cnot_kwargs', {'tau_dd_fix': 2e-06}), ('tau_cnot', 3.967e-07), ('dd_type_cnot', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('dd_order', 1), ('alternating', True), ('num_of_points', 25), ('f_mw_2', '2744900000.0'), ('rabi_period_mw_2', '5.22e-08'), ('ampl_mw_2', '0.25'), ('measurement_time', 900), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('pulse_envelope', PulseEnvelopeType(sin_n)

In [195]:
[f"_init_{i.name}" for i in init]

['_init_ux90_on_1', '_init_ux90_on_2']

### tomographic set for different tau_cnot

In [ ]:
tomo_inits, tomo_rots_init

In [ ]:
dd_cnot_taus = np.linspace(100e-9, 400e-9, 6)[::-1]
#dd_cnot_taus = np.insert(dd_cnot_taus, 0, -85e-9)
dd_cnot_taus*1e9

In [ ]:
uglobals.abort.clear()

#use_shaped_cnot = False
use_shaped_cnot = True
use_ac_stark_comp = False
#use_oc_both = True
#dd_cnot_taus = np.linspace(-160e-9, -350e-9, 6)[::-1]
#dd_cnot_taus = np.insert(dd_cnot_taus, 0, -395e-9)
#dd_cnot_taus = np.insert(dd_cnot_taus, 0, 0e-9)

#ampl_acstark_comp_2on1 = 0.045
#ampl_acstark_comp_1on2 = 0#0.053
#df_calib_offset = -15e6
#df_offset_2on1 = (ampl_acstark_comp_2on1/common.mw_amp)**2*df_calib_offset
#df_offset_1on2 = (ampl_acstark_comp_1on2/common.mw_amps_2[0])**2*df_calib_offset
#fac_comp = 3


#tomo_inits =  [TomoInit(val) for val in TomoInit][:10][::-1][
debug = []
tomo_rots_init = []

try:
    # Create list of measurements
    for init in tomo_inits:
        
        # reference measurements for init of any state, except 00
        if init != TomoInit.none:
            tomo_add = [
                  (1, 0, TomoRotations.none, TomoInit.none),
                  #(1, 90, TomoRotations.none, TomoInit.none),  # not needed as ref, but usefull to check init state
                  (2, 0, TomoRotations.none, TomoInit.none),
                  #(2, 90, TomoRotations.none, TomoInit.none)   # not needed as ref, but usefull to check init state
            ]
            [tomo_rots_init.append(rot) for rot in tomo_add]
        
        # for 00, we add again (possibly missing) basic tomo rots. They also server as ref measurements.
        elif init == TomoInit.none:
            tomo_add = [
                  (1, 0, TomoRotations.none, TomoInit.none),
                  (1, 90, TomoRotations.none, TomoInit.none),  # must, because not only ref, but also tomography
                  (2, 0, TomoRotations.none ,TomoInit.none),
                  (2, 90, TomoRotations.none, TomoInit.none)   # must, because not only ref, but also tomography
            ]
            [tomo_rots_init.append(rot) for rot in tomo_add if rot[0:-1] not in tomo_rotations]
        
        for (on_nv, phase,  rot) in tomo_rotations:
            tomo_rots_init.append([on_nv, phase, rot, init])
        
    # do experiments
    for tau_cnot in dd_cnot_taus:
        for (on_nv, phase,  rot, init) in tomo_rots_init:
            qmeas = setup_tomography_params(n_tau=50, 
                                        read_rotation=rot, init_state=init,
                                        rabi_on_nv=on_nv, rabi_phase_deg=phase,
                                        f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                         tau_cnot=tau_cnot, dd_type_cnot=dd_type, dd_order=dd_order,
                                         #tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=1,  # DEBUG ONLY
                                         use_shaped_cnot=use_shaped_cnot, tau_dd_fix=dd_cnot_tau_fix,
                                         alternating=True)

            qmeas['measurement_time'] = 1*60
            qmeas['savetag'] += f"_init_{init.name}_tau_cnot={1e9*tau_cnot:.1f}ns" 

            if use_ac_stark_comp:
                qmeas = add_ac_stark_compensation(qmeas, ampl_comp=[ampl_acstark_comp_2on1,ampl_acstark_comp_1on2], 
                                      fac_comp=fac_comp, df_offset=[df_offset_2on1,df_offset_1on2])

            if 'c1not2' in rot.name or 'c2not1' in rot.name:
                qmeas['measurement_time'] = 20*60
                if use_oc_both:
                    qmeas['cnot_kwargs']['env_type'] = Evm.optimal

            debug.append((rot, qmeas))
            
            #continue
            laser_on()
            time.sleep(1)
            #optimize_position()
            optimize_position()
            laser_off()
            
            run_generic(qmeas)
            
            

except:
    logger.exception("")

[qm[1]['savetag'] for qm in debug][:10]

## Tomography with single points (no Rabi)

In [ ]:
f_deer = [common.mw_freq_2[0]]
ampl_2 = common.mw_amps_2[0]
t_rabi_2 = common.rabi_period_2[0]


dd_cnot_tau = -225e-9#-75.5e-9
dd_cnot_tau_fix = common.dd_tau  # set 'None' for deer_dd_par_tau
dd_order = common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8

dd_type, dd_order, dd_cnot_tau

benchmark c2not1 with all inits

In [ ]:
rots = [TomoRotations.c2not1, TomoRotations.c2not1, TomoRotations.c2not1, TomoRotations.c2not1]
#rots = [TomoRotations.none, TomoRotations.none, TomoRotations.none, TomoRotations.none]
inits = [TomoInit.none, TomoInit.ux180_on_1, TomoInit.ux180_on_2, TomoInit.ux180_on_both]


try:
    qmeas = setup_tomography_single_params(
                                 read_rotation=rots, init_state=inits,
                                 f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                 tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
                                 tau_dd_fix=dd_cnot_tau_fix,
                                 alternating=True)
        
    qmeas['cnot_kwargs']['env_type'] = Evm.optimal
    
    uglobals.abort.clear() 
    run_generic(qmeas)
except:
    logger.exception("")
    

qmeas

In [ ]:
qmeas

In [ ]:
qmeas['experiment'] = 'tomography_single'

In [ ]:
rots = [TomoRotations.none, TomoRotations.none]
inits = [TomoInit.ux180_on_both, TomoInit.ux180_on_both]


try:
    qmeas = setup_tomography_single_params(
                                 read_rotation=rots, init_state=inits,
                                 f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                 tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
                                 tau_dd_fix=dd_cnot_tau_fix,
                                 alternating=True)
        
    qmeas['init_state_kwargs']['env_type'] = Evm.optimal
    
    uglobals.abort.clear() 
    run_generic(qmeas)
except:
    logger.exception("")
    

qmeas

## Create Bell

In [ ]:
# drive all HFS lines
common.mw_freq = 2736.2e6
common.mw_amp = 0.25/(2*1.3)
common.rabi_period_rect = 131.42e-9

mw_freq_nv2 = 2820.59e6
mw_amp_nv2 = 0.25/1.6
t_rabi_nv2 = 131.8e-9#common.rabi_period

ampl_mw2 = [mw_amp_nv2]#[common.mw_amp]
f_mw2 = [mw_freq_nv2]

np.sum([common.mw_amp] + ampl_mw2)


In [ ]:
tau_start = np.max([t_rabi_nv2, common.rabi_period])/2 + 5e-9

# drive only one, SQT
qmeas = setup_ent_create_bell_params(tau_start=tau_start, tau_step=20e-9, n_tau=75,
                             f_mw_1=[], f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             dd_type=DDMethods.XY8, dd_order=4, alternating=True)

# drive two NVs, SQT
qmeas = setup_ent_create_bell_params(tau_start=tau_start, tau_step=20e-9, n_tau=60,
                             f_mw_1=[], f_mw_2=f_mw2, ampl_mw_2=ampl_mw2, rabi_period_mw_2=[t_rabi_nv2],
                             dd_type=DDMethods.XY8, dd_order=4, alternating=True)

uglobals.abort.clear()
run_generic(qmeas)

qmeas

## Bell Ramsey

In [ ]:
tau_cnot = -300e-9

f_mw2 = common.mw_freq_2[0]
ampl_mw2 = common.mw_amps_2[0]
t_rabi_nv2 = common.rabi_period_2[0]
dd_cnot_tau_fix = common.dd_tau
dd_order = common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8


In [ ]:

# drive two NVs, SQT
try:
    qmeas = setup_bell_ramsey_params(tau_start=10e-9, tau_step=2e-9, n_tau=20, tau_cnot=tau_cnot,
                                     tau_dd_fix = dd_cnot_tau_fix,
                                 f_mw_1=[], f_mw_2=[f_mw2], ampl_mw_2=ampl_mw2, rabi_period_mw_2=[t_rabi_nv2],
                                 dd_type=dd_type, dd_order=dd_order, alternating=True)
except:
    logger.exception("")
uglobals.abort.clear()
run_generic(qmeas)

qmeas

## Bell Hahn

In [ ]:
tau_cnot = -125e-9
use_oc_both = True

f_mw2 = common.mw_freq_2[0]
ampl_mw2 = common.mw_amps_2[0]
t_rabi_nv2 = common.rabi_period_2[0]
dd_cnot_tau_fix = common.dd_tau
dd_order = common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8


In [ ]:

# drive two NVs, SQT
try:
    qmeas = setup_bell_hahnecho_params(tau_start=10e-9, tau_step=10e-9, n_tau=20, tau_cnot=tau_cnot,
                                     tau_dd_fix = dd_cnot_tau_fix,
                                 f_mw_1=[], f_mw_2=[f_mw2], ampl_mw_2=ampl_mw2, rabi_period_mw_2=[t_rabi_nv2],
                                 dd_type=dd_type, dd_order=dd_order, alternating=True)
    if use_oc_both:
        qmeas['cnot_kwargs']['env_type'] = Evm.optimal
except:
    logger.exception("")

uglobals.abort.clear()
run_generic(qmeas)

qmeas

# Auto pilot multiple NVs

## define pois

In [ ]:
opt_on_poi =  poimanagerlogic.poi_names[79]  #poimanagerlogic.poi_names[7]
opt_on_poi



In [ ]:
pois_double, pois_special = [],[]
# south
# need to redo xy8
for idx in [42, 45, 28, 39, 8, 20, 43, 25, 27, 18]:
    pois_double.append(poimanagerlogic.poi_names[int(idx)-1])
# north
#for idx in [28, 39, 84, 101, 104]:
#    pois_double.append(poimanagerlogic.poi_names[int(idx)+53])
#pois_double

# for DEER
pois_special = []
# south
#25, 43, 20, 8, 53, 2, 30, 29, 24, 31, 47, 14, 26, 33, 23, 58, 60, 48, 
# 18, 41, 55
for idx in [80]:
    pois_special.append(poimanagerlogic.poi_names[int(idx)-1])
# north
#for idx in [90, 108, 67, 84]:
#    pois_special.append(poimanagerlogic.poi_names[int(idx)+53])
pois_special

In [ ]:
"21_1",  "20_7", "20_2", "20_4"

Rename pois (here swapped day and month in POI string)

In [ ]:
[poimanagerlogic.rename_poi(p.replace('0301','0103'),p) for p in poimanagerlogic.poi_names[65:]]

In [ ]:
poimanagerlogic.poi_names[-20:-10]

In [ ]:
[f"nv_20220228_{i+1}" for i, p in enumerate(poimanagerlogic.poi_names) if 'poi' in p]

In [ ]:
[poimanagerlogic.rename_poi(f"nv_20220228_{i+1}",p) for i, p in enumerate(poimanagerlogic.poi_names) if 'poi' in p]

In [ ]:
# CAREFUL: might be buggy
[poimanagerlogic.rename_poi(f"nv_mayd_20220103_{i+17}b",p) for i, p in enumerate(poimanagerlogic.poi_names[-20:-10])]
#[(f"nv_mayd_20220103_{i+17}a") for i, p in enumerate(poimanagerlogic.poi_names[80:90])]

## pODMR (with adjusted Rabi times) for 4 NV orientations

In [ ]:
uglobals.abort.clear()

# for pois=None, all experiment will run on the current confocal pos
pois = None
#pois = poimanagerlogic.poi_names[3:14]
pois = [poimanagerlogic.poi_names[120]]#,
       # poimanagerlogic.poi_names[3+1],
       # poimanagerlogic.poi_names[3+3],
       # poimanagerlogic.poi_names[3+5]]
opt_on_poi = poimanagerlogic.poi_names[119]

logger.info("Autopilot is taking of for pois {}".format(pois))

common.rabi_period_rect = 85.0e-9 
f_center = 3.05e9
f_step_odmr_fine = 2e6
n_f_odmr_fine = 150
assume_double_nv = True

qmeas_list = []

#qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=3000e3, n_f=200)
qexp =  setup_podmr_params(f_res=f_center, p_scale=1, f_step=3000e3, n_f=100)
qexp['type'] = 'pulsedodmr'
qexp['name'] = 'podmr_coarse'
qexp['measurement_time'] = 60#200
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_dip'
qexp['fit_parameter'] = 'Position'
if assume_double_nv:
    qexp['fit_experiment'] = 'lor_dip_2x'
    qexp['fit_parameter'] = 'Position 0'
    
qexp['update_parameters'] = {'rabi': 'microwave_frequency',
                             #'Hahn_Echo': 'microwave_frequency',
                             #'Ramsey': 'microwave_frequency',
                             #'podmr_fine': {'target_name': 'freq_start',
                             #               'func': f"_x_ - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"
                             #              }
                            }
if assume_double_nv:
    qexp['update_parameters'] = {'rabi': {'target_name': 'microwave_frequency',
                                         'func': f"fit_result.result_str_dict['Position 0']['value']"},
                                 'rabi_B': {'target_name': 'microwave_frequency',
                                         'func': f"fit_result.result_str_dict['Position 1']['value']"},
                             #'Hahn_Echo': 'microwave_frequency',
                             #'Ramsey': 'microwave_frequency',
                             #'podmr_fine': {'target_name': 'freq_start',
                             #               'func': f"_x_ - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"
                             #              }
                              }

qmeas_list.append(copy.deepcopy(qexp))
#"""

#Experiment2
#"""
qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50, alternating=False, n_tau=100)
qexp['type'] = 'rabi'
qexp['measurement_time'] = 60#60#2*60
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'sine'
qexp['fit_parameter'] = 'Period'
qexp['update_parameters'] = {'Ramsey': 'rabi_period',
                             'Hahn_Echo': 'rabi_period',
                             'podmr_fine': 'rabi_period'}  # feed forware parameters

qmeas_list.append(copy.deepcopy(qexp))

if assume_double_nv:
    qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50, alternating=False, n_tau=100)
    qexp['type'] = 'rabi'
    qexp['name'] = 'rabi_B'
    qexp['measurement_time'] = 60#60#2*60
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'sine'
    qexp['fit_parameter'] = 'Period'
    qexp['update_parameters'] = {'Ramsey': 'rabi_period',
                                 'Hahn_Echo': 'rabi_period',
                                 'podmr_fine': 'rabi_period'}  # feed forware parameters

    qmeas_list.append(copy.deepcopy(qexp))
                                 
                                 
qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
qexp =  setup_podmr_params(f_res=f_center, p_scale=1, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
qexp['type'] = 'pulsedodmr'
qexp['name'] = 'podmr_fine'
qexp['measurement_time'] = 60
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_dip'
qexp['fit_parameter'] = 'Position'
qexp['update_parameters'] = {'Rabi': 'microwave_frequency',
                             'Hahn_Echo': 'microwave_frequency',
                             'Ramsey': 'microwave_frequency',
                              }
#qmeas_list.append(copy.deepcopy(qexp))

"""
#Experiment3
qexp = setup_ramsey_params(tau_start=50e-9, n_tau=400, tau_step=50e-9)  # res= 50 kHz 
qexp['experiment'] = 'ramsey'
qexp['measurement_time'] = 120
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
qexp['fit_experiment'] = 'double_sine_decay'
qmeas_list.append(copy.deepcopy(qexp))

"""

laser_off()

try:
    #run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False, poi_list=pois)
    run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=True,
                        opt_on_poi=opt_on_poi,  # NV as reference
    #                    poi_list=pois)
    pass

except Exception as e:
    stack_trace = traceback.format_exc()
    logger.exception("auto run failed: {}.\n".format(stack_trace))




In [ ]:
qmeas_list[1]

In [ ]:
opt_on_poi = poimanagerlogic.poi_names[10]
opt_on_poi

In [ ]:
pois = poimanagerlogic.poi_names[8:]
pois

## T2 char script

If NV counts while pulsed mes low, make sure tilt correction is off before running!

In [95]:
# In order to stop the automized measurement insert pulsedmasterlogic.break_variable = True into manager
import copy
uglobals.abort.clear()

# for pois=None, all experiment will run on the current confocal pos
pois = None
# start from south
#pois = poimanagerlogic.poi_names[9:]
# start with potential doubles, then rest
#pois = (pois_double + poimanagerlogic.poi_names[12+9:])
pois = pois_special
opt_on_poi = opt_on_poi

logger.info("Autopilot is taking of for pois {}".format(pois))

common.rabi_period_rect = 50e-9#85.0e-9 
common.mw_amp = 0.25

# if rabi frequencies are known already and roughly matched 
#common.rabi_period_rect = 131e-9#85.0e-9 
#common.mw_amp = 0.25/(2*1.3)
mw_amp_nv2 = None
#mw_amp_nv2 = 0.25/1.6

f_center = 2.780e9
f_step_odmr_fine = 1.0e6  # make tight to avoid fitting the wrong NV
n_f_odmr_fine = 50
assume_double_nv = True

qmeas_list = []

#qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=3000e3, n_f=200)
qexp =  setup_podmr_params(f_res=f_center, p_scale=1, f_step=3000e3, n_f=70)
qexp['type'] = 'pulsedodmr'
qexp['name'] = 'podmr_coarse'
qexp['measurement_time'] = 2*60  # 6*60
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_dip'
qexp['fit_parameter'] = 'Position'
if assume_double_nv:
    qexp['fit_experiment'] = 'lor_dip_2x'
    qexp['fit_parameter'] = 'Position 0'
#qexp['fit_experiment'] = 'lor_dip'
#qexp['fit_parameter'] = 'Position 0'

qexp['update_parameters'] = {'rabi': 'microwave_frequency',
                             #'Hahn_Echo': 'microwave_frequency',
                             #'Ramsey': 'microwave_frequency',
                             #'podmr_fine': {'target_name': 'freq_start',
                             #               'func': f"_x_ - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"
                             #              }
                            }
if assume_double_nv:
    qexp['update_parameters'] = {'rabi': {'target_name': 'microwave_frequency',
                                         'func': f"fit_result.result_str_dict['Position 0']['value']"},
                                 'rabi_B': {'target_name': 'microwave_frequency',
                                         'func': f"fit_result.result_str_dict['Position 1']['value']"},
                                 'podmr_fine': {'target_name': 'freq_start',
                                          'func': f"fit_result.result_str_dict['Position 0']['value'] - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"},
                                 'podmr_fine_B': {'target_name': 'freq_start',
                                  'func': f"fit_result.result_str_dict['Position 1']['value'] - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"}
                                }
qmeas_list.append(copy.deepcopy(qexp))
#"""

#Experiment2
#"""
qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50, alternating=False, n_tau=100)
qexp['type'] = 'rabi'
qexp['measurement_time'] = 2*60
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'sine'
qexp['fit_parameter'] = 'Period'
qexp['update_parameters'] = {#'Ramsey': 'rabi_period',
                             #'Hahn_Echo': 'rabi_period',
                             'podmr_fine': 'rabi_period'}  # feed forware parameters

qmeas_list.append(copy.deepcopy(qexp))

if assume_double_nv:
    qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50, alternating=False, n_tau=100)
    if mw_amp_nv2:
        qexp['microwave_amplitude'] = mw_amp_nv2
    qexp['type'] = 'rabi'
    qexp['name'] = 'rabi_B'
    qexp['measurement_time'] = 2*60
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'sine'
    qexp['fit_parameter'] = 'Period'

    qexp['update_parameters'] = {#'Ramsey_B': 'rabi_period',
                                 #'Hahn_Echo_B': 'rabi_period',
                                 'podmr_fine_B': 'rabi_period'}  # feed forware parameters

    
    qmeas_list.append(copy.deepcopy(qexp))

# next Exp
qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
qexp =  setup_podmr_params(f_res=f_center, p_scale=1, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
qexp['type'] = 'pulsedodmr'
qexp['name'] = 'podmr_fine'
qexp['measurement_time'] = 2*60
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_dip'
qexp['fit_parameter'] = 'Position'
qexp['update_parameters'] = {'rabi_fine': 'microwave_frequency',
                             'ramsey': 'microwave_frequency',
                             'hahn_echo': 'microwave_frequency',
                             'xy8_n': 'microwave_frequency',
                              }
qmeas_list.append(cp.deepcopy(qexp))

if assume_double_nv:
    qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=p_scale_odmr_fine, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
    qexp =  setup_podmr_params(f_res=f_center, p_scale=p_scale_odmr_fine, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
    if mw_amp_nv2:
        qexp['microwave_amplitude'] = mw_amp_nv2
    qexp['type'] = 'pulsedodmr'
    qexp['name'] = 'podmr_fine_B'
    qexp['measurement_time'] = 1 *60
    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None
    qexp['generate_new'] = True
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information
    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'lor_dip'
    qexp['fit_parameter'] = 'Position'
    qexp['update_parameters'] = {'rabi_fine_B': 'microwave_frequency',
                                 'ramsey': 'microwave_frequency',
                                 'hahn_echo_B': 'microwave_frequency',
                                 'xy8_n_B': 'microwave_frequency',
                                  }
    qmeas_list.append(cp.deepcopy(qexp))

    
# next Exp
qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50, alternating=False, n_tau=100)
qexp['type'] = 'rabi'
qexp['name'] = 'rabi_fine'
qexp['measurement_time'] = 2*60
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'sine'
qexp['fit_parameter'] = 'Period'
qexp['update_parameters'] = {#'Ramsey': 'rabi_period',
                             'hahn_echo': 'rabi_period',
                             'xy8_n': 'rabi_period'  # feed forware parameters
                            }
qmeas_list.append(cp.deepcopy(qexp))

if assume_double_nv:
    qexp = setup_rabi_params(tau_step=1*common.rabi_period/50, alternating=False, n_tau=100)
    if mw_amp_nv2:
        qexp['microwave_amplitude'] = mw_amp_nv2
    qexp['type'] = 'rabi'
    qexp['name'] = 'rabi_fine_B'
    qexp['measurement_time'] = 2*60
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'sine'
    qexp['fit_parameter'] = 'Period'
    qexp['update_parameters'] = {#'Ramsey': 'rabi_period',
                                 'hahn_echo_B': 'rabi_period',
                                 'xy8_n_B': 'rabi_period'  # feed forware parameters
                            }
    qmeas_list.append(cp.deepcopy(qexp))

# next Exp Ramsey
ramsey_n_tau = 75
ramsey_tau_step = 25e-9
qmeas = setup_ramsey_params(ramsey_tau_step, n_tau=ramsey_n_tau, tau_step=ramsey_tau_step, alternating=True)
qexp['type'] = 'Ramsey'
qexp['name'] = 'ramsey'
qexp['measurement_time'] = 5*60  #15
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

qexp['fit_experiment'] = 'sine_33'
#qmeas_list.append(cp.deepcopy(qexp))

if assume_double_nv:
    qmeas = setup_ramsey_params(ramsey_tau_step, n_tau=ramsey_n_tau, tau_step=ramsey_tau_step, alternating=True)
    qexp['type'] = 'Ramsey'
    qexp['name'] = 'ramsey_B'
    qexp['measurement_time'] = 5*60  #15
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    qexp['fit_experiment'] = 'sine_33'
    #qmeas_list.append(cp.deepcopy(qexp))



# next Exp Hahn     
he_tau_step = 5*100e-9
he_n_tau = 50
#he_n_tau = 50
qexp = setup_hahn_params(tau_start=10e-9, n_tau=he_n_tau, tau_step=he_tau_step, alternating=True)
qexp['type'] = 'hahn_echo'
qexp['name'] = 'hahn_echo'
qexp['measurement_time'] = 3*60  #15
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

qexp['fit_experiment'] = 'decay_exp'
qmeas_list.append(cp.deepcopy(qexp))

if assume_double_nv:
    #he_tau_step = 50e-9 #400e-9
    #he_n_tau = 50
    #he_n_tau = 50
    qexp = setup_hahn_params(tau_start=10e-9, n_tau=he_n_tau, tau_step=he_tau_step, alternating=True)
    qexp['type'] = 'hahn_echo'
    qexp['name'] = 'hahn_echo_B'
    qexp['measurement_time'] = 3*60 #15
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    qexp['fit_experiment'] = 'decay_exp'
    qmeas_list.append(cp.deepcopy(qexp))
    

# next Exp XY8

# MUST not be shorter than any pi pulse!
xy_tau = 120e-9 # 200e-9#440.0e-9

qexp = setup_xy8_nsweep_params(xy_tau, n_n=40, xy8_start=1, xy8_step=2, alternating=True)  
qexp['type'] = 'xy8_nsweep'
qexp['name'] = 'xy8_n'
qexp['measurement_time'] = 12*60  #15
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

qexp['fit_experiment'] = 'decay_exp'
#qmeas_list.append(cp.deepcopy(qexp))

if assume_double_nv:
    qexp = setup_xy8_nsweep_params(xy_tau, n_n=40, xy8_start=1, xy8_step=2, alternating=True)  
    qexp['type'] = 'xy8_nsweep'
    qexp['name'] = 'xy8_n_B'
    qexp['measurement_time'] = 12*60 #15
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    qexp['fit_experiment'] = 'decay_exp'
    #qmeas_list.append(cp.deepcopy(qexp))

# next Exp
#

    
    
    
laser_off()
try:
    #run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False, poi_list=pois)
    run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=True,
                        opt_on_poi=opt_on_poi,  # reference for sample shift (on stable, unique NV)
                        poi_list=pois)
    #pass
except Exception as e:
    stack_trace = traceback.format_exc()
    logger.exception("auto run failed: {}.\n".format(stack_trace))




Traceback (most recent call last):
  File "C:\Software\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-95-111e39d79b49>", line 11, in <module>
    pois = pois_special
NameError: name 'pois_special' is not defined


In [ ]:
qmeas_list[-5]

## DEER

DEER DD requires shaped pulses that are currently not implemented for xy8 sequences. Thus, we have to run the calibration completly again!

In [ ]:
pois_special[:]

In [ ]:
if common.rabi_shaped['shaped_on'] == False:
    raise ValueError("DEER DD only supports shaped pulses.")

import copy
uglobals.abort.clear()

# for pois=None, all experiment will run on the current confocal pos
pois = None
# start from south
#pois = poimanagerlogic.poi_names[10:]
# start with potential doubles, then rest
#pois = (pois_double + poimanagerlogic.poi_names[12+9:])
pois = pois_special
opt_on_poi = opt_on_poi #poimanagerlogic.poi_names[64]

logger.info("Autopilot is taking of for pois {}".format(pois))

common.rabi_period_rect = 90e-9#85.0e-9 
f_center = 2.780e9
f_step_odmr_fine = 0.5e6  # make tight to avoid fitting the wrong NV
n_f_odmr_fine = 50
common.he_tau = 200e-9
assume_double_nv = True

qmeas_list = []

#qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=3000e3, n_f=200)
qexp =  setup_podmr_params(f_res=f_center, p_scale=1, f_step=3000e3, n_f=70)
qexp['type'] = 'pulsedodmr'
qexp['name'] = 'podmr_coarse'
qexp['measurement_time'] = 6*60
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_dip'
qexp['fit_parameter'] = 'Position'
if assume_double_nv:
    qexp['fit_experiment'] = 'lor_dip_2x'
    qexp['fit_parameter'] = 'Position 0'
#qexp['fit_experiment'] = 'lor_dip'
#qexp['fit_parameter'] = 'Position 0'

qexp['update_parameters'] = {'rabi': 'microwave_frequency',
                             #'Hahn_Echo': 'microwave_frequency',
                             #'Ramsey': 'microwave_frequency',
                             #'podmr_fine': {'target_name': 'freq_start',
                             #               'func': f"_x_ - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"
                             #              }
                            }
if assume_double_nv:
    qexp['update_parameters'] = {'rabi': {'target_name': 'microwave_frequency',
                                         'func': f"fit_result.result_str_dict['Position 0']['value']"},
                                 'rabi_B': {'target_name': 'microwave_frequency',
                                         'func': f"fit_result.result_str_dict['Position 1']['value']"},
                                 'deer_f': {'target_name': 'microwave_frequency',
                                         'func': f"fit_result.result_str_dict['Position 0']['value']"},
                                 'podmr_fine': {'target_name': 'freq_start',
                                          'func': f"fit_result.result_str_dict['Position 0']['value'] - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"},
                                 'podmr_fine_B': {'target_name': 'freq_start',
                                  'func': f"fit_result.result_str_dict['Position 1']['value'] - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"}
                                }
qmeas_list.append(copy.deepcopy(qexp))
#"""

#Experiment2
#"""
qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50, alternating=False, n_tau=100)
qexp['type'] = 'rabi'
qexp['name'] = 'rabi'
qexp['measurement_time'] = 2*60
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'sine'
qexp['fit_parameter'] = 'Period'
qexp['update_parameters'] = {#'Ramsey': 'rabi_period',
                             #'Hahn_Echo': 'rabi_period',
                             'podmr_fine': 'rabi_period',
                             'deer_f':'rabi_period',
                             'deer_t':'rabi_period'}  

qmeas_list.append(copy.deepcopy(qexp))

if assume_double_nv:
    qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50, alternating=False, n_tau=100)
    qexp['type'] = 'rabi'
    qexp['name'] = 'rabi_B'
    qexp['measurement_time'] = 2*60
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'sine'
    qexp['fit_parameter'] = 'Period'

    qexp['update_parameters'] = {#'Ramsey_B': 'rabi_period',
                                 #'Hahn_Echo_B': 'rabi_period',
                                 'podmr_fine_B': 'rabi_period',
                                 }
    
    qmeas_list.append(copy.deepcopy(qexp))

# next Exp
qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
qexp =  setup_podmr_params(f_res=f_center, p_scale=2, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
qexp['type'] = 'pulsedodmr'
qexp['name'] = 'podmr_fine'
qexp['measurement_time'] = 2*60
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_dip'
qexp['fit_parameter'] = 'Position'
qexp['update_parameters'] = {'rabi_fine': 'microwave_frequency',
                             'hahn_echo': 'microwave_frequency',
                             'xy8_n': 'microwave_frequency',
                             'deer_f':'microwave_frequency',
                             'deer_t':'microwave_frequency'
                              }
qmeas_list.append(copy.deepcopy(qexp))

if assume_double_nv:
    qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
    qexp =  setup_podmr_params(f_res=f_center, p_scale=2, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
    qexp['type'] = 'pulsedodmr'
    qexp['name'] = 'podmr_fine_B'
    qexp['measurement_time'] = 2*60
    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None
    qexp['generate_new'] = True
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information
    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'lor_dip'
    qexp['fit_parameter'] = 'Position'
    qexp['update_parameters'] = {'rabi_fine_B': 'microwave_frequency',
                                 'hahn_echo_B': 'microwave_frequency',
                                 'xy8_n_B': 'microwave_frequency',
                                 'deer_t':'mw_freq2'
                                  }
    qmeas_list.append(copy.deepcopy(qexp))

    
# next Exp
qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50, alternating=False, n_tau=100)
qexp['type'] = 'rabi'
qexp['name'] = 'rabi_fine'
qexp['measurement_time'] = 2*60
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'sine'
qexp['fit_parameter'] = 'Period'
qexp['update_parameters'] = {#'Ramsey': 'rabi_period',
                             'hahn_echo': 'rabi_period',
                             'xy8_n': 'rabi_period',
                             'deer_f':'rabi_period',
                             'deer_t': 'rabi_period'
                            }
qmeas_list.append(copy.deepcopy(qexp))

if assume_double_nv:
    qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50, alternating=False, n_tau=100)
    qexp['type'] = 'rabi'
    qexp['name'] = 'rabi_fine_B'
    qexp['measurement_time'] = 2*60
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'sine'
    qexp['fit_parameter'] = 'Period'
    qexp['update_parameters'] = {#'Ramsey': 'rabi_period',
                                 'hahn_echo_B': 'rabi_period',
                                 'xy8_n_B': 'rabi_period',
                                 'deer_f': 'rabi_period2',
                                 'deer_t': 'rabi_period2'
                            }
    qmeas_list.append(copy.deepcopy(qexp))

# next Exp
#

qexp = setup_deer_dd_f_params(f_center, dd_tau=common.he_tau, f_step=3e6, n_f=70,
                               t_rabi_2=None, dd_type=DDMethods.XY8, dd_order=30)
qexp['type'] = 'DEER_DD_freq'
qexp['name'] = 'deer_f'
qexp['measurement_time'] = 25*60  #15
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

qexp['fit_experiment'] = 'lor_dip_2x'
#qmeas_list.append(copy.deepcopy(qexp))


t_rabi_2 = 196.5e-9
f_deer = 1500e6
# range of tau2: 0 to (tau/2-t_pi_1/2-t_pi_2/2)
# highest g= n_pi*1/tau_2
# lowest  g= n_pi*n_tau/tau_2
# T2= 8*order*he_tau
qexp = setup_deer_dd_tau_params(f_deer, dd_tau=common.he_tau, tau2_step=-2e-9, n_tau=50,
                                t_rabi_2=t_rabi_2, dd_type=DDMethods.XY8, dd_order=45)
qexp['type'] = 'DEER_DD_tau'
qexp['name'] = 'deer_t'
qexp['measurement_time'] = 15*60  #15
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qmeas_list.append(copy.deepcopy(qexp))
    
    
laser_off()
try:
    #run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False, poi_list=pois)
    run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=True,
                        opt_on_poi=opt_on_poi,  # reference for sample shift (on stable, unique NV)
                        poi_list=pois)
    #pass
except Exception as e:
    stack_trace = traceback.format_exc()
    logger.exception("auto run failed: {}.\n".format(stack_trace))




In [ ]:
qexp

In [ ]:
common.he_tau = 0.2e-6
t_rabi_2 = 10e-9
qexp = setup_deer_dd_tau_params(1e9, dd_tau=common.he_tau, tau2_step=-5e-9, n_tau=50,
                                t_rabi_2=t_rabi_2, dd_type=DDMethods.XY8, dd_order=10)
qexp['rabi_period'] = 1.3676809838312765e-07
qexp['rabi_period2'] = 1.3158247448460443e-07

uglobals.abort.clear()
run_generic(qexp)

In [ ]:
qexp

## 2 qubit Fidelity (non-random) for different Rabi drive

In [250]:
fitter = pulsedmeasurementlogic.fc
fitter.fit_list['sine']['parameters']['amplitude']
fit_data, fit_result = pulsedmeasurementlogic.do_fit('sine')
fit_result.result_str_dict

{'Period': {'value': 7146712.966533834, 'error': 180374.2966262698, 'unit': 'Hz'}, 'Frequency': {'value': 1.3992446663000675e-07, 'error': 3.5315224449869387e-09, 'unit': '1/Hz'}, 'Phase': {'value': 179.9999687162911, 'error': 364.6212661332073, 'unit': 'deg'}, 'Offset': {'value': 0.8373633185298925, 'error': 0.008214096487033805, 'unit': ''}, 'Amplitude': {'value': 0.036301784486111724, 'error': 0.011633692541313698, 'unit': ''}, 'Contrast': {'value': 8.310228796089781, 'error': 2.851983177058617, 'unit': '%'}}

In [140]:
def setup_f_2q_nrep_auto(nrep_arr=None, dd_cnot_tau=None, savetag_add='',
                         f_step_odmr_fine=1.5e6, n_f_odmr_fine=75, df_both=0e6, 
                         deer_tau2_step=None, deer_tau2_start=None,
                         deer_init=None, deer_end=None, deer_read_phase=None):
    
    if nrep_arr is None:
        nrep_arr = [2,4,6,8]
    
    # for pois=None, all experiment will run on the current confocal pos
    pois = None

    qmeas_list = []
    assume_double_nv = True

    qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=f_step_odmr_fine*1/common.p_scale, n_f=n_f_odmr_fine)
    qexp['type'] = 'pulsedodmr'
    qexp['name'] = 'podmr_fine'
    qexp['measurement_time'] = 2*60
    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None
    qexp['generate_new'] = True
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information
    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'lor_dip'
    qexp['fit_parameter'] = 'Position'
    qexp['update_parameters'] = {'rabi_fine': 'microwave_frequency',
                                 'deer_dd_tau': 'microwave_frequency',
                                 'nrep_cnot_2q': 'microwave_frequency',

                                  'deer_dd_tau': {'target_name': 'f_mw',
                                  'func': f"{df_both}+fit_result.result_str_dict['Position']['value']"},
                                  'nrep_cnot_2q': {'target_name': 'f_mw',
                                   'func': f"{df_both}+fit_result.result_str_dict['Position']['value']"}


                                  }
    qexp['savetag'] = (get_savetag(qexp) + savetag_add)
    qmeas_list.append(cp.deepcopy(qexp))

    if assume_double_nv:
        # todo: broken for p_scale != 1
        qexp = setup_podmr_params(f_res=common.mw_freq_2[0], p_scale=1, f_step=f_step_odmr_fine*1/common.p_scale, n_f=n_f_odmr_fine)
        #qmeas = setup_podmr_params(f_res=common.mw_freq_2[0], p_scale=1, f_step=0.8e6, n_f=70)
        qexp['measurement_time'] = 2.0*60

        qexp['rabi_period'] = common.rabi_period_2[0] #/(qmeas['microwave_amplitude']/common.mw_amp)
        qexp['microwave_amplitude'] = common.mw_amps_2[0] # *qmeas['microwave_amplitude']/common.mw_amp

        qexp['type'] = 'pulsedodmr'
        qexp['name'] = 'podmr_fine_B'
        qexp['optimize_time'] = None
        qexp['freq_optimize_time'] = None
        qexp['generate_new'] = True
        qexp['meas_type'] = conventional_measurement
        qexp['meas_info'] = add_conventional_information
        # if empty  or commented out, no fit is used
        qexp['fit_experiment'] = 'lor_dip'
        qexp['fit_parameter'] = 'Position'
        qexp['update_parameters'] = {'rabi_fine_B': 'microwave_frequency',
                                     'deer_dd_tau': {'target_name': 'f_mw_2',
                                      'func': f"str({df_both}+fit_result.result_str_dict['Position']['value'])"},
                                     'nrep_cnot_2q': {'target_name': 'f_mw_2',
                                      'func': f"str({df_both}+fit_result.result_str_dict['Position']['value'])"}
                                      }
        qexp['savetag'] = (get_savetag(qexp) + savetag_add)
        qmeas_list.append(cp.deepcopy(qexp))


    # next Exp
    qexp = setup_rabi_params(tau_step=common.rabi_period/20, alternating=False, n_tau=75)
    qexp['type'] = 'rabi'
    qexp['name'] = 'rabi_fine'
    qexp['measurement_time'] = 2*60
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'sine'
    qexp['fit_edit_params'] = [lmfit.Parameter('phase', value=1.57, vary=False)]
    qexp['fit_parameter'] = 'Period'
    qexp['update_parameters'] = {'deer_dd_tau': 'rabi_period',
                                 'nrep_cnot_2q': 'rabi_period'  # feed forware parameters
                                }
    qexp['savetag'] = (get_savetag(qexp) + savetag_add)
    qmeas_list.append(cp.deepcopy(qexp))

    if assume_double_nv:

        qexp = setup_rabi_params(tau_step=common.rabi_period_2[0]/20, n_tau=75, alternating=False) 
        qexp['microwave_frequency'] = common.mw_freq_2[0]
        qexp['microwave_amplitude'] = common.mw_amps_2[0]
        qexp['rabi_period'] = common.rabi_period_2[0]

        qexp['type'] = 'rabi'
        qexp['name'] = 'rabi_fine_B'
        qexp['measurement_time'] = 2*60



        qexp['meas_type'] = conventional_measurement
        qexp['meas_info'] = add_conventional_information

        qexp['optimize_time'] = None
        qexp['freq_optimize_time'] = None

        # if empty  or commented out, no fit is used
        qexp['fit_experiment'] = 'sine'
        qexp['fit_edit_params'] = [lmfit.Parameter('phase', value=1.57, vary=False)]
        qexp['fit_parameter'] = 'Period'
        qexp['update_parameters'] = {'deer_dd_tau': {'target_name': 'rabi_period_mw_2',
                                      'func': f"str(fit_result.result_str_dict['Period']['value'])"},
                                     'nrep_cnot_2q': {'target_name': 'rabi_period_mw_2',
                                      'func': f"str(fit_result.result_str_dict['Period']['value'])"}
                                }
        qexp['savetag'] = (get_savetag(qexp) + savetag_add)
        qmeas_list.append(cp.deepcopy(qexp))


    # calibration of cphase^2 gate
    f_deer = [common.mw_freq_2[0]]
    ampl_2 = common.mw_amps_2[0]
    t_rabi_2 = common.rabi_period_2[0]

    dd_cnot_tau_fix = common.dd_tau  # set 'None' for deer_dd_par_tau
    dd_order =  2*common.dd_order # 2* for cphase^2
    dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8

    if dd_cnot_tau is None:
        n_tau = 20
        tau2_pspacing = dd_cnot_tau_fix - common.rabi_period
        if deer_tau2_step is None:    
            tau2_step = tau2_pspacing/n_tau #tau2_pspacing/(n_tau)
        else:
            tau2_step = deer_tau2_step
        if deer_tau2_start is None:
            tau2_start = -tau2_pspacing/2
        else:
            tau2_start = deer_tau2_start
        
        init = [0.5,0] if deer_init is None else deer_init
        end = [0.5, 0] if deer_end is None else deer_end
        end_phi = 90 if deer_read_phase is None else deer_read_phase

        init_1 = init[0]
        init_2 = init[1]
        qexp = setup_deer_dd_tau_params(f_deer, dd_tau=dd_cnot_tau_fix, 
                                     tau2_start=tau2_start, tau2_step=tau2_step, n_tau=n_tau, # 40
                                 t_rabi_2=t_rabi_2, deer_ampl=ampl_2, dd_type=dd_type, dd_order=dd_order, ## dd_order
                                 read_phase=end_phi) #90 for CNOT, 0 for cphase
        qexp['tau2_rel_to_pi1'] = False
        qexp['alternating'] = True

        # OLD: CNOT
        #qexp['init_pix_on_1'] = init_1  #1
        #qexp['init_pix_on_2'] = init_2  #1
        #qexp['end_pix_on_2'] = 1  #1
        #qexp['alternating'] = False

        # new: cphase^2
        qexp['start_pix_on_1'] = 0
        qexp['init_pix_on_1'] = init_1  #1
        qexp['init_pix_on_2'] = init_2  #1
        qexp['end_pix_on_1'] = end[0]
        qexp['read_pix_on_2'] = end[1]
        qexp['end_pix_on_2'] = 1  #1
        #qexp['init_pix_on_1'] = 0.5  #1
        #qexp['init_pix_on_2'] = 0.5  #1

        #qexp['end_pix_on_1'] = 0.5
        #qexp['end_pix_on_2'] = 1+0.5


        qexp['savetag'] = f"deer_dd_{dd_type.name}-{dd_order}_init={init_1}_{init_2}"
        qexp['fit_experiment'] = 'sine'
        qexp['fit_parameter'] = 'Period'
        #qexp['fit_edit_params'] = [lmfit.Parameter('phase', value=1.57, min=0, max=np.pi, vary=True),
        #                           lmfit.Parameter('amplitude', value=0.05, min=0, max=1, vary=True)]
        qexp['fit_edit_params'] = [lmfit.Parameter('phase', value=1.57, min=-np.pi, max=np.pi, vary=True),
                                   lmfit.Parameter('amplitude', value=0.05, min=5e-3, max=1, vary=True)]
        qexp['update_parameters'] = {
                                    'nrep_cnot_2q': {'target_name': 'tau_cnot',
                                    'func': f"fit_result.result_str_dict['Period']['value']*(1-1/360*(90+fit_result.result_str_dict['Phase']['value']))"}
                                    #'func': f"1/4*fit_result.result_str_dict['Period']['value']-1/(360)*fit_result.result_str_dict['Period']['value']*fit_result.result_str_dict['Phase']['value']"}
                                }

        qexp['measurement_time'] = 90*60
        qexp['optimize_time'] = 20*60
        qexp['optimize_func_toggle_pause'] = toggle_pg

        qexp['type'] = qexp['experiment']
        qexp['name'] = 'deer_dd_tau'
        qexp['meas_type'] = conventional_measurement
        qexp['meas_info'] = add_conventional_information

        qexp['savetag'] = f"deer_dd_{dd_type.name}-{dd_order}"\
                   f"_init={init_1}_{init_2}_read={qexp['end_pix_on_1']}_{qexp['read_pix_on_2']}"\
                   f"_readph={qexp['read_phase_deg']}" 
        qexp['savetag'] = (get_savetag(qexp) + savetag_add)
        qmeas_list.append(cp.deepcopy(qexp))


    # CNOT nrep fidelity mes
    # 00, 10, 01, 11
    inits = [[TomoRotations.none],# [TomoRotations.ux180_on_2], [TomoRotations.ux180_on_1],
            [TomoRotations.ux180_on_1, TomoRotations.ux180_on_2]]
    # -i0, 0-i, -i1, 1-i
    inits.extend([[TomoRotations.ux90_on_1], [TomoRotations.ux90_on_2], [TomoRotations.ux90_on_1, TomoRotations.ux180_on_2],
             [TomoRotations.ux90_on_2, TomoRotations.ux180_on_1]])
    # +0, 0+, +1, 1+
    #inits.extend([[TomoRotations.uy90_on_1], [TomoRotations.uy90_on_2], [TomoRotations.uy90_on_1, TomoRotations.ux180_on_2],
    #         [TomoRotations.uy90_on_2, TomoRotations.ux180_on_1]])
    # +-i, -i+, ++, -i-i
    inits.extend([[TomoRotations.uy90_on_1, TomoRotations.ux90_on_2], [TomoRotations.ux90_on_1, TomoRotations.uy90_on_2],
             #[TomoRotations.uy90_on_1, TomoRotations.uy90_on_2], [TomoRotations.ux90_on_1, TomoRotations.ux90_on_2]
                 ])


    rots, xticks = [[TomoRotations.none]], [0]
    rots, xticks = [], []


    """
    if dd_cnot_tau is None:
        dd_cnot_tau = -999e-9# dummy value replaced by autoexp
    for idx_i, init in enumerate(inits):
        for n_cnots in nrep_arr:
            rot_idx = cp.copy(init)
            rot_idx.extend([TomoRotations.c2phase1_dd]*n_cnots)
            end = reverse_gate(init, name_gate='cphase_dd', n_gate=n_cnots)
            rot_idx.extend(end)

            rots.append(rot_idx)
            xticks.append(n_cnots)



    qexp = setup_rand_benchmark_params(rotations=rots, x_max=50,
                                 f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                 tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
                                 tau_dd_fix=dd_cnot_tau_fix,
                                 alternating=True, charge_init=False)

    qexp['xticks'] = list_2_csv(xticks)

    qexp['type'] = 'rand_benchmark_2q'
    qexp['name'] = 'nrep_cnot_2q'
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['measurement_time'] = 90*60
    qexp['optimize_time'] = 35*60
    qexp['optimize_func_toggle_pause'] = toggle_pg

    qexp['fit_experiment'] = 'decay_exp'
    qexp['savetag'] = (get_savetag(qexp) + savetag_add)
    qmeas_list.append(cp.deepcopy(qexp))
    """


    return qmeas_list

0

0

In [81]:
0

0

In [45]:
run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False,
                        opt_on_poi=None,  # reference for sample shift (on stable, unique NV)
                        poi_list=None)

In [88]:
dd_cnot_tau = None

try:
    uglobals.abort.clear()
    laser_off()

    qmeas_list = setup_f_2q_nrep_auto(savetag_add='', dd_cnot_tau=dd_cnot_tau)
    run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False,
                        opt_on_poi=None, poi_list=None)

except Exception as e:
    stack_trace = traceback.format_exc()
    logger.exception("auto run failed: {}.\n".format(stack_trace))
#"""

0

0

In [85]:
[q['savetag'] for q in qmeas_list if 'deer' in q['savetag']]
qmeas_list[-1]

OrderedDict([('experiment', 'deer_dd_tau'), ('name', 'deer_dd_tau'), ('microwave_frequency', 3129400000.0), ('microwave_amplitude', 0.23148148148148145), ('rabi_period', 7.55e-08), ('dd_type', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('dd_order', 2), ('num_of_points', 20), ('tau_start', -7.1225e-07), ('tau_step', 3.56125e-08), ('f_mw_2', '2744300000.0'), ('rabi_period_mw_2', '6.29e-08'), ('ampl_mw_2', '0.23148148148148145'), ('read_phase_deg', 90), ('tau1', 1.5e-06), ('measurement_time', 1800), ('optimize_time', 1200), ('alternating', True), ('pulse_envelope', PulseEnvelopeType(sin_n))), ('tau2_rel_to_pi1', False), ('start_pix_on_1', 0), ('init_pix_on_1', 0.5), ('init_pix_on_2', 0.5), ('end_pix_on_1', 0), ('read_pix_on_2', 0.5), ('end_pix_on_2', 1), ('savetag', 'deer_dd_XY8-2_init=0.5_0.5_read=0_0.5_readph=90'), ('fit_experiment', 'sine'), ('fit_parameter', 'Period'), ('fit_edit_params', [<Parameter 'phase', 1.57, bounds=[-3.141592653589793:3.141592653589793]>, <

For different Rabi frequencies / powers

In [47]:
pscales = 1/(np.linspace(1, 0.2, 8))
#pscales = np.insert(pscales, 5, 1)
#pscales = np.append(pscales, 1/np.linspace(0.9, 0.5, 4))

pscales = pscales
1/pscales

array([1.        , 0.88571429, 0.77142857, 0.65714286, 0.54285714,
       0.42857143, 0.31428571, 0.2       ])

In [205]:
qmeas_list[4]['fit_edit_params']

[<Parameter 'phase', 1.57, bounds=[0:3.141592653589793]>, <Parameter 'amplitude', 0.05, bounds=[0:1]>]

In [95]:

uglobals.abort.clear()
dd_cnot_tau = -395e-9   # for Rabi~ 13 MHz, may be overwritten by fit in autoexp


uglobals.abort.clear()
#pscales = np.linspace(1, 0.5, 2)[::-1]
nreps = [0,2,4,6,8,12,18]

p_start = 1


for i, p in enumerate(pscales):
    if uglobals.abort.is_set():
        break

    save_mw_amp, save_mw_amps_2 = common.mw_amp, cp.copy(common.mw_amps_2)
    save_p = common.p_scale
    common.mw_amp = 1/p* common.mw_amp
    common.mw_amps_2[0] = 1/p* common.mw_amps_2[0] 
    common.p_scale = p


    try:
        qmeas_list = setup_f_2q_nrep_auto(nrep_arr=nreps, savetag_add=f'_pscale={1/p:.2f}_idx={i}', dd_cnot_tau=None)

        #laser_off()
        run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False,
                        opt_on_poi=None,  # reference for sample shift (on stable, unique NV)
                        poi_list=None)
                    #pass
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.exception("auto run failed: {}.\n".format(stack_trace))
    finally:
        common.mw_amp, common.mw_amps_2 = save_mw_amp, save_mw_amps_2 
        common.p_scale = save_p 



        
    
        
#"""

logger.info("Autopilot has landed.")
            
    



qmeas_list[1]



OrderedDict([('name', 'podmr_fine_B'), ('experiment', 'pulsedodmr'), ('rabi_period', 2.049090909090909e-07), ('freq_step', 471428.57142857136), ('num_of_points', 75), ('freq_start', 2725821428.571429), ('microwave_amplitude', 0.07857142857142857), ('measurement_time', 480.0), ('optimize_time', None), ('fit_experiment', 'lor_dip'), ('pulse_envelope', PulseEnvelopeType(sin_n))), ('type', 'pulsedodmr'), ('freq_optimize_time', None), ('generate_new', True), ('meas_type', <function conventional_measurement at 0x000000661BD07378>), ('meas_info', <function add_conventional_information at 0x000000661BD03D90>), ('fit_parameter', 'Position'), ('update_parameters', {'rabi_fine_B': 'microwave_frequency', 'deer_dd_tau': {'target_name': 'f_mw_2', 'func': "str(fit_result.result_str_dict['Position']['value'])"}, 'nrep_cnot_2q': {'target_name': 'f_mw_2', 'func': "str(fit_result.result_str_dict['Position']['value'])"}}), ('savetag', 'podmr_fine_B_pscale=0.31_idx=6')])

For different MW detuning

In [54]:
dfs = np.linspace(0, 1.5e6, 5)[::-1]

In [57]:

uglobals.abort.clear()
dd_cnot_tau = -395e-9   # for Rabi~ 13 MHz, may be overwritten by fit in autoexp


uglobals.abort.clear()
#pscales = np.linspace(1, 0.5, 2)[::-1]
nreps = [2]



for i, df in enumerate(dfs):
    if uglobals.abort.is_set():
        break

    try:
        qmeas_list = setup_f_2q_nrep_auto(nrep_arr=nreps, savetag_add=f'_df={df/1e6:.2f}MHz_idx={i}',
                                          dd_cnot_tau=None, df_both=df)

        #laser_off()
        run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False,
                        opt_on_poi=None,  # reference for sample shift (on stable, unique NV)
                        poi_list=None)
                    #pass
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.exception("auto run failed: {}.\n".format(stack_trace))
    finally:
        pass



        
    
        
#"""

logger.info("Autopilot has landed.")
            
    



qmeas_list[1]



OrderedDict([('name', 'podmr_fine_B'), ('experiment', 'pulsedodmr'), ('rabi_period', 6.29e-08), ('freq_step', 1500000.0), ('num_of_points', 75), ('freq_start', 2688150000.0), ('microwave_amplitude', 0.23148148148148145), ('measurement_time', 120.0), ('optimize_time', None), ('fit_experiment', 'lor_dip'), ('pulse_envelope', PulseEnvelopeType(sin_n))), ('type', 'pulsedodmr'), ('freq_optimize_time', None), ('generate_new', True), ('meas_type', <function conventional_measurement at 0x0000009F09D37378>), ('meas_info', <function add_conventional_information at 0x0000009F09D32D90>), ('fit_parameter', 'Position'), ('update_parameters', {'rabi_fine_B': 'microwave_frequency', 'deer_dd_tau': {'target_name': 'f_mw_2', 'func': "str(0.0+fit_result.result_str_dict['Position']['value'])"}, 'nrep_cnot_2q': {'target_name': 'f_mw_2', 'func': "str(0.0+fit_result.result_str_dict['Position']['value'])"}}), ('savetag', 'podmr_fine_B_df=0.00MHz_idx=4')])

For differnt dd orders

In [29]:
dd_orders = [1,2,3,4,5]

In [41]:

uglobals.abort.clear()
dd_cnot_tau = -395e-9   # for Rabi~ 13 MHz, may be overwritten by fit in autoexp


uglobals.abort.clear()


for i, order in enumerate(dd_orders):
    if uglobals.abort.is_set():
        break

    try:
        save_dd_order = common.dd_order 
        common.dd_order = order
        dd_cnot_tau_fix = common.dd_tau  # set 'None' for deer_dd_par_tau
   
        n_tau = 20
        tau2_pspacing = dd_cnot_tau_fix - common.rabi_period
        tau2_step = tau2_pspacing/n_tau * 1/(order/dd_orders[0])
        tau2_start = -tau2_pspacing/2 * 1/(order/dd_orders[0])
        
        qmeas_list = setup_f_2q_nrep_auto(savetag_add=f'_o={order}_idx={i}',
                                          dd_cnot_tau=None, deer_tau2_step=tau2_step, deer_tau2_start=tau2_start)
        
        qmeas_list[-1]['measurement_time'] = order/dd_orders[0] *10*60
        #laser_off()
        run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False,
                        opt_on_poi=None,  # reference for sample shift (on stable, unique NV)
                        poi_list=None)
            
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.exception("auto run failed: {}.\n".format(stack_trace))
    finally:
        common.dd_order = save_dd_order



        
    
        
#"""

logger.info("Autopilot has landed.")
            
    



qmeas_list[-1]



OrderedDict([('experiment', 'deer_dd_tau'), ('name', 'deer_dd_tau'), ('microwave_frequency', 3129100000.0), ('microwave_amplitude', 0.23148148148148145), ('rabi_period', 7.55e-08), ('dd_type', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('dd_order', 10), ('num_of_points', 20), ('tau_start', -9.245e-08), ('tau_step', 9.245e-09), ('f_mw_2', '2744400000.0'), ('rabi_period_mw_2', '6.29e-08'), ('ampl_mw_2', '0.23148148148148145'), ('read_phase_deg', 0), ('tau1', 1e-06), ('measurement_time', 3000.0), ('optimize_time', 600), ('alternating', True), ('pulse_envelope', PulseEnvelopeType(sin_n))), ('tau2_rel_to_pi1', False), ('init_pix_on_1', 0.5), ('init_pix_on_2', 0.5), ('start_pix_on_1', 0), ('end_pix_on_1', 0.5), ('end_pix_on_2', 1.5), ('savetag', 'deer_dd_XY8-10_init=0_0_o=5_idx=4'), ('fit_experiment', 'sine'), ('fit_parameter', 'Period'), ('fit_edit_params', [<Parameter 'phase', 1.57, bounds=[-3.141592653589793:3.141592653589793]>, <Parameter 'amplitude', 0.05, bounds=[0

For different inits/ends

In [139]:

inits = [[0.5,0], [0.5,0],[0.5,0], [0.5,0], [0.5,0.5], [0.5,0.5],[0.5,0.5], [0.5,0.5]]#[::-1]
ends = [[0.5, 0], [0.5, 0], [0, 0.5], [0, 0.5],[0.5, 0], [0.5, 0], [0, 0.5], [0, 0.5]]
ends_phi = [0, 90, 0, 90, 0, 90, 0, 90]

inits = [[0.5,0.5],[0.5,0.5],  [0,0], [0,0],[0,0],[0,0]]#[::-1]
ends = [[0.5, 0], [0.5, 0],  [0, 0.5],[0, 0.5],[0.5,0],[0.5,0]]
ends_phi = [90, 90, 90,90, 0, 90, 0, 90]

inits = [[0,0],[0,0]]#[::-1]
ends = [[0.5,0],[0.5,0]]
ends_phi = [0,90]

inits += [[0.5,0], [0.5,0],[-0.5,0], [-0.5,0]]#[::-1]
ends += [[0, 0.5], [0, 0.5], [0, 0.5], [0, 0.5]]
ends_phi += [0, 90, 0, 90, 0, 90]

ends

[[0.5, 0], [0.5, 0], [0, 0.5], [0, 0.5], [0, 0.5], [0, 0.5]]

In [141]:

uglobals.abort.clear()
dd_cnot_tau = None


uglobals.abort.clear()


for idx, init in enumerate(inits):
    if uglobals.abort.is_set():
        break

    try:

        dd_cnot_tau_fix = common.dd_tau  # set 'None' for deer_dd_par_tau
   
        n_tau = 20
        tau2_pspacing = dd_cnot_tau_fix - common.rabi_period

        tau2_step = tau2_pspacing/(2*n_tau)
        tau2_start = -tau2_pspacing/2
        
        qmeas_list = setup_f_2q_nrep_auto(savetag_add=f'',
                                          dd_cnot_tau=dd_cnot_tau, deer_tau2_step=tau2_step, deer_tau2_start=tau2_start,
                                          deer_init=init, deer_end=ends[idx], deer_read_phase=ends_phi[idx])
        
        #laser_off()
        run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False,
                        opt_on_poi=None,  # reference for sample shift (on stable, unique NV)
                        poi_list=None)
        
            
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.exception("auto run failed: {}.\n".format(stack_trace))
    finally:
        pass



        
    
        
#"""

logger.info("Autopilot has landed.")
            
    



qmeas_list[0]



OrderedDict([('name', 'podmr_fine'), ('experiment', 'pulsedodmr'), ('rabi_period', 7.55e-08), ('freq_step', 1500000.0), ('num_of_points', 75), ('freq_start', 3073150000.0), ('microwave_amplitude', 0.23148148148148145), ('measurement_time', 120), ('optimize_time', None), ('fit_experiment', 'lor_dip'), ('pulse_envelope', PulseEnvelopeType(sin_n))), ('type', 'pulsedodmr'), ('freq_optimize_time', None), ('generate_new', True), ('meas_type', <function conventional_measurement at 0x000000B007555510>), ('meas_info', <function add_conventional_information at 0x000000B007563378>), ('fit_parameter', 'Position'), ('update_parameters', {'rabi_fine': 'microwave_frequency', 'deer_dd_tau': {'target_name': 'f_mw', 'func': "0.0+fit_result.result_str_dict['Position']['value']"}, 'nrep_cnot_2q': {'target_name': 'f_mw', 'func': "0.0+fit_result.result_str_dict['Position']['value']"}}), ('savetag', 'podmr_fine')])

In [112]:
qmeas_list[0]

OrderedDict([('experiment', 'deer_dd_tau'), ('name', 'deer_dd_tau'), ('microwave_frequency', 3129400000.0), ('microwave_amplitude', 0.23148148148148145), ('rabi_period', 7.55e-08), ('dd_type', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('dd_order', 2), ('num_of_points', 20), ('tau_start', -7.1225e-07), ('tau_step', 3.56125e-08), ('f_mw_2', '2744300000.0'), ('rabi_period_mw_2', '6.29e-08'), ('ampl_mw_2', '0.23148148148148145'), ('read_phase_deg', 90), ('tau1', 1.5e-06), ('measurement_time', 5400), ('optimize_time', 1200), ('alternating', True), ('pulse_envelope', PulseEnvelopeType(sin_n))), ('tau2_rel_to_pi1', False), ('start_pix_on_1', 0), ('init_pix_on_1', 0.5), ('init_pix_on_2', -0.5), ('end_pix_on_1', 0.5), ('read_pix_on_2', 0), ('end_pix_on_2', 1), ('savetag', 'deer_dd_XY8-2_init=0.5_-0.5_read=0.5_0_readph=90'), ('fit_experiment', 'sine'), ('fit_parameter', 'Period'), ('fit_edit_params', [<Parameter 'phase', 1.57, bounds=[-3.141592653589793:3.141592653589793]>,

## Ramsey power series with odmr refocus
Eg. to find the power that cancels left/right HFS lines.

In [83]:
def setup_odmr_ramsey_auto(savetag_add='', f_step_odmr_fine=1.0e6):
    
    # for pois=None, all experiment will run on the current confocal pos
    pois = None

    qmeas_list = []

    n_f_odmr_fine = 50
    # do odmr+rabi with higher power to drive all HFS lines
    ampl_manual = 0.09
    t_pi_manual = 71.0e-9/2
    p_scale_odmr = common.mw_amp/ampl_manual 
    
    qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=p_scale_odmr, f_step=f_step_odmr_fine*common.p_scale,
                               n_f=n_f_odmr_fine, t_pi_manual=t_pi_manual)
    qexp['type'] = 'pulsedodmr'
    qexp['name'] = 'podmr_fine'
    qexp['measurement_time'] = 4*60
    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None
    qexp['generate_new'] = True
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information
    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'lor_dip'
    qexp['fit_parameter'] = 'Position'
    qexp['update_parameters'] = {'rabi_fine': 'microwave_frequency',
                                 'ramsey': 'microwave_frequency',
                                  }
    qexp['savetag'] = (get_savetag(qexp) + savetag_add)
    qmeas_list.append(cp.deepcopy(qexp))

    
    # Rabi
    qexp = setup_rabi_params(tau_step=2*t_pi_manual/20, alternating=False, n_tau=75, p_scale=p_scale_odmr)
    qexp['type'] = qexp['experiment']
    qexp['name'] = 'rabi_fine'
    qexp['measurement_time'] = 4*60
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    qexp['fit_experiment'] = 'sine'
    qexp['fit_edit_params'] = [lmfit.Parameter('phase', value=1.57, vary=False),
                              ]
    qexp['fit_parameter'] = 'Period'
    #qexp['fit_experiment'] = 'sine_3x'
    #qexp['fit_edit_params'] = [lmfit.Parameter('s1_phase', value=1.57, vary=False),
    #                          lmfit.Parameter('s2_phase', value=1.57, vary=False),
    #                          lmfit.Parameter('s3_phase', value=1.57, vary=False)]
    #qexp['fit_parameter'] = 'Period 1'
    # Period 1 will be the oscillation with highest amplitude
    qexp['update_parameters'] = {
                                    'ramsey': {'target_name': 'rabi_period',
                                    'func': f"1/{p_scale_odmr}*fit_result.result_str_dict['Period']['value']"}
                                }
    qexp['savetag'] = (get_savetag(qexp) + savetag_add)
    qmeas_list.append(cp.deepcopy(qexp))
    
    
    # Ramsey
    qexp = setup_ramsey_params(tau_start, n_tau=50, tau_step=20e-9, alternating=True)
    qexp['type'] = qexp['experiment']
    qexp['name'] = 'ramsey'
    qexp['measurement_time'] = 10*60
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information
    
    qexp['savetag'] = (get_savetag(qexp) + savetag_add)
    qmeas_list.append(cp.deepcopy(qexp))
    

    return qmeas_list

0

0

In [76]:

uglobals.abort.clear()




#"""
    
    
laser_off()
try:
    qmeas_list = setup_odmr_ramsey_auto(savetag_add='_test')
    #run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False, poi_list=pois)
    run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False,
                        opt_on_poi=None, poi_list=None)
    #pass
except Exception as e:
    stack_trace = traceback.format_exc()
    logger.exception("auto run failed: {}.\n".format(stack_trace))
#"""

Power series.

In [80]:
pscales = (np.linspace(1.2, 0.8, 5))
pscales

array([1.2, 1.1, 1. , 0.9, 0.8])

In [84]:

uglobals.abort.clear()

p_start = 1

for i, p in enumerate(pscales):
    if uglobals.abort.is_set():
        break
    try:
        save_mw_amp, save_mw_amps_2 = common.mw_amp, cp.copy(common.mw_amps_2)
        save_p = common.p_scale
        common.mw_amp = p* common.mw_amp
        common.mw_amps_2[0] = p* common.mw_amps_2[0] 
        common.p_scale = p
        
        qmeas_list = qmeas_list = setup_odmr_ramsey_auto(savetag_add=f'_pscale={p:.2f}_idx={i}')
        
        #laser_off()
        run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False,
                        opt_on_poi=None,  # reference for sample shift (on stable, unique NV)
                        poi_list=None)
        
        
        #pass
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.exception("auto run failed: {}.\n".format(stack_trace))
    finally:
        common.mw_amp, common.mw_amps_2 = save_mw_amp, save_mw_amps_2 
        common.p_scale = save_p 
        
    
        
#"""

logger.info("Autopilot has landed.")
            
    



qmeas_list[1]



OrderedDict([('name', 'rabi_fine'), ('experiment', 'rabi'), ('rabi_period', 1.5235000000000001e-06), ('tau_step', 3.55e-09), ('tau_start', 0.0), ('alternating', False), ('leave_out_tau_idx', ''), ('microwave_frequency', 2615600000.0), ('num_of_points', 75), ('microwave_amplitude', 0.09), ('measurement_time', 240), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('pulse_envelope', PulseEnvelopeType(rectangle))), ('type', 'rabi'), ('meas_type', <function conventional_measurement at 0x0000007B0234D840>), ('meas_info', <function add_conventional_information at 0x0000007B0234D2F0>), ('fit_edit_params', [<Parameter 'phase', value=1.57 (fixed), bounds=[-inf:inf]>]), ('fit_parameter', 'Period'), ('update_parameters', {'Ramsey': {'target_name': 'rabi_period', 'func': "1/0.08555555555555557*fit_result.result_str_dict['Period']['value']"}}), ('savetag', 'rabi_fine_pscale=1.10_idx=1')])

## OLD auto logic

In [ ]:
# In order to stop the automized measurement insert pulsedmasterlogic.break_variable = True into manager

uglobals.abort.clear()
qmeas=OrderedDict()
# Either specify a list with points of interest or take all poi from the current ROI (use empty list)
#qmeas['list_pois'] = []
qmeas['list_pois'] = poimanagerlogic.poi_names[10:]
qmeas['generate_new'] = True
qmeas['no_optimize'] = True
logger.info("Autopilot is taking of for pois {}".format(qmeas['list_pois']))
# Create another dictionary for the experiments and parameters
autoexp=OrderedDict()
# Then specify all necessary experiment and parameters


# optional parameters 
qmeas['optimize_between_experiments'] = False


#Experiment1
# need to specify for the automatic update
f_step_odmr_fine=100e3
n_f_odmr_fine=100

#"""
experiment1 = 'pODMR'
qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=3000e3, n_f=150)

qexp['type'] = 'pulsedodmr'
qexp['measurement_time'] = 60
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_dip'
qexp['fit_parameter'] = 'Position'
qexp['update_parameters'] = {'Rabi': 'microwave_frequency',
                             'Hahn_Echo': 'microwave_frequency',
                             'Ramsey': 'microwave_frequency',
                             'pODMR_fine': {'target_name': 'freq_start',
                                            'func': f"_x_ - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"}
                            }
autoexp[experiment1] = copy.deepcopy(qexp)

#"""
#Experiment2
#"""

experiment2 = 'Rabi'
qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50)
qexp['type'] = 'rabi'
qexp['measurement_time'] = 60#2*60
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'sine'
qexp['fit_parameter'] = 'Period'
qexp['update_parameters'] = {'Ramsey': 'rabi_period',
                             'Hahn_Echo': 'rabi_period',
                             'pODMR_fine': 'rabi_period'}  # feed forware parameters

autoexp[experiment2] = copy.deepcopy(qexp)


#"""
#Experiment2
#"""
"""
experiment3 = 'Hahn_Echo'
#qexp = setup_hahn_params(tau_start=10e-9, n_tau=50, tau_step=0.2e-6, alternating=True)
#
#qexp = setup_hahn_params(tau_start=10e-9, n_tau=150, tau_step=125e-9, alternating=True)
qexp = setup_hahn_params(tau_start=10e-9, n_tau=75, tau_step=500e-9, alternating=True)
qexp['type'] = 'hahnecho'
qexp['measurement_time'] = 10*60
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'decay'
qexp['fit_parameter'] = 'Lifetime'

autoexp[experiment3] = copy.deepcopy(qexp)
"""



#Experiment4
#"""
experiment4 = 'Ramsey'
qexp = setup_ramsey_params(tau_start=50e-9, n_tau=200, tau_step=10e-9)  # res= 50 kHz 
qexp['type'] = 'ramsey'
qexp['measurement_time'] = 240
qexp['alternating'] = True
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
qexp['fit_experiment'] = 'sine_2x'
autoexp[experiment4] = copy.deepcopy(qexp)

#"""

"""
experiment3 = 'pODMR_fine'
qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=15, f_step=100e3, n_f=100)

qexp['type'] = 'pulsedodmr'
qexp['measurement_time'] = 15
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_n15'
qexp['fit_parameter'] = 'Position'
qexp['update_parameters'] = {'Rabi': 'microwave_frequency',
                             'pODMR_fine':'microwave_frequency',
                             'xy8': 'microwave_frequency',
                             'xy8_2': 'microwave_frequency'}

autoexp[experiment3] = copy.deepcopy(qexp)
"""
# create another dictionary for the parameters
"""
autoexp[experiment1]=OrderedDict()
autoexp[experiment1]['type'] = 'pulsed_odmr'
autoexp[experiment1]['name'] = experiment1
autoexp[experiment1]['rabi_period'] = common.rabi_period
autoexp[experiment1]['freq_start'] = common.mw_freq -25e6
autoexp[experiment1]['freq_step'] = 1e6
autoexp[experiment1]['num_of_points'] = 50
autoexp[experiment1]['microwave_amplitude'] = common.mw_amp
autoexp[experiment1]['measurement_time'] = 10
autoexp[experiment1]['optimize_time'] = None
autoexp[experiment1]['freq_optimize_time'] = None
autoexp[experiment1]['generate_new'] = False
autoexp[experiment1]['meas_type'] = conventional_measurement
autoexp[experiment1]['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
autoexp[experiment1]['fit_experiment'] = 'lor_neg'
autoexp[experiment1]['fit_parameter'] = 'Position'
autoexp[experiment1]['update_parameters'] = {'Rabi': 'microwave_frequency', 'xy8': 'microwave_frequency', 'xy8_2': 'microwave_frequency'}
"""

# broken to add experiments in loop
#autoexp[experiment1] = copy.deepcopy(qexp)
#"""

#Experiment2
"""
experiment2 = 'Rabi'
qexp = setup_rabi_params(tau_step=10e-9)
qexp['type'] = 'rabi'
qexp['measurement_time'] = 30
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'sine'
qexp['fit_parameter'] = 'Period'
qexp['update_parameters'] = {'ramsey': 'rabi_period'}  # feed forware parameters
autoexp[experiment2] = copy.deepcopy(qexp)

#Experiment3
experiment3 = 'ramsey'
qexp = setup_ramsey_params(tau_start=50e-9, n_tau=400, tau_step=50e-9)  # res= 50 kHz 
qexp['type'] = 'ramsey'
qexp['measurement_time'] = 120
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
qexp['fit_experiment'] = 'double_sine_decay'
autoexp[experiment3] = copy.deepcopy(qexp)


"""

laser_off()
do_automized_measurements(qm_dict=qmeas, autoexp=autoexp)




In [ ]:
poimanagerlogic.poi_names[-1]

In [ ]:
qmeas

# Randomized benchmarking (Old version)

In [ ]:
#data_dir = r"E:\Data\2023\01\20230127\rand_bench"
#fname = '20230127-1553-55_rb_seq_2q.json'

# This circuit for the 2 qubit system is in the near of the generator jupyter notebook
data_dir = r"C:\Software\qudi\user_scripts\Timo\own\rand_benchmark"
fname = '20230127-1553-55_rb_seq_2q.json'

with open(data_dir + "/" + fname) as f:
    qudi_gates = json.load(f)

data = parse_rbjson(qudi_gates)
0

In [ ]:
a = np.asarray([1,2])
a == []

In [ ]:
common = common.swap_nvs([1,2])

f_deer = [common.mw_freq_2[0]]
ampl_2 = common.mw_amps_2[0]
t_rabi_2 = common.rabi_period_2[0]
#t_rabi_2 = common.t_rabi_2_shaped[0]

dd_cnot_tau = 400e-9#-75.5e-9
dd_cnot_tau_fix = common.dd_tau  # set 'None' for deer_dd_par_tau
dd_order =  common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8


# load from manually defined gate list of lists
rots = [
        [TomoRotations.ux90_on_1, TomoRotations.ux90_on_2, TomoRotations.ux90_on_1, TomoRotations.ux90_on_2,
        TomoRotations.c2not1]]

# repetitions of cnots
rots, xticks = [[TomoRotations.none]], [0]
for idx in range(0, 8):
    n_cnots = 2*(idx+1)
    #n_cnots = (idx+1)
    rots.append([TomoRotations.c2not1]*n_cnots)
    xticks.append(n_cnots)


qmeas = setup_rand_benchmark_params(rotations=rots, x_max=1,
                                 f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                 tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
                                 tau_dd_fix=dd_cnot_tau_fix,
                                 alternating=True)



#qmeas['xticks'] = list_2_csv(xticks)

# load from (qiskit generated) file
data_dir =  os.path.abspath(savelogic.get_daily_directory())
#data_dir = r"E:\Data\2023\01\20230127\rand_bench"
#fname = '20230127-1553-55_rb_seq_2q.json'
data_dir = r"C:\Software\qudi\user_scripts\Timo\own\rand_benchmark"
#fname = '20230127-1553-55_rb_seq_2q.json'
#fname = '20230202-1815-55_rb_seq_2q_ncxfix.json'
fname = '20230206-0849-14_rb_seq_2q_ncxfix.json'
#fname = '20230213-1230-17_rb_seq_1q.json'
#qmeas = setup_rand_benchmark_params(rotations=data_dir + "/" + fname, x_max=5,
 #                                f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
  #                               tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
   #                              tau_dd_fix=dd_cnot_tau_fix,
    #                             alternating=True, xticks_ncliff=False)

#qmeas['t_idle'] = 25e-9
qmeas['ampl_idle_mult'] = 0.0
qmeas['measurement_time'] =60*60

qmeas['optimize_time'] = 30*60 # Previus optimization time was 15 minutes
#qmeas['optimize_func_toggle_pause'] = toggle_pg  # needs to be reworked for laser_on with awg

uglobals.abort.clear()
run_generic(qmeas)
qmeas

#common = common.swap_nvs([1,2])
len(qmeas['rotations'])
qmeas

In [ ]:
qmeas
#t_rabi_2
#print(pulsedmasterlogic.generate_methods.keys())
#print(pulsedmasterlogic.get_ensemble_info(pulsedmasterlogic.saved_pulse_block_ensembles[qmeas['name']])[0])
#print(pulsedmasterlogic.saved_pulse_block_ensembles[qmeas['name']])
#experiment = qmeas['experiment']
#print(pulsedmasterlogic.generate_predefined_sequence(experiment, qmeas.copy()))
#print(pulsedmasterlogic.get_ensemble_info)
#common.t_rabi_2_shaped[0]

In [ ]:
len([])

# Gate set Tomography 

In [ ]:
def setup_GST2Q_param(init_dict,meas_dict,gate_set_dict,
                             f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             tau_cnot=100e-9, dd_type_cnot=DDMethods.SE, dd_order=1,
                             tau_dd_fix=None,
                             alternating=True, charge_init=False):
    #  point_range: range of measured gates.
    
    qmeas=OrderedDict()
    
    is_multi_drive = False
    if len(f_mw_2) > 0:
        is_multi_drive = True
    
    # for awg8190 name must be < 32 chars incl ch extension
    qmeas['name'] = f'linear_gate_set_tom'  
    qmeas['experiment'] = 'linear_gate_set_tom'
    qmeas['savetag'] = f'linear_gate_set_tom'

    if charge_init:
        qmeas['name'] = 'linear_gate_tom_fci'
        qmeas['experiment'] = 'linear_gate_tom_fci'
        qmeas['sequence_mode'] = True
        qmeas = add_fci(qmeas)

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['microwave_frequency'] = common.mw_freq
    qmeas['rabi_period'] = common.rabi_period
    
    
    rotations = []
    qmeas['GST_process_key'] = [] # For looking the axis of the
    
    for i in meas_dict.keys():# After measurement
        for j in init_dict.keys(): # From state preparation
            for k in gate_set_dict.keys():
                key = '('+i+')'+'('+j+')'+'('+k+')' # (meas op)(gate)(init)
                qmeas['GST_process_key'].append(key)
                rotations.append([ init_dict[j], gate_set_dict[k],meas_dict[i]])
                
    if rotations:
        if type(rotations) == list:
            qmeas['rotations'] = list_2_csv(rotations, line_delimiter=";")
        else:
            raise ValueError("Don't understand given rotations.")
            
    else:
        # leaving will default init to "None" rotation
        pass
    
    qmeas['name'] = qmeas['name'] + '_2q' if len(f_mw_2)= 1 elif len(f_mw_2)= 0 qmeas['name'] + '_1q' else: qmeas['name'] + '_manyq'# If the second array is filled with one number
    
    qmeas['init_state_kwargs'] = init_dict   # todo, needed for cnot/ent state. Look for correct set
    #{'dd_type': DDMethods.CPMG}
    
    qmeas['cnot_kwargs'] = {'tau_dd_fix': tau_dd_fix}  
    
    
    qmeas['tau_cnot'] = tau_cnot
    qmeas['dd_type_cnot'] = dd_type_cnot
    qmeas['dd_order'] = dd_order
    qmeas['alternating'] = alternating
    
    if tau_cnot < 0 and tau_dd_fix == None:
        raise ValueError(f"Negative tau_cnot {tau_cnot} not allowed for cnot based on deer_dd_par!")
    
    if is_multi_drive:
        # currently drives 1,2 have same amplitude, respectively
        qmeas['f_mw_2'] = list_2_csv(f_mw_2)
        qmeas['rabi_period_mw_2'] = list_2_csv(rabi_period_mw_2)
        qmeas['ampl_mw_2'] = list_2_csv(ampl_mw_2)            
    

    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas = add_shaped(qmeas)
    

    return qmeas

## 1 Qubit Case

In [ ]:
# 1) Generate a dictionary with every Pulse Blocks for init state, meas state and gate set
init_state_dict = dict()
meas_dict  = dict()
gate_set_dict = dict()

total_dict = dict()

# Use the order of the Enum class of Timo to declare every initial state with the given number
init_state_list = [['00',TomoInit.none],
 ['0',TomoInit.ux180_on_2],
 ['1',TomoInit.ux180_on_1],
['+',TomoInit.uy90_on_2],
 ['-i',TomoInit.ux90_on_2]]

gate_set_list = [['x',TomoRotations.ux180_on_1], # Put more gates until the amount of gate set is 16
                ['y',TomoRotations.uy180_on_1],
                ['1',TomoRotations.none],
                ['+',TomoRotations.ux90_on_1]]
                #['',Tomo]]

meas_state_list = [['0',TomoInit.none], # Opposite pulse direction
 ['1',TomoInit.opp_ux180_on_1],
['+',TomoInit.opp_uy90_on_2],
 ['-i',TomoInit.opp_ux90_on_2]]


for h in meas_state_list: #Only for the measure state it's useful to inverse the pulse
    meas_dict[h[0]] = h[1]#tomo_init_elements(h[1])
    
for i in init_state_list: 
    init_state_dict[i[0]] = i[1]#tomo_init_elements(i[1])
    
for j in gate_set_list:
    gate_set_dict[j[0]] = j[1]#tomo_rotation_element(j[1])
    
total_dict['E'] = meas_dict # This key 'E' is for the measurement
total_dict['F'] = init_state_dict # This key 'F' is for the initial state
total_dict['G'] = gate_set_dict # This key 'G' is for the gate set

amp_mw2= common.mw_amps_2
rabi_period2 = common.t_rabi_2_rect

tau_cnot = common.dd_tau
dd_order = common.dd_order
dd_type = commom.

qmeas = setup_GST2Q_param(total_dict['F'],total_dict['E'],total_dict['G'],f_mw_2=[], ampl_mw_2=[amp_mw2], rabi_period_mw_2=[rabi_period2],
                             tau_cnot=tau_cnot, dd_type_cnot=DDMethods.SE, dd_order=dd_order,
                             tau_dd_fix=None,
                             alternating=True, charge_init=False)

qmeas['measurement_time'] = 1*60
#run_generic(qmeas)
qmeas

## 2 Qubit Case

In [ ]:
# 1) Generate a dictionary with every Pulse Blocks for init state, meas state and gate set
init_state_dict = dict()
meas_dict  = dict()
gate_set_dict = dict()

total_dict = dict()

# Use the order of the Enum class of Timo to declare every initial state with the given number
init_state_list = [['00',TomoInit.none],
 ['01',TomoInit.ux180_on_2],
 ['10',TomoInit.ux180_on_1],
 ['11',TomoInit.ux180_on_both],
 ['0+',TomoInit.uy90_on_2],
 ['0-i',TomoInit.ux90_on_1],
 ['1+',TomoInit.uy90_on_2],
 ['1-i',TomoInit.ux180_on_1_ux90_on_2],
 ['+0',TomoInit.uy90_on_1],
 ['+1',TomoInit.uy90_on_1_ux180_on_2],
 ['++',TomoInit.uy90_on_1_uy90_on_2],
 ['+-i',TomoInit.uy90_on_1_ux90_on_2],
 ['-i0',TomoInit.ux90_on_1],
 ['-i1',TomoInit.ux90_on_1_ux180_on_2],
 ['-i+',TomoInit.ux90_on_1_uy90_on_2],
 ['-i-i',TomoInit.ux90_on_1_ux90_on_2]]

gate_set_list = [['c1not2',TomoRotations.c1not2], # Put more gates until the amount of gate set is 16
                ['c2not1',TomoRotations.c2not1],
                ['1',TomoRotations.none]]
                #['',Tomo]]

meas_state_list = [['00',TomoInit.none], # ToDo: Put the Initial states in the multi-nv method
 ['01',TomoInit.opp_ux180_on_2],
 ['10',TomoInit.opp_ux180_on_1],
 ['11',TomoInit.opp_ux180_on_both],
 ['0+',TomoInit.opp_uy90_on_2], # For obtaining the measurement state the pulse should go in other direction. phase of pulse should be changed
 ['0-i',TomoInit.opp_ux90_on_1],# The phase of the 180 deg rotation (pi-pulse) will be rotated too, according to pulse errors
 ['1+',TomoInit.opp_uy90_on_2],
 ['1-i',TomoInit.opp_ux180_on_1_opp_ux90_on_2],
 ['+0',TomoInit.opp_uy90_on_1],
 ['+1',TomoInit.opp_uy90_on_1_opp_ux180_on_2],
 ['++',TomoInit.opp_uy90_on_1_opp_uy90_on_2],
 ['+-i',TomoInit.opp_uy90_on_1_opp_ux90_on_2],
 ['-i0',TomoInit.opp_ux90_on_1],
 ['-i1',TomoInit.opp_ux90_on_1_opp_ux180_on_2],
 ['-i+',TomoInit.opp_ux90_on_1_opp_uy90_on_2],
 ['-i-i',TomoInit.opp_ux90_on_1_opp_ux90_on_2]]


for h in meas_state_list: #Only for the measure state it's useful to inverse the pulse
    meas_dict[h[0]] = h[1]#tomo_init_elements(h[1])
    
for i in init_state_list: 
    init_state_dict[i[0]] = i[1]#tomo_init_elements(i[1])
    
for j in gate_set_list:
    gate_set_dict[j[0]] = j[1]#tomo_rotation_element(j[1])
    
total_dict['E'] = meas_dict # This key 'E' is for the measurement
total_dict['F'] = init_state_dict # This key 'F' is for the initial state
total_dict['G'] = gate_set_dict # This key 'G' is for the gate set

# From the concatenation of the init, gate and the pulse sequence the function for the setup will be similar to Timo's method setup_randomized

In [ ]:
qmeas = setup_GST2Q_param(total_dict['F'],total_dict['E'],total_dict['G'],f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             tau_cnot=100e-9, dd_type_cnot=DDMethods.SE, dd_order=1,
                             tau_dd_fix=None,
                             alternating=True, charge_init=False,point_range=[0,10])



In [4]:
type(TomoInit.none) == TomoInit

True